In [1695]:
escher_map2 = None
with open('escher/maps/Saccharomyces cerevisiae/iMM904.Central carbon metabolism.json', "r") as fh:
    jsonS = fh.read()
    escher_map2 = json.loads(jsonS)
    

In [56]:
escher_model = None
with open('escher/models/Saccharomyces cerevisiae/iMM904.json', "r") as fh:
    jsonS = fh.read()
    escher_model = json.loads(jsonS)
    

In [2718]:
def add_prim_nodes(prim_react, prim_prod, orient, cc_coords, reaction_length, midmark, layout):
    prim_prod['x'] = cc_coords[prim_prod['bigg_id']]['x']
    prim_prod['y'] = cc_coords[prim_prod['bigg_id']]['y']
    prim_prod['label_x'] = prim_prod['x'] - 30
    prim_prod['label_y'] = prim_prod['y'] - 30
    for n in layout['nodes']:
        if layout['nodes'][n]['node_type'] == 'metabolite':
            while (math.cos(orient) * reaction_length) + prim_prod['x'] == layout['nodes'][n]['x'] and  - (math.sin(orient) * reaction_length) + prim_prod['y'] == layout['nodes'][n]['y']:
                orient += math.pi / 2
    prim_react['x'] = (math.cos(orient) * reaction_length) + prim_prod['x'] 
    prim_react['y'] = - (math.sin(orient) * reaction_length) + prim_prod['y'] 
    prim_react['label_x'] = (math.cos(orient) * reaction_length) + prim_prod['label_x']
    prim_react['label_y'] = - (math.sin(orient) * reaction_length) + prim_prod['label_y']
    midmark['x'] = prim_react['x'] - (math.cos(orient) * (reaction_length / 2))   
    midmark['y'] = prim_react['y'] + (math.sin(orient) * (reaction_length / 2)) 
    return orient


In [4409]:
def add_segment(layout, to_node_id, from_node_id, maxR, maxS, midmark, to_node, from_node, top, orient, n):
    dist1 = 50
    dist2 = 150
    if to_node['node_type'] == 'metabolite':
        if not(to_node['node_is_primary']):
            dist1 = 50 - (2 * n)
            dist2 = 150 - (2 * n)
    if from_node['node_type'] == 'metabolite':
        if not(from_node['node_is_primary']):
            dist1 = 50 - (2 * n)
            dist2 = 150 - (2 * n)
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)] = {}
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['to_node_id'] = to_node_id
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['from_node_id'] = from_node_id
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2'] = {}
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1'] = {}
    if to_node['node_type'] == 'midmarker' and from_node['node_type'] == 'midmarker':
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2'] = None
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1'] = None
    else:
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1']['y'] = ops[not(top)](midmark['y'], dist1 * math.sin(orient))
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2']['y'] = ops[not(top)](midmark['y'], dist2 * math.sin(orient))
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1']['x'] = ops[top](midmark['x'], dist1 * math.cos(orient))
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2']['x'] = ops[top](midmark['x'], dist2 * math.cos(orient))
        
    

In [4144]:
stoich3['5'] = {'cpd00117': -1, 'cpd021829': 1}

In [3678]:
def get_nodes(stoich, rxn_numb):
    prim_node_ids = list()
    second_node_ids = list()
    node_ids2 = list()
    
    for k in stoich[str(rxn_numb)]:
        node_ids2.append(k)
      
    for k in range(0, len(node_ids2)):
        if stoich[str(rxn_numb)][node_ids2[k]] < 0:
            prim_node_ids.append(node_ids2.pop(k))
            break
            
    for k in range(0, len(node_ids2)):
        if stoich[str(rxn_numb)][node_ids2[k]] > 0:
            prim_node_ids.append(node_ids2.pop(k))
            break
            
    for k in node_ids2:
        second_node_ids.append(k)
        
    return prim_node_ids, second_node_ids
    

In [4131]:
def get_max(layout):
    max = 0
    for n in layout['nodes']:
        if int(n) > max:
            max = int(n)
    return max

In [4324]:
def add_curved_segment(layout, to_node_id, from_node_id, to_node, from_node, orient, maxR):
    maxS = 0
    for k in layout['reactions']:
        for j in layout['reactions'][k]['segments']:
            if int(j) > maxS:
                maxS = int(j)

    
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)] = {}
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['to_node_id'] = to_node_id
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['from_node_id'] = from_node_id
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2'] = {}
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1'] = {}
    if to_node['node_type'] == 'midmarker':
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2']['x'] = to_node['x'] - 100 * math.cos(orient)
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2']['y'] = to_node['y'] + 100 * math.sin(orient)
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1']['y'] = from_node['y'] - 100 * math.sin(orient)
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1']['x'] = from_node['x'] + 100 * math.cos(orient)
    else:
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2']['x'] = to_node['x'] + 100 * math.cos(math.pi)
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2']['y'] = to_node['y'] 
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1']['y'] = from_node['y'] - 100 * math.sin(orient)
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1']['x'] = from_node['x'] + 100 * math.cos(orient)

In [4482]:
def add_branches(layout, prim_react, prim_react_id, prim_nodes, numb, nodes_in_layout, reaction_length, max, second_react_nodes, second_prod_nodes, rxn_numb, cc):
    maxS = 0
    for k in layout['reactions']:
        for j in layout['reactions'][k]['segments']:
            if int(j) > maxS:
                maxS = int(j)
   
    x = 0
    for key, n in prim_nodes.items():
        if n[0][0] == prim_react['bigg_id'] and n[1][0] not in nodes_in_layout:
            max = get_max(layout)
            prim_prod = {'node_is_primary': True,
                    'name': 'compound1',
                    'label_x': ((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5 + prim_react['x'],
                    'node_type': 'metabolite',
                    'y': prim_react['y'],
                    'x': ((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5 + prim_react['x'],
                    'label_y': prim_react['y'],
                    'bigg_id': n[1][0]
                     }
            maxR = 0
            for k in layout['reactions'].keys():
                k = int(k)
                if k > maxR:
                    maxR = k
            
            orient = math.atan((x * reaction_length) / (reaction_length))
            rotate_node(prim_react['x'], prim_react['y'], prim_prod, orient)
            prim_prod['label_x'] = prim_prod['x']
            prim_prod['label_y'] = prim_prod['y'] + 35
            if x != 0:
                midmark = {'y': prim_react['y'] - x * 150, 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'], 'node_type': 'midmarker'}
                rotate_node(prim_react['x'], prim_react['y'], midmark, orient)
                midmark_top = {'y': prim_react['y'] - x * 150, 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'] + 50, 'node_type': 'midmarker'}
                rotate_node(prim_react['x'], prim_react['y'], midmark_top, orient)
                midmark_bottom = {'y': prim_react['y'] - x * 150, 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'] - 50, 'node_type': 'midmarker'}
                rotate_node(prim_react['x'], prim_react['y'], midmark_bottom, orient)
            else:
                midmark = {'y': prim_react['y'], 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'], 'node_type': 'midmarker'}
                midmark_top = {'y': prim_react['y'], 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'] + 50, 'node_type': 'midmarker'}
                midmark_bottom = {'y': prim_react['y'], 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'] - 50, 'node_type': 'midmarker'}          
            layout['reactions'][str(maxR + 1)] = {'name': rxn_numb + 1,
              'bigg_id': 1,
              'segments': {},
              'genes': [],
              'reversibility': False,
              'metabolites': [],
              'label_x': midmark['x'] - 30 * math.sin(orient),
              'label_y': midmark['y'] - 30 * math.cos(orient),
              'gene_reaction_rule': ''}
            layout['nodes'][str(max + 1)] = prim_prod
            layout['nodes'][str(max + 2)] = midmark
            layout['nodes'][str(max + 3)] = midmark_top 
            layout['nodes'][str(max + 4)] = midmark_bottom
            prim_prod_id = max + 1
            midmark_id = max + 2
            midmark_top_id = max + 3
            midmark_bottom_id = max + 4
            second_react = second_react_nodes[key]
            second_prod = second_prod_nodes[key]
            add_curved_segment(layout, prim_prod_id, midmark_top_id, prim_prod, midmark_top, orient, maxR)
            add_curved_segment(layout, midmark_bottom_id, prim_react_id, midmark_bottom, prim_react, orient, maxR)
            maxes = add_secondary_branched(second_react, second_prod, orient, (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'], prim_react['y'] - x * 150, prim_react['x'], prim_react['y'], layout, stoich, max + 3, max + 4, max + 2, midmark_top, midmark_bottom, midmark, maxR, False)
            maxS = maxes[0]
            max = maxes[1]
            add_segment(layout, midmark_id, midmark_bottom_id, maxR, maxS + 1, midmark, midmark, midmark_bottom, True, orient, 1)
            add_segment(layout, midmark_top_id, midmark_id, maxR, maxS + 2, midmark, midmark_top, midmark, True, orient, 1)
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': 1, 'bigg_id': prim_prod['bigg_id']})                        
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': prim_react['bigg_id']}) 
            maxS += 2
            max += 4
            x += 1
            prim_nodes2 = copy.deepcopy(prim_nodes)
            del prim_nodes2[key]
            add_reaction2(layout, prim_prod, max, prim_nodes2, prim_prod['x'] + 500, prim_prod['y'], reaction_length, 0, rxn_numb + 1, cc, nodes_in_layout, second_react_nodes, second_prod_nodes, prim_prod_id)
            rxn_numb += 1
        if n[1][0] == prim_react['bigg_id'] and n[0][0] not in nodes_in_layout:
            max = get_max(layout)            
            prim_prod = {'node_is_primary': True,
                    'name': 'compound1',
                    'label_x': ((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5 + prim_react['x'],
                    'node_type': 'metabolite',
                    'y': prim_react['y'],
                    'x': ((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5 + prim_react['x'],
                    'label_y': prim_react['y'],
                    'bigg_id': n[0][0]
                     }
            maxR = 0
            for k in layout['reactions'].keys():
                k = int(k)
                if k > maxR:
                    maxR = k
            orient = math.atan((x * reaction_length) / (reaction_length))
            rotate_node(prim_react['x'], prim_react['y'], prim_prod, orient)
            prim_prod['label_x'] = prim_prod['x']
            prim_prod['label_y'] = prim_prod['y'] + 35
            if x != 0:
                midmark = {'y': prim_react['y'] - x * 150, 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'], 'node_type': 'midmarker'}
                rotate_node(prim_react['x'], prim_react['y'], midmark, orient)
                midmark_top = {'y': prim_react['y'] - x * 150, 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'] + 50, 'node_type': 'midmarker'}
                rotate_node(prim_react['x'], prim_react['y'], midmark_top, orient)
                midmark_bottom = {'y': prim_react['y'] - x * 150, 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'] - 50, 'node_type': 'midmarker'}
                rotate_node(prim_react['x'], prim_react['y'], midmark_bottom, orient)
            else:
                midmark = {'y': prim_react['y'], 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'], 'node_type': 'midmarker'}
                midmark_top = {'y': prim_react['y'], 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'] + 50, 'node_type': 'midmarker'}
                midmark_bottom = {'y': prim_react['y'], 'x': (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'] - 50, 'node_type': 'midmarker'}                      
            layout['reactions'][str(maxR + 1)] = {'name': rxn_numb + 1,
              'bigg_id': 1,
              'segments': {},
              'genes': [],
              'reversibility': False,
              'metabolites': [],
              'label_x': midmark['x'] - 30 * math.sin(orient),
              'label_y': midmark['y'] - 30 * math.cos(orient),
              'gene_reaction_rule': ''}
            layout['nodes'][str(max + 1)] = prim_prod
            layout['nodes'][str(max + 2)] = midmark
            layout['nodes'][str(max + 3)] = midmark_top 
            layout['nodes'][str(max + 4)] = midmark_bottom
            prim_prod_id = max + 1
            midmark_id = max + 2
            midmark_top_id = max + 3
            midmark_bottom_id = max + 4
            second_react = second_react_nodes[key]
            second_prod = second_prod_nodes[key]
            add_curved_segment(layout, prim_prod_id, midmark_top_id, prim_prod, midmark_top, orient, maxR)
            add_curved_segment(layout, midmark_bottom_id, prim_react_id, midmark_bottom, prim_react, orient, maxR)
            maxes = add_secondary_branched(second_react, second_prod, orient, (((reaction_length ** 2) + (x * reaction_length) ** 2) ** 0.5) / 2 + prim_react['x'], prim_react['y'] - x * 150, prim_react['x'], prim_react['y'], layout, stoich, max + 3, max + 4, max + 2, midmark_top, midmark_bottom, midmark, maxR, True)
            maxS = maxes[0]
            max = maxes[1]
            add_segment(layout, midmark_id, midmark_bottom_id, maxR, maxS + 1, midmark, midmark, midmark_bottom, True, orient, 1)
            add_segment(layout, midmark_top_id, midmark_id, maxR, maxS + 2, midmark, midmark_top, midmark, True, orient, 1)
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': prim_prod['bigg_id']})             
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': 1, 'bigg_id': prim_react['bigg_id']})             
            maxS += 2
            max += 4
            x += 1
            prim_nodes2 = copy.deepcopy(prim_nodes)
            del prim_nodes2[key]
            add_reaction2(layout, prim_prod, max, prim_nodes2, prim_prod['x'] + 500, prim_prod['y'], reaction_length, 0, rxn_numb + 1, cc, nodes_in_layout, second_react_nodes, second_prod_nodes, prim_prod_id)
            rxn_numb += 1
    return max

In [4042]:
def rotate_node(x, y, node, orient):
    qx = x + math.cos(orient) * (node['x'] - x) - math.sin(orient) * - (node['y'] - y)
    qy = y - (math.sin(orient) * (node['x'] - x) + math.cos(orient) * - (node['y'] - y))
    node['x'] = qx
    node['y'] = qy


In [3526]:
def curve_reaction(layout):
    prim_react = {'node_is_primary': True,
                    'name': 'compound1',
                    'label_x': 5000,
                    'node_type': 'metabolite',
                    'y': 5000,
                    'x': 5000,
                    'label_y': 5000,
                    'bigg_id': 'compound1'
                     }
    prim_prod = {'node_is_primary': True,
                    'name': 'compound1',
                    'label_x': 5500,
                    'node_type': 'metabolite',
                    'y': 5000,
                    'x': 5500,
                    'label_y': 5000,
                    'bigg_id': 'compound1'
                     }
    midmark = {'y': 4850, 'x': 5250, 'node_type': 'midmarker'}
    midmark_top = {'y': 4850, 'x': 5300, 'node_type': 'midmarker'}
    midmark_bottom = {'y': 4850, 'x': 5200, 'node_type': 'midmarker'}

    layout['nodes']['0'] = prim_react
    layout['nodes']['1'] = prim_prod
    layout['nodes']['2'] = midmark
    layout['nodes']['3'] = midmark_top
    layout['nodes']['4'] = midmark_bottom
    add_curved_segment(layout, 4, 0, midmark_bottom, prim_react)
    add_curved_segment(layout, 1, 3, midmark_top, prim_prod)
    add_segment(layout, 2, 4, 1, 3, midmark, midmark, midmark_bottom, True, 0, 1)
    add_segment(layout, 3, 2, 1, 4, midmark, midmark_top, midmark, True, 0, 1)


In [4110]:
def rotate(x, y, x1, y1, orient):
    orient = - orient
    qx = x + math.cos(orient) * (x1 - x) - math.sin(orient) * (y1 - y)
    qy = y + (math.sin(orient) * (x1 - x) + math.cos(orient) * (y1 - y))
    return qx, qy

In [4259]:
def generate_coords(func, n, x1, x2, x, y, ox, oy, orient, prod, cap):
    a = ox
    coords = []
    for pos_x in range(x1, x2 - cap, int((x2 - x1) / n)):
        if prod:
            a -= 500
        coords.append([rotate(ox, oy, pos_x + x, func(pos_x) + y, orient)[0], rotate(ox, oy, pos_x + x, func(pos_x) + y, orient)[1]])
        if prod:
            a += 500
    return coords

In [4486]:
def add_secondary_branched(second_react, second_prod, orient, x_coord, y_coord, ox, oy, layout, stoich, midmark_top_id, midmark_bottom_id, midmark_id, midmark_top, midmark_bottom, midmark, maxR, prim):
    react_coords = []
    react_nodes = []
    prod_coords = []
    prod_nodes = []
    for n in second_react:
        react_nodes.append(n[0])
    for n in second_prod:
        prod_nodes.append(n[0])
            
    max = 0
    for n in layout['nodes']:
        if int(n) > max:
            max = int(n)
    maxS = 0
    for k in layout['reactions']:
        for j in layout['reactions'][k]['segments']:
            if int(j) > maxS:
                maxS = int(j)
                
    if prim:
        temp = []
        temp = copy.deepcopy(react_nodes)
        react_nodes = copy.deepcopy(prod_nodes)
        prod_nodes = copy.deepcopy(temp)
        
    if len(react_nodes) > 0: 
        react_coords = generate_coords(lambda x :  pow(x, 2) / 250 , len(react_nodes), -100, -260, x_coord, y_coord, ox, oy, orient, False, -1)
    for c in range(0, len(react_coords)):
        secondary = {'node_is_primary': False,
                      'name': 'compound1',
                      'label_x': 0,
                      'node_type': 'metabolite',
                      'y': 0,
                      'x': 0,
                      'bigg_id': '',
                      'label_y': 0
                     }
        secondary['x'] = react_coords[c][0]
        secondary['label_x'] = react_coords[c][0] - 50
        secondary['y'] = react_coords[c][1]
        secondary['label_y'] = react_coords[c][1] + 30
        secondary['bigg_id'] = react_nodes[c]
        layout['nodes'][str(max + 1)] = secondary
        add_secondary_segment(layout, str(max + 1), midmark_bottom_id, maxR, maxS, midmark, secondary, midmark_bottom, True, orient, c)
        if prim:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': 1, 'bigg_id': react_nodes[c]})
        else:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': react_nodes[c]})

        max += 1
        maxS += 1
        
    if len(prod_nodes) > 0:         
        prod_coords = generate_coords(lambda x : - pow(x, 2) / 250, len(prod_nodes), 100, 260, x_coord, y_coord, ox, oy, orient, True, 1)
    for c in range(0, len(prod_coords)):
        secondary = {'node_is_primary': False,
                      'name': 'compound1',
                      'label_x': 0,
                      'node_type': 'metabolite',
                      'y': 0,
                      'x': 0,
                      'bigg_id': '',
                      'label_y': 0
                     }
        secondary['x'] = prod_coords[c][0]
        secondary['label_x'] = prod_coords[c][0] + 50
        secondary['y'] = prod_coords[c][1]
        secondary['label_y'] = prod_coords[c][1] - 30
        secondary['bigg_id'] = prod_nodes[c]
        layout['nodes'][str(max + 1)] = secondary
        add_secondary_segment(layout, str(max + 1), midmark_top_id, maxR, maxS, midmark, secondary, midmark_top, False, orient, c)
        if prim:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': prod_nodes[c]})
        else:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': 1, 'bigg_id': prod_nodes[c]})

        max += 1
        maxS += 1
    
    return maxS, max

In [4283]:
stoich4 = {'0': {'cpd02611': 1, 'cpd00037': -1, 'cpd00062': -1, 'cpd00012': 1},
 '1': {'cpd00288': -1,
  'cpd00293': 1,
  'cpd00022': -1,
  'cpd00010': 1,
  'cpd00067': 1},
 '2': {'cpd00773': -1,
  'cpd02820': 1,
  'cpd00006': -1,
  'cpd00005': 1,
  'cpd00067': 1},
 '3': {'cpd03671': 1,
  'cpd02611': -1,
  'cpd00022': -1,
  'cpd00010': 1,
  'cpd00067': 1},
 '4': {'cpd00037': 1, 'cpd02820': -1, 'cpd00061': -1, 'cpd00009': 1},
 '5': {'cpd03671': -1, 'cpd00288': 1},
 '6': {'cpd19035': 1, 'cpd00288': -1, 'cpd00053': -1, 'cpd00023': 1}}

In [4488]:
stoich = stoich5
layout['nodes'] = {}
layout['reactions'] = {}
add_reactions2(layout, stoich, 2000, 5000, 500, 0, 'cpd00216')
#curve_reaction(layout)
map_json_str = json.dumps(escher_map)
builder = escher.Builder(map_json=map_json_str)
builder.display_in_notebook()

In [4480]:
layout['nodes'] = {}
layout['reactions'] = {}
add_reactions2(layout, stoich, 4500, 5000, 500, 0, 'cpd00123')
#curve_reaction(layout)
with open('escher/maps/Saccharomyces cerevisiae/iMM904.Central carbon metabolism2.json', 'w') as fh:
    fh.write(json.dumps(escher_map))

KeyError: 'cpd00123'

In [4437]:
def add_secondary_segment(layout, to_node_id, from_node_id, maxR, maxS, midmark, to_node, from_node, top, orient, n):
    dist1 = 50
    dist2 = 150
    if to_node['node_type'] == 'metabolite':
        if not(to_node['node_is_primary']):
            dist1 = 50 - (2 * n)
            dist2 = 150 - (2 * n)
    if from_node['node_type'] == 'metabolite':
        if not(from_node['node_is_primary']):
            dist1 = 50 - (2 * n)
            dist2 = 150 - (2 * n)
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)] = {}
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['to_node_id'] = to_node_id
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['from_node_id'] = from_node_id
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2'] = {}
    layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1'] = {}
    if to_node['node_type'] == 'midmarker' and from_node['node_type'] == 'midmarker':
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2'] = None
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1'] = None
    else:
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1']['y'] = ops[not(top)](midmark['y'], dist1 * math.sin(orient))
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2']['y'] = ops[not(top)](midmark['y'], dist1 * math.sin(orient))
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b1']['x'] = to_node['x']
        layout['reactions'][str(maxR + 1)]['segments'][str(maxS + 1)]['b2']['x'] = to_node['x']
        
    

In [4278]:
stoich6 = {'0': {'cpd01716': -1, 'cpd00197': 1, 'cpd00001': 1},
 '1': {'cpd00699': -1, 'cpd01716': 1, 'cpd00001': 1},
 '2': {'cpd00248': 1, 'cpd00699': -1, 'cpd00097': -1, 'cpd00986': 1},
 '3': {'cpd00383': 1,
  'cpd01716': -1,
  'cpd00006': -1,
  'cpd00005': 1,
  'cpd00067': 1},
 '4': {'cpd00383': 1, 'cpd01716': -1, 'cpd00097': -1, 'cpd00986': 1}}

In [4203]:
stoich3 = {'0': {'cpd00035': -1, 'cpd00117': 1},
 '1': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1},
 '2': {'cpd00156': 1, 'cpd00123': -1, 'cpd00024': -1, 'cpd00023': 1},
 '3': {'cpd00084': -1, 'cpd00035': 1, 'cpd14547': -1, 'cpd14548': 1},
 '4': {'cpd00035': -1, 'cpd00024': -1, 'cpd00020': 1, 'cpd00023': 1},
 '5': {'cpd00084': -1, 'cpd021829': 1}}

In [4284]:
stoich2 = {'6': {'cpd00078': -1, 'cpd00279': 1, 'cpd00142': -1, 'cpd00036': 1},
 '2': {'cpd00938': -1, 'cpd02255': 1, 'cpd00067': -1, 'cpd00011': 1},
 '3': {'cpd02255': 1, 'cpd00630': -1, 'cpd00001': -1, 'cpd00067': 1},
 '4': {'cpd01507': 1, 'cpd00630': -1, 'cpd00078': 1, 'cpd00036': 1},
 '5': {'cpd00078': 1, 'cpd01507': -1, 'cpd00022': -1, 'cpd00010': 1},
 '0': {'cpd00197': -1, 'cpd00856': 1, 'cpd00007': -1, 'cpd00067': 2},
 '1': {'cpd00938': 1, 'cpd00856': -1, 'cpd00067': 1}}

In [4487]:
def add_secondary_nodes2(second_react, second_prod, orient, x_coord, y_coord, ox, oy, layout, stoich, midmark_top_id, midmark_bottom_id, midmark_id, midmark_top, midmark_bottom, midmark, maxR, prim):
    react_coords = []
    react_nodes = []
    prod_coords = []
    prod_nodes = []
    for n in second_react:
        react_nodes.append(n[0])
    for n in second_prod:
        prod_nodes.append(n[0])
            
    max = 0
    for n in layout['nodes']:
        if int(n) > max:
            max = int(n)
    maxS = 0
    for k in layout['reactions']:
        for j in layout['reactions'][k]['segments']:
            if int(j) > maxS:
                maxS = int(j)
    
    if prim:
        temp = []
        temp = copy.deepcopy(react_nodes)
        react_nodes = copy.deepcopy(prod_nodes)
        prod_nodes = copy.deepcopy(temp)

        
        
    if len(react_nodes) > 0: 
        react_coords = generate_coords(lambda x : - pow(x, 2) / 75, len(react_nodes), 60, 160, x_coord, y_coord, ox, oy, orient, False, 1)
    for c in range(0, len(react_coords)):
        secondary = {'node_is_primary': False,
                      'name': 'compound1',
                      'label_x': 0,
                      'node_type': 'metabolite',
                      'y': 0,
                      'x': 0,
                      'bigg_id': '',
                      'label_y': 0
                     }
        secondary['x'] = react_coords[c][0]
        secondary['label_x'] = react_coords[c][0] - 80
        secondary['y'] = react_coords[c][1]
        secondary['label_y'] = react_coords[c][1] + 30       
        secondary['bigg_id'] = react_nodes[c]  
        layout['nodes'][str(max + 1)] = secondary
        add_segment(layout, str(max + 1), midmark_top_id, maxR, maxS, midmark, secondary, midmark_top, True, orient, c)
        if prim:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': 1, 'bigg_id': react_nodes[c]})
        else:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': react_nodes[c]})

        max += 1
        maxS += 1
        
    if len(prod_nodes) > 0:         
        prod_coords = generate_coords(lambda x : - pow(x, 2) / 75, len(prod_nodes), -60, -160, x_coord + 500, y_coord, ox, oy, orient, True, -1)
    for c in range(0, len(prod_coords)):
        secondary = {'node_is_primary': False,
                      'name': 'compound1',
                      'label_x': 0,
                      'node_type': 'metabolite',
                      'y': 0,
                      'x': 0,
                      'bigg_id': '',
                      'label_y': 0
                     }
        secondary['x'] = prod_coords[c][0]
        secondary['label_x'] = prod_coords[c][0] - 20
        secondary['y'] = prod_coords[c][1]
        secondary['label_y'] = prod_coords[c][1] + 30
        secondary['bigg_id'] = prod_nodes[c]
        layout['nodes'][str(max + 1)] = secondary
        add_segment(layout, str(max + 1), midmark_bottom_id, maxR, maxS, midmark, secondary, midmark_bottom, False, orient, c)
        if prim:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': prod_nodes[c]})
        else:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': 1, 'bigg_id': prod_nodes[c]})
        max += 1
        maxS += 1
    
    return maxS, max
    

       


In [4452]:
def add_reaction2(layout, prim_react, prim_react_id, prim_nodes, x, y, reaction_length, orient, rxn_numb, cc, nodes_in_layout, second_react_nodes, second_prod_nodes, prim_id):
    if len(prim_nodes) == 0:
        return
    max = 0
    for k in layout['nodes'].keys():
        k = int(k)
        if k > max:
            max = k
    if cc[prim_react['bigg_id']] > 2:
        max = add_branches(layout, prim_react, prim_react_id, prim_nodes, 4, nodes_in_layout, reaction_length, max, second_react_nodes, second_prod_nodes, rxn_numb, cc)
    else:
        prim_prod = {'node_is_primary': True,
                        'name': 'compound1',
                        'label_x': 0,
                        'node_type': 'metabolite',
                        'y': 0,
                        'x': 0,
                        'label_y': 0
                         }
        found = False
        coefficient = 0
        for key,reaction in prim_nodes.items():
            if reaction[0][0] == prim_react['bigg_id']:
                prim_prod['bigg_id'] = reaction[1][0]
                second_react = second_react_nodes[key]
                second_prod = second_prod_nodes[key]
                del prim_nodes[key]
                found = True
                coefficient = reaction[1][1]
                break
        
        if not(found):
            for key,reaction in prim_nodes.items():
                if reaction[1][0] == prim_react['bigg_id']:
                    prim_prod['bigg_id'] = reaction[0][0]
                    second_react = second_react_nodes[key]
                    second_prod = second_prod_nodes[key]
                    del prim_nodes[key]
                    found = True
                    coefficient = reaction[0][1]
                    break
                
        
        if not(found):
            return
            
        
        prim_prod['x'] = prim_react['x'] + reaction_length
        prim_prod['y'] = prim_react['y'] 
        prim_prod['label_x'] = prim_react['label_x'] + reaction_length
        prim_prod['label_y'] = prim_react['label_y']

        
        # get largest key for reactions
        maxR = 0
        for k in layout['reactions'].keys():
            k = int(k)
            if k > maxR:
                maxR = k

        # get largest key for segments
        maxS = 0
        for k in layout['reactions'].keys():
            for j in layout['reactions'][k]['segments'].keys():
                if int(j) > maxS:
                    maxS = int(j)
        midmark = {'y': prim_react['y'], 'x': prim_react['x'] + (reaction_length / 2), 'node_type': 'midmarker'}
        midmark_top = copy.deepcopy(midmark)
        midmark_bottom = copy.deepcopy(midmark)   
        midmark_top['x'] = midmark['x'] - 25 * (math.cos(orient))
        midmark_bottom['x'] = midmark['x'] + 25 * (math.cos(orient))
        midmark_top['y'] = midmark['y'] + 25 * (math.sin(orient))
        midmark_bottom['y'] = midmark['y'] - 25 * (math.sin(orient))
        if rxn_numb == 0:
            layout['nodes'][str(max + 1)] = prim_react
            prim_id = max + 1
        layout['nodes'][str(max + 2)] = prim_prod
        layout['nodes'][str(max + 3)] = midmark
        layout['nodes'][str(max + 4)] = midmark_top
        layout['nodes'][str(max + 5)] = midmark_bottom

        layout['reactions'][str(maxR + 1)] = {'name': str(rxn_numb),
              'bigg_id': str(rxn_numb),
              'segments': {},
              'genes': [],
              'reversibility': False,
              'metabolites': [],
              'label_x': midmark['x'] + 50 * math.sin(orient),
              'label_y': midmark['y'] + 50 * math.cos(orient),
              'gene_reaction_rule': ''}

        midmark_id = max + 3
        midmark_top_id = max + 4
        midmark_bottom_id = max + 5
        midmark_top['x'] = midmark['x'] - 25 * (math.cos(orient))
        midmark_bottom['x'] = midmark['x'] + 25 * (math.cos(orient))
        midmark_top['y'] = midmark['y'] + 25 * (math.sin(orient))
        midmark_bottom['y'] = midmark['y'] - 25 * (math.sin(orient))
        add_segment(layout, max + 4, prim_id, maxR, maxS + 1, midmark, midmark_top, prim_react, 1, orient, 0)
        add_segment(layout, max + 3, max + 4, maxR, maxS + 2, midmark, midmark, midmark_top, 1, orient, 0)
        add_segment(layout, max + 5, max + 3, maxR, maxS + 3, midmark, midmark_bottom, midmark, 1, orient, 0)
        add_segment(layout, max + 2, max + 5, maxR, maxS + 4, midmark, prim_prod, midmark_bottom, 1, orient, 0)
        prim = True
        if coefficient > 0:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': 1, 'bigg_id': prim_prod['bigg_id']}) 
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': prim_react['bigg_id']})         
            prim = False        
        else:
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': prim_prod['bigg_id']}) 
            layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': 1, 'bigg_id': prim_react['bigg_id']}) 
       
        nodes_in_layout.append(prim_react)
        nodes_in_layout.append(prim_prod)
        add_secondary_nodes2(second_react, second_prod, orient, prim_react['x'], prim_react['y'],  prim_react['x'], prim_react['y'], layout, stoich, midmark_top_id, midmark_bottom_id, midmark_id, midmark_top, midmark_bottom, midmark, maxR, prim)
        add_reaction2(layout, prim_prod, max + 2, prim_nodes, x + 500, y, reaction_length, orient, rxn_numb + 1, cc, nodes_in_layout, second_react_nodes, second_prod_nodes, max + 2)

    

In [4375]:
def add_reactions2(layout, stoich, x, y, reaction_length, orient, bigg_id):
    prim_react = {'node_is_primary': True,
                    'name': 'compound1',
                    'label_x': x,
                    'node_type': 'metabolite',
                    'y': y,
                    'x': x,
                    'label_y': y + 35,
                    'bigg_id': bigg_id
                     }
    prim_nodes = {}
    second_react_nodes = {}
    second_prod_nodes = {}
    for r in stoich:
        prim_nodes[r] = []
        second_react_nodes[r] = []
        second_prod_nodes[r] = []
        numb = 0
        for n in stoich[r]:
            if stoich[r][n] < 0 and numb == 0:
                dict = []
                dict.append(n)
                dict.append(stoich[r][n])
                prim_nodes[r].append(dict)
                numb +=1
            elif stoich[r][n] < 0:
                dict1 = []
                dict1.append(n)
                dict1.append(stoich[r][n])
                second_react_nodes[r].append(dict1)
        numb = 0
        for n in stoich[r]:
            if stoich[r][n] > 0 and numb == 0:
                dict2 = []
                dict2.append(n)
                dict2.append(stoich[r][n])
                prim_nodes[r].append(dict2)
                numb +=1
            elif stoich[r][n] > 0:
                dict3 = []
                dict3.append(n)
                dict3.append(stoich[r][n])
                second_prod_nodes[r].append(dict3)
    #count number of times compound appears in dict                
    nodes = []
    for r in prim_nodes.values():
        for n in r:
            if n[0] not in nodes:
                nodes.append(n[0])

    cc = {}
    for n in nodes:
        cc[n] = 0
    for n in nodes:
        for r in prim_nodes.values():
            for node in r[0]:
                if n == node:
                    cc[n] += 1
            for node in r[1]:
                if n == node:
                    cc[n] += 1


    nodes_in_layout = []
    add_reaction2(layout, prim_react, 1, prim_nodes, x, y, reaction_length, orient, 0, cc, nodes_in_layout, second_react_nodes, second_prod_nodes, 0)

In [3242]:
add_reactions2(layout, stoich, 5000, 5000)

In [4228]:
stoich4

{'0': {'cpd02611': -1, 'cpd00037': 1, 'cpd00062': -1, 'cpd00012': 1},
 '1': {'cpd00288': -1,
  'cpd00293': 1,
  'cpd00022': -1,
  'cpd00010': 1,
  'cpd00067': 1},
 '2': {'cpd00773': -1,
  'cpd02820': 1,
  'cpd00006': -1,
  'cpd00005': 1,
  'cpd00067': 1},
 '3': {'cpd03671': -1,
  'cpd02611': 1,
  'cpd00022': -1,
  'cpd00010': 1,
  'cpd00067': 1},
 '4': {'cpd00037': -1, 'cpd02820': 1, 'cpd00061': -1, 'cpd00009': 1},
 '5': {'cpd03671': -1, 'cpd00288': 1},
 '6': {'cpd19035': -1, 'cpd00288': 1, 'cpd00053': -1, 'cpd00023': 1}}

In [3676]:
def add_reaction(stoich, x_coord, y_coord, orient, reaction_length, prim_react, prim_prod, layout, rxn_numb, cc_coords, cc, prev_prod, prev_id):
    # base case
    if rxn_numb >= len(stoich):
        return
    # add midmarker node
    
    midmark = {'y': 0, 'x': 0, 'node_type': 'midmarker'}
    midmark_top = copy.deepcopy(midmark)
    midmark_bottom = copy.deepcopy(midmark)
    prim_react_id = 0
    prim_prod_id = 0 
    midmark_id = 0
    midmark_top_id = 0
    midmark_bottom_id = 0
    
    
    prim_nodes = get_nodes(stoich, rxn_numb)[0]
    second_nodes = get_nodes(stoich, rxn_numb)[1]
    # find largest key in layout
    max = 0
    for k in layout['nodes'].keys():
        k = int(k)
        if k > max:
            max = k
    prim_react = {'node_is_primary': True,
                    'name': 'compound1',
                    'label_x': 0,
                    'node_type': 'metabolite',
                    'y': 0,
                    'x': 0,
                    'label_y': 0
                     }
    prim_prod = {'node_is_primary': True,
                    'name': 'compound1',
                    'label_x': 0,
                    'node_type': 'metabolite',
                    'y': 0,
                    'x': 0,
                    'label_y': 0
                     }  
    
    prim_react['bigg_id'] = prim_nodes[0]
    prim_prod['bigg_id'] = prim_nodes[1]
    
    # Check to see if either node is chained    
    if prim_react['bigg_id'] not in cc or (prev_prod == prim_react['bigg_id'] or prev_prod == None):
        # check to see if product is chained
        if prim_prod['bigg_id'] in cc and (prev_prod != None and prev_prod != prim_react['bigg_id']):
            orient += math.pi / 2
            orient = add_prim_nodes(prim_react, prim_prod, orient, cc_coords, reaction_length, midmark, layout)
            orient += math.pi
            layout['nodes'][str(max + 1)] = prim_react
            layout['nodes'][str(max + 2)] = midmark
            layout['nodes'][str(max + 3)] = midmark_top
            layout['nodes'][str(max + 4)] = midmark_bottom
            prim_prod_id = cc_coords[prim_prod['bigg_id']]['node_id']
            prim_react_id = str(max + 1)
            midmark_id = str(max + 2)
            midmark_top_id = str(max + 3)
            midmark_bottom_id = str(max + 4)
            max += 4
        # chained but linear    
        else:
            prim_react['x'] = x_coord
            prim_react['y'] = y_coord
            prim_react['label_x'] = x_coord - 30
            prim_react['label_y'] = y_coord - 30
            for n in layout['nodes']:
                if layout['nodes'][n]['node_type'] == 'metabolite':
                    while (math.cos(orient) * reaction_length) + prim_react['x'] == layout['nodes'][n]['x'] and  - (math.sin(orient) * reaction_length) + prim_react['y'] == layout['nodes'][n]['y']:
                        orient += math.pi / 2
            prim_prod['x'] = (math.cos(orient) * reaction_length) + prim_react['x'] 
            prim_prod['y'] = - (math.sin(orient) * reaction_length) + prim_react['y'] 
            prim_prod['label_x'] = (math.cos(orient) * reaction_length) + prim_react['label_x']
            prim_prod['label_y'] = - (math.sin(orient) * reaction_length) + prim_react['label_y']
            midmark['x'] = (math.cos(orient) * (reaction_length / 2)) + prim_react['x'] 
            midmark['y'] = - (math.sin(orient) * (reaction_length / 2)) + prim_react['y']
            layout['nodes'][str(max + 1)] = midmark
            layout['nodes'][str(max + 2)] = prim_prod
            layout['nodes'][str(max + 3)] = midmark_top
            layout['nodes'][str(max + 4)] = midmark_bottom
            prim_prod_id = str(max + 2)
            prim_react_id = prev_id
            midmark_top_id = str(max + 3)
            midmark_bottom_id = str(max + 4)
            midmark_id = str(max + 1)
            max += 4

            if rxn_numb == 0:
                max -= 4
                layout['nodes'][str(max + 1)] = prim_react
                layout['nodes'][str(max + 2)] = prim_prod
                layout['nodes'][str(max + 3)] = midmark
                layout['nodes'][str(max + 4)] = midmark_top
                layout['nodes'][str(max + 5)] = midmark_bottom
                prim_react_id = str(max + 1)
                midmark_id = str(max + 3)
                midmark_top_id = str(max + 4)
                midmark_bottom_id = str(max + 5)
                max += 5
    else:
        if prim_prod['bigg_id'] in cc_coords.keys():
            orient += math.pi / 2
            add_prim_nodes(prim_react, prim_prod, orient, cc_coords, reaction_length, midmark, layout)
            orient += math.pi
            layout['nodes'][str(max + 1)] = prim_react
            layout['nodes'][str(max + 2)] = midmark
            layout['nodes'][str(max + 3)] = midmark_top
            layout['nodes'][str(max + 4)] = midmark_bottom
            prim_prod_id = cc_coords[prim_prod['bigg_id']]['node_id']
            prim_react_id = str(max + 1)
            midmark_id = str(max + 2)
            midmark_top_id = str(max + 3)
            midmark_bottom_id = str(max + 4)
            max += 4
            
        else:
            orient += 3 * math.pi / 2
            add_prim_nodes(prim_prod, prim_react, orient, cc_coords, reaction_length, midmark, layout)
            layout['nodes'][str(max + 2)] = prim_prod
            layout['nodes'][str(max + 3)] = midmark
            layout['nodes'][str(max + 4)] = midmark_top
            layout['nodes'][str(max + 5)] = midmark_bottom
            prim_react_id = cc_coords[prim_react['bigg_id']]['node_id']
            prim_prod_id = str(max + 2)
            midmark_id = str(max + 3)
            midmark_top_id = str(max + 4)
            midmark_bottom_id = str(max + 5)
            max += 5

              
    
    # add coordinates of branched reactions
    if prim_prod['bigg_id'] in cc:
        cc_coords[prim_prod['bigg_id']]['x'] = prim_prod['x']
        cc_coords[prim_prod['bigg_id']]['y'] = prim_prod['y']
        cc_coords[prim_prod['bigg_id']]['node_id'] = prim_prod_id
        
    if prim_react['bigg_id'] in cc:
        cc_coords[prim_react['bigg_id']]['x'] = prim_react['x']
        cc_coords[prim_react['bigg_id']]['y'] = prim_react['y']
        cc_coords[prim_react['bigg_id']]['node_id'] = prim_react_id
        
        
    if rxn_numb == 0 and prim_react['bigg_id'] in cc:
        cc_coords[prim_react['bigg_id']]['x'] = prim_react['x']
        cc_coords[prim_react['bigg_id']]['y'] = prim_react['y']
        cc_coords[prim_react['bigg_id']]['node_id'] = prim_react_id
    
    # get largest key for reactions
    maxR = 0
    for k in layout['reactions'].keys():
        k = int(k)
        if k > maxR:
            maxR = k
            
    # get largest key for segments
    maxS = 0
    for k in layout['reactions'].keys():
        for j in layout['reactions'][k]['segments'].keys():
            if int(j) > maxS:
                maxS = int(j)
    
    layout['reactions'][str(maxR + 1)] = {'name': str(rxn_numb),
          'bigg_id': str(rxn_numb),
          'segments': {},
          'genes': [],
          'reversibility': False,
          'metabolites': [],
          'label_x': midmark['x'] + 50 * math.sin(orient),
          'label_y': midmark['y'] + 50 * math.cos(orient),
          'gene_reaction_rule': ''}
    
    
    midmark_top['x'] = midmark['x'] - 25 * (math.cos(orient))
    midmark_bottom['x'] = midmark['x'] + 25 * (math.cos(orient))
    midmark_top['y'] = midmark['y'] + 25 * (math.sin(orient))
    midmark_bottom['y'] = midmark['y'] - 25 * (math.sin(orient))
    
        
    # alternate secondary node positions    
    ops = [operator.add, operator.sub]
    
    top = False
    p_to_s = 100
    mod = 0
    deg = 0
    # add secondary nodes to layout
    if len(second_nodes) > 0:
        maxS = add_secondary_nodes(second_nodes, orient, prim_react['x'], prim_react['y'], layout, stoich[str(rxn_numb)], midmark_top_id, midmark_bottom_id, midmark_id, midmark_top, midmark_bottom, midmark, maxR)
    maxS += 1
    #     for n in range(2, len(node_ids)):
#         add_length = reaction_length
#         if mod % 2 == 0:
#             p_to_s += 60
#             deg += math.pi / 30
            
#         secondary = {'node_is_primary': False,
#                       'name': 'compound1',
#                       'label_x': 0,
#                       'node_type': 'metabolite',
#                       'y': 0,
#                       'x': 0,
#                       'bigg_id': node_ids[n],
#                       'label_y': 0
#                      }
        
#         # secondary nodes are reactants
#         react = False
#         if int(stoich[str(rxn_numb)][str(node_ids[n])]) < 0:
#             add_length = 0
#             react = True
        
#         # transformation matrix
#         secondary['x'] = (ops[not(top)](abs(add_length - p_to_s * math.cos(math.pi / 6)) * (math.cos(orient)), (p_to_s * math.sin(math.pi / 6) * (math.sin(orient)))) + prim_react['x']) 
#         secondary['y'] = (- ops[top](abs(add_length - p_to_s * math.cos(math.pi / 6)) * (math.sin(orient)), (p_to_s * math.sin(math.pi / 6) * (math.cos(orient)))) + prim_react['y']) 
#         secondary['label_x'] = ops[top](secondary['x'] ,(30 * math.sin(orient)))
#         secondary['label_y'] = ops[not(top)](secondary['y'] ,(30 * math.cos(orient)))
#         layout['nodes'][str(max + 1)] = secondary
#         add_segment(layout, midmark_top_id, str(max + 1), maxR, maxS, midmark, midmark_top, secondary, react, orient, n)
#         layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': stoich[str(rxn_numb)][secondary['bigg_id']], 'bigg_id': secondary['bigg_id'] })
#         maxS += 1
#         max += 1
#         top = not(top)
#         mod += 1
    
    
    
    # add segments
    add_segment(layout, midmark_top_id, prim_react_id, maxR, maxS, midmark, midmark_top, prim_react, 1, orient, 0)
    layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': stoich[str(rxn_numb)][prim_react['bigg_id']], 'bigg_id': prim_react['bigg_id'] })
    add_segment(layout, prim_prod_id, midmark_bottom_id, maxR, maxS + 1, midmark, prim_prod, midmark_bottom, 0, orient, 0)
    layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': stoich[str(rxn_numb)][prim_prod['bigg_id']], 'bigg_id': prim_prod['bigg_id'] })
    add_segment(layout, midmark_id, midmark_top_id, maxR, maxS + 2, midmark, midmark, midmark_top, None, orient, 0)
    add_segment(layout, midmark_bottom_id, midmark_id, maxR, maxS + 3, midmark, midmark_bottom, midmark, None, orient, 0)
    

    # recursively place next reaction
    add_reaction(stoich, prim_prod['x'], prim_prod['y'], 3 * math.pi / 2, 500, 't1', 't2', layout, rxn_numb + 1, cc_coords, cc, prim_prod['bigg_id'], prim_prod_id )

In [3095]:
 def add_reactions(stoich, cc, x, y, layout):
    cc_coords = {}
    for c in cc:
        cc_coords[c] = {'x': 0, 'y': 0}
    add_reaction(stoich, x, y, 3 * math.pi / 2, 500, 't1', 't2', layout, 0, cc_coords, cc, None, None)

In [3726]:
def add_secondary_nodes(second_nodes, orient, x_coord, y_coord, layout, stoich, midmark_top_id, midmark_bottom_id, midmark_id, midmark_top, midmark_bottom, midmark, maxR):
    react_coords = []
    react_nodes = []
    prod_coords = []
    for n in range(0, len(second_nodes) - 1):
        if stoich[second_nodes[n]] < 0:
            react_nodes.append(second_nodes.pop(n))
            
    prod_nodes = second_nodes 
    
    max = 0
    for n in layout['nodes']:
        if int(n) > max:
            max = int(n)
    maxS = 0
    for k in layout['reactions']:
        for j in layout['reactions'][k]['segments']:
            if int(j) > maxS:
                maxS = int(j)
    
    if len(react_nodes) > 0: 
        react_coords = generate_coords(lambda x : pow(x, 2) / 90, len(react_nodes), 60, 160, x_coord, y_coord, orient, False)
    for c in range(0, len(react_coords)):
        secondary = {'node_is_primary': False,
                      'name': 'compound1',
                      'label_x': 0,
                      'node_type': 'metabolite',
                      'y': 0,
                      'x': 0,
                      'bigg_id': '',
                      'label_y': 0
                     }
        secondary['x'] = react_coords[c][0]
        secondary['label_x'] = react_coords[c][0]
        secondary['y'] = react_coords[c][1]
        secondary['label_y'] = react_coords[c][1] + 30
        secondary['bigg_id'] = react_nodes[c]
        layout['nodes'][str(max + 1)] = secondary
        add_segment(layout, str(max + 1), midmark_top_id, maxR, maxS, midmark, secondary, midmark_top, True, orient, c)
        layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': react_nodes[c]})
        max += 1
        maxS += 1
        
    if len(prod_nodes) > 0:         
        prod_coords = generate_coords(lambda x : pow(x, 2) / 90, len(prod_nodes), -60, -160, x_coord + 500, y_coord, orient, True)
    for c in range(0, len(prod_coords)):
        secondary = {'node_is_primary': False,
                      'name': 'compound1',
                      'label_x': 0,
                      'node_type': 'metabolite',
                      'y': 0,
                      'x': 0,
                      'bigg_id': '',
                      'label_y': 0
                     }
        secondary['x'] = prod_coords[c][0]
        secondary['label_x'] = prod_coords[c][0]
        secondary['y'] = prod_coords[c][1]
        secondary['label_y'] = prod_coords[c][1] + 30
        secondary['bigg_id'] = prod_nodes[c]
        layout['nodes'][str(max + 1)] = secondary
        add_segment(layout, str(max + 1), midmark_bottom_id, maxR, maxS, midmark, secondary, midmark_bottom, False, orient, c)
        layout['reactions'][str(maxR + 1)]['metabolites'].append({'coefficient': -1, 'bigg_id': prod_nodes[c]})
        max += 1
        maxS += 1
    
    return maxS
    

       

In [3728]:
stoich = stoich3 
cc = cc3
layout['nodes'] = {}
layout['reactions'] = {}
add_reactions(stoich, cc, 5000, 5000, layout)
with open('escher/maps/Saccharomyces cerevisiae/iMM904.Central carbon metabolism2.json', 'w') as fh:
    fh.write(json.dumps(escher_map))

In [3708]:
stoich4 = {'0': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1, 'cpd00021': -1, 'cpd00022': -1, 'cpd00024': -1,'cpd00025': -1,'cpd00026': -1,'cpd00027': -1,'cpd00028': -1}}

In [3685]:
layout

{'reactions': {'1': {'name': '0',
   'bigg_id': '0',
   'segments': {'2': {'to_node_id': '4',
     'from_node_id': '1',
     'b2': {'y': 5200.0, 'x': 5000.0},
     'b1': {'y': 5100.0, 'x': 5000.0}},
    '3': {'to_node_id': '2',
     'from_node_id': '5',
     'b2': {'y': 5300.0, 'x': 5000.0},
     'b1': {'y': 5400.0, 'x': 5000.0}},
    '4': {'to_node_id': '3', 'from_node_id': '4', 'b2': None, 'b1': None},
    '5': {'to_node_id': '5', 'from_node_id': '3', 'b2': None, 'b1': None}},
   'genes': [],
   'reversibility': False,
   'metabolites': [{'coefficient': -1, 'bigg_id': 'cpd00035'},
    {'coefficient': 1, 'bigg_id': 'cpd00117'}],
   'label_x': 4950.0,
   'label_y': 5250.0,
   'gene_reaction_rule': ''},
  '2': {'name': '1',
   'bigg_id': '1',
   'segments': {'7': {'to_node_id': '8',
     'from_node_id': '6',
     'b2': {'y': 5000.0, 'x': 5300.0},
     'b1': {'y': 5000.0, 'x': 5400.0}},
    '8': {'to_node_id': '1',
     'from_node_id': '9',
     'b2': {'y': 5000.0, 'x': 5200.0},
     'b1

In [3102]:
stoich = {'0': {'cpd00065_c0': -1,
  'cpd01747_c0': 1,
  'cpd00007_c0': -1,
  'cpd00001_c0': 1,
  'cpd00011_c0': 1,
  },
 '1': {'cpd01747_c0': -1,
  'cpd00703_c0': 1,
  'cpd00001_c0': -1,
  'cpd00013_c0': 1}}

In [3043]:
for n in range(0,5):
    stoich['0'][str(n)] = 1

In [3042]:
stoich['0'] = {'cpd00065_c0': -1,
  'cpd01747_c0': 1,
  'cpd00007_c0': -1,
  'cpd00001_c0': 1,
  'cpd00011_c0': 1,}

In [3447]:
stoich = stoich2
cc = ['']
layout['nodes'] = {}
layout['reactions'] = {}
add_reactions(stoich, cc, 3000, 3000, layout)
with open('escher/maps/Saccharomyces cerevisiae/iMM904.Central carbon metabolism2.json', 'w') as fh:
    fh.write(json.dumps(escher_map))

['cpd00197', 'cpd00856']
3060 3120.0
3440 3120.0
['cpd00856', 'cpd00938']
['cpd00938', 'cpd02255']
3060.0 4120.0
3440.0 4120.0
['cpd02255', 'cpd00630']
3060.0 4620.0
3440.0 4620.0
['cpd00630', 'cpd01507']
['cpd01507', 'cpd00078']
3060.0 5620.0
3440.0 5620.0
['cpd00078', 'cpd00279']
3060.0 6120.0
3440.0 6120.0


In [3135]:
stoich = stoich3
cc = cc3
layout['nodes'] = {}
layout['reactions'] = {}
add_reactions(stoich, cc, 5000, 5000, layout)
with open('escher/maps/Saccharomyces cerevisiae/iMM904.Central carbon metabolism2.json', 'w') as fh:
    fh.write(json.dumps(escher_map))

['cpd00035', 'cpd00117']
['cpd00156', 'cpd00035']
['cpd00123', 'cpd00156']
6060.0 5120.0
6440.0 5120.0
['cpd00084', 'cpd00035']
5060.0 4620.0
5440.0 4620.0
['cpd00035', 'cpd00020']
5060 5120.0
5440 5120.0


In [1814]:
add_reactions(stoich, cc, 5000, 5000, layout)

In [3125]:
stoich3

{'0': {'cpd00035': -1, 'cpd00117': 1},
 '1': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1},
 '2': {'cpd00156': 1, 'cpd00123': -1, 'cpd00024': -1, 'cpd00023': 1},
 '3': {'cpd00084': -1, 'cpd00035': 1, 'cpd14547': -1, 'cpd14548': 1},
 '4': {'cpd00035': -1, 'cpd00024': -1, 'cpd00020': 1, 'cpd00023': 1}}

In [2573]:
stoich['2'] = {'cpd00156': 1, 'cpd00123': -1, 'cpd00024': -1, 'cpd00023': 1}

In [1933]:
import networkx as nx

In [2682]:
cc1 = cc

In [2734]:
stoich1 = stoich

In [3623]:
stoich3 = {'0': {'cpd00035': -1, 'cpd00117': 1},
 '1': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1},
 '2': {'cpd00156': 1, 'cpd00123': -1, 'cpd00024': -1, 'cpd00023': 1},
 '3': {'cpd00084': -1, 'cpd00035': 1, 'cpd14547': -1, 'cpd14548': 1},
 '4': {'cpd00035': -1, 'cpd00024': -1, 'cpd00020': 1, 'cpd00023': 1}}

In [2681]:
cc = ['']

In [ ]:
'3': {'cpd00084': -1, 'cpd00035': 1, 'cpd14547': -1, 'cpd14548': 1}}

In [2596]:
stoich = {'0': {'cpd00035': -1, 'cpd00117': 1},
 '1': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1},
 '2': {'cpd00156': 1, 'cpd00123': -1, 'cpd00024': -1, 'cpd00023': 1},
 '3': {'cpd00084': -1, 'cpd00035': 1, 'cpd14547': -1, 'cpd14548': 1}}

In [2670]:
stoich = {'6': {'cpd00078': -1, 'cpd00279': 1, 'cpd00142': -1, 'cpd00036': 1},
 '2': {'cpd00938': -1, 'cpd02255': 1, 'cpd00067': -1, 'cpd00011': 1},
 '3': {'cpd02255': -1, 'cpd00630': 1, 'cpd00001': -1, 'cpd00067': 1},
 '4': {'cpd01507': 1, 'cpd00630': -1,'cpd00078': 1, 'cpd00036': 1},
 '5': {'cpd00078': 1, 'cpd01507': -1, 'cpd00022': -1, 'cpd00010': 1},
 '0': {'cpd00197': -1, 'cpd00856': 1, 'cpd00007': -1, 'cpd00067': 2},
 '1': {'cpd00938': 0, 'cpd00856': -1, 'cpd00067': 1}}

In [2486]:
map2 = [{"map_name": "new_map", "map_id": "lLGzTsO3Cyot", "map_description": "\nLast Modified Tue Jun 11 2019 14:05:27 GMT-0500 (Central Daylight Time)", "homepage": "https://escher.github.io", "schema": "https://escher.github.io/escher/jsonschema/1-0-0#"}, {"reactions": {"2": {"name": "rxn00283", "bigg_id": "rxn00283", "reversibility": false, "label_x": -378.9999694824219, "label_y": 767, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00035", "coefficient": -1}, {"bigg_id": "cpd00117", "coefficient": 1}], "segments": {"12": {"from_node_id": "15", "to_node_id": "16", "b1": null, "b2": null}, "13": {"from_node_id": "17", "to_node_id": "16", "b1": null, "b2": null}, "14": {"from_node_id": "15", "to_node_id": "14", "b1": {"x": -393.9999694824219, "y": 695}, "b2": {"x": -393.9999694824219, "y": 640.75}}, "15": {"from_node_id": "17", "to_node_id": "18", "b1": {"x": -393.9999694824219, "y": 819}, "b2": {"x": -393.9999694824219, "y": 873.25}}}}, "4": {"name": "rxn00904", "bigg_id": "rxn00904", "reversibility": false, "label_x": -378.9999694824219, "label_y": 417, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00020", "coefficient": -1}, {"bigg_id": "cpd00156", "coefficient": -1}, {"bigg_id": "cpd00035", "coefficient": 1}, {"bigg_id": "cpd00123", "coefficient": 1}], "segments": {"22": {"from_node_id": "26", "to_node_id": "27", "b1": null, "b2": null}, "23": {"from_node_id": "28", "to_node_id": "27", "b1": null, "b2": null}, "24": {"from_node_id": "26", "to_node_id": "29", "b1": {"x": -393.9999694824219, "y": 341}, "b2": {"x": -375.9999694824219, "y": 335.75}}, "25": {"from_node_id": "26", "to_node_id": "30", "b1": {"x": -393.9999694824219, "y": 345}, "b2": {"x": -393.9999694824219, "y": 290.75}}, "26": {"from_node_id": "28", "to_node_id": "14", "b1": {"x": -393.9999694824219, "y": 469}, "b2": {"x": -393.9999694824219, "y": 523.25}}, "27": {"from_node_id": "28", "to_node_id": "31", "b1": {"x": -393.9999694824219, "y": 473}, "b2": {"x": -375.9999694824219, "y": 478.25}}}}, "5": {"name": "rxn00903", "bigg_id": "rxn00903", "reversibility": false, "label_x": -378.9999694824219, "label_y": 67, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00024", "coefficient": -1}, {"bigg_id": "cpd00156", "coefficient": -1}, {"bigg_id": "cpd00023", "coefficient": 1}, {"bigg_id": "cpd00123", "coefficient": 1}], "segments": {"28": {"from_node_id": "32", "to_node_id": "33", "b1": null, "b2": null}, "29": {"from_node_id": "34", "to_node_id": "33", "b1": null, "b2": null}, "30": {"from_node_id": "32", "to_node_id": "35", "b1": {"x": -393.9999694824219, "y": 123}, "b2": {"x": -375.9999694824219, "y": 128.25}}, "31": {"from_node_id": "32", "to_node_id": "30", "b1": {"x": -393.9999694824219, "y": 119}, "b2": {"x": -393.9999694824219, "y": 173.25}}, "32": {"from_node_id": "34", "to_node_id": "36", "b1": {"x": -393.9999694824219, "y": -5}, "b2": {"x": -365.0718028977366, "y": -28.538923188751095}}, "33": {"from_node_id": "34", "to_node_id": "37", "b1": {"x": -393.9999694824219, "y": -9}, "b2": {"x": -394.8191047060389, "y": -16.465505058936202}}}}, "6": {"name": "rxn07292", "bigg_id": "rxn07292", "reversibility": false, "label_x": -618.9999694824219, "label_y": 542, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00084", "coefficient": -1}, {"bigg_id": "cpd14547", "coefficient": -1}, {"bigg_id": "cpd00035", "coefficient": 1}, {"bigg_id": "cpd14548", "coefficient": 1}], "segments": {"34": {"from_node_id": "38", "to_node_id": "39", "b1": null, "b2": null}, "35": {"from_node_id": "40", "to_node_id": "39", "b1": null, "b2": null}, "36": {"from_node_id": "38", "to_node_id": "41", "b1": {"x": -634.9999694824219, "y": 582}, "b2": {"x": -672.8771600962205, "y": 583.5044735311532}}, "37": {"from_node_id": "38", "to_node_id": "42", "b1": {"x": -630.9999694824219, "y": 582}, "b2": {"x": -649.119464412386, "y": 551.1994883450482}}, "38": {"from_node_id": "40", "to_node_id": "14", "b1": {"x": -506.9999694824219, "y": 582}, "b2": {"x": -452.7499694824219, "y": 582}}, "39": {"from_node_id": "40", "to_node_id": "43", "b1": {"x": -502.9999694824219, "y": 582}, "b2": {"x": -497.7499694824219, "y": 564}}}}, "7": {"name": "rxn00191", "bigg_id": "rxn00191", "reversibility": false, "label_x": -268.9999694824219, "label_y": 542, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00024", "coefficient": -1}, {"bigg_id": "cpd00035", "coefficient": -1}, {"bigg_id": "cpd00020", "coefficient": 1}, {"bigg_id": "cpd00023", "coefficient": 1}], "segments": {"40": {"from_node_id": "44", "to_node_id": "45", "b1": null, "b2": null}, "41": {"from_node_id": "46", "to_node_id": "45", "b1": null, "b2": null}, "42": {"from_node_id": "44", "to_node_id": "47", "b1": {"x": -284.9999694824219, "y": 582}, "b2": {"x": -290.2499694824219, "y": 600}}, "43": {"from_node_id": "44", "to_node_id": "14", "b1": {"x": -280.9999694824219, "y": 582}, "b2": {"x": -335.2499694824219, "y": 582}}, "44": {"from_node_id": "46", "to_node_id": "48", "b1": {"x": -152.99996948242188, "y": 582}, "b2": {"x": -141.66637696200453, "y": 580.9015238972845}}, "45": {"from_node_id": "46", "to_node_id": "49", "b1": {"x": -156.99996948242188, "y": 582}, "b2": {"x": -152.25396188800758, "y": 613.3914219455767}}}}}, "nodes": {"14": {"node_type": "metabolite", "x": -393.9999694824219, "y": 582, "bigg_id": "cpd00035", "name": "L-Alanine", "label_x": -436.0650939941406, "label_y": 608.8260498046875, "node_is_primary": true}, "15": {"node_type": "multimarker", "x": -393.9999694824219, "y": 737}, "16": {"node_type": "midmarker", "x": -393.9999694824219, "y": 757}, "17": {"node_type": "multimarker", "x": -393.9999694824219, "y": 777}, "18": {"node_type": "metabolite", "x": -393.9999694824219, "y": 932, "bigg_id": "cpd00117", "name": "D-Alanine", "label_x": -368.9999694824219, "label_y": 932, "node_is_primary": true}, "26": {"node_type": "multimarker", "x": -393.9999694824219, "y": 387}, "27": {"node_type": "midmarker", "x": -393.9999694824219, "y": 407}, "28": {"node_type": "multimarker", "x": -393.9999694824219, "y": 427}, "29": {"node_type": "metabolite", "x": -313.9999694824219, "y": 302, "bigg_id": "cpd00020", "name": "Pyruvate", "label_x": -298.9999694824219, "label_y": 302, "node_is_primary": false}, "30": {"node_type": "metabolite", "x": -393.9999694824219, "y": 232, "bigg_id": "cpd00156", "name": "L-Valine", "label_x": -368.9999694824219, "label_y": 232, "node_is_primary": true}, "31": {"node_type": "metabolite", "x": -313.9999694824219, "y": 512, "bigg_id": "cpd00123", "name": "3-Methyl-2-oxobutanoate", "label_x": -298.9999694824219, "label_y": 512, "node_is_primary": false}, "32": {"node_type": "multimarker", "x": -393.9999694824219, "y": 77}, "33": {"node_type": "midmarker", "x": -393.9999694824219, "y": 57}, "34": {"node_type": "multimarker", "x": -393.9999694824219, "y": 37}, "35": {"node_type": "metabolite", "x": -313.9999694824219, "y": 162, "bigg_id": "cpd00024", "name": "2-Oxoglutarate", "label_x": -298.9999694824219, "label_y": 162, "node_is_primary": false}, "36": {"node_type": "metabolite", "x": -309.9999694824219, "y": -49, "bigg_id": "cpd00023", "name": "L-Glutamate", "label_x": -284.9999694824219, "label_y": -49, "node_is_primary": false}, "37": {"node_type": "metabolite", "x": -391.9999694824219, "y": -87, "bigg_id": "cpd00123", "name": "3-Methyl-2-oxobutanoate", "label_x": -376.9999694824219, "label_y": -87, "node_is_primary": true}, "38": {"node_type": "multimarker", "x": -588.9999694824219, "y": 582}, "39": {"node_type": "midmarker", "x": -568.9999694824219, "y": 582}, "40": {"node_type": "multimarker", "x": -548.9999694824219, "y": 582}, "41": {"node_type": "metabolite", "x": -743.4676208496094, "y": 583.7266235351562, "bigg_id": "cpd00084", "name": "L-Cysteine", "label_x": -786.6676208496094, "label_y": 563.7266235351562, "node_is_primary": true}, "42": {"node_type": "metabolite", "x": -674.5323181152344, "y": 498.23016357421875, "bigg_id": "cpd14547", "name": "C15811", "label_x": -717.7323181152344, "label_y": 538.2301635742188, "node_is_primary": false}, "43": {"node_type": "metabolite", "x": -463.9999694824219, "y": 502, "bigg_id": "cpd14548", "name": "C15812", "label_x": -507.19996948242186, "label_y": 482, "node_is_primary": false}, "44": {"node_type": "multimarker", "x": -238.99996948242188, "y": 582}, "45": {"node_type": "midmarker", "x": -218.99996948242188, "y": 582}, "46": {"node_type": "multimarker", "x": -198.99996948242188, "y": 582}, "47": {"node_type": "metabolite", "x": -323.9999694824219, "y": 662, "bigg_id": "cpd00024", "name": "2-Oxoglutarate", "label_x": -367.19996948242186, "label_y": 702, "node_is_primary": false}, "48": {"node_type": "metabolite", "x": -71.07829284667969, "y": 581.5219116210938, "bigg_id": "cpd00020", "name": "Pyruvate", "label_x": -114.27829284667968, "label_y": 621.5219116210938, "node_is_primary": false}, "49": {"node_type": "metabolite", "x": -124.47811889648438, "y": 665.1607666015625, "bigg_id": "cpd00023", "name": "L-Glutamate", "label_x": -167.67811889648436, "label_y": 705.1607666015625, "node_is_primary": false}}, "text_labels": {"0": {"x": -625.5147705078125, "y": -252.08319091796875, "text": "Alanine Biosynthesis"}}, "canvas": {"x": -1048, "y": -476.61676025390625, "width": 1343.9468383789062, "height": 1652.9218139648438}}]

In [2512]:
stoich['4'] = {'cpd00035': -1, 'cpd00020': 1, 'cpd00024': -1, 'cpd00023': 1}

In [2277]:
rxn_order = sort_reactions(pwy)[0]

In [3136]:
stoich3

{'0': {'cpd00035': -1, 'cpd00117': 1},
 '1': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1},
 '2': {'cpd00156': 1, 'cpd00123': -1, 'cpd00024': -1, 'cpd00023': 1},
 '3': {'cpd00084': -1, 'cpd00035': 1, 'cpd14547': -1, 'cpd14548': 1},
 '4': {'cpd00035': -1, 'cpd00024': -1, 'cpd00020': 1, 'cpd00023': 1}}

In [2592]:
stoich = {'0': {'cpd00035': -1, 'cpd00117': 1},
 '1': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1},
 '2': {'cpd00156': 1, 'cpd00123': 1, 'cpd00024': -1, 'cpd00023': 1}}

In [2733]:
rxn_numb = 0
for r in rxn_order:
    stoich[str(rxn_numb)] = {}
    stoich[str(rxn_numb)][pwy['rxns'][r]['main_reactants'][0][1]] = pwy['rxns'][r]['main_reactants'][0][0]
    stoich[str(rxn_numb)][pwy['rxns'][r]['main_products'][0][1]] = pwy['rxns'][r]['main_products'][0][0]
    for s in pwy['rxns'][r]['cofactor_reactants']:
        stoich[str(rxn_numb)][s[1]] = s[0]
    for t in pwy['rxns'][r]['cofactor_products']:
        stoich[str(rxn_numb)][t[1]] = t[0]
    rxn_numb += 1

In [3112]:
stoich2 = {'6': {'cpd00078': -1, 'cpd00279': 1, 'cpd00142': -1, 'cpd00036': 1},
 '2': {'cpd00938': -1, 'cpd02255': 1, 'cpd00067': -1, 'cpd00011': 1},
 '3': {'cpd02255': -1, 'cpd00630': 1, 'cpd00001': -1, 'cpd00067': 1},
 '4': {'cpd01507': 1, 'cpd00630': -1, 'cpd00078': 1, 'cpd00036': 1},
 '5': {'cpd00078': 1, 'cpd01507': -1, 'cpd00022': -1, 'cpd00010': 1},
 '0': {'cpd00197': -1, 'cpd00856': 1, 'cpd00007': -1, 'cpd00067': 2},
 '1': {'cpd00938': 1, 'cpd00856': -1, 'cpd00067': 1}}

In [1941]:
def sort_reactions(pwy):
    rxns = pwy['rxns']
    g = nx.DiGraph()
    for rxn_id in pwy['rxns']:
        for o in pwy['rxns'][rxn_id]['main_reactants']:
            g.add_edge(o[1], rxn_id)
        for o in pwy['rxns'][rxn_id]['main_products']:
            g.add_edge(rxn_id, o[1])
    source = pwy['main_reactants'][0][1]
    target = pwy['main_products'][0][1]
    path = nx.algorithms.shortest_path(G=g, source=source, target=target)
    rxn_order = list(filter(lambda o : o in rxns.keys(), path))
    cpd_order = list(filter(lambda o : o not in rxns.keys(), path))[1:-1]
    
    return rxn_order, cpd_order
    nx.draw_networkx(g)


In [1939]:
pwy = {
      "cofactor_products" : [
         [
            1,
            "cpd00011_c0",
            "CO2"
         ],
         [
            1,
            "cpd00013_c0",
            "NH3"
         ]
      ],
      "main_products" : [
         [
            1,
            "cpd00703_c0",
            "Indoleacetate"
         ]
      ],
      "gfrxn" : 0,
      "name" : "Indoleacetate",
      "cofactor_reactants" : [
         [
            -1,
            "cpd00007_c0",
            "O2"
         ]
      ],
      "rxns" : {
         "rxn02222" : {
            "gfrxn" : 0,
            "cofactor_reactants" : [
               [
                  -1,
                  "cpd00001_c0",
                  "H2O"
               ]
            ],
            "flux" : 1,
            "cofactor_products" : [
               [
                  1,
                  "cpd00013_c0",
                  "NH3"
               ]
            ],
            "main_reactants" : [
               [
                  -1,
                  "cpd01747_c0",
                  "Indole-3-acetamide"
               ]
            ],
            "main_products" : [
               [
                  1,
                  "cpd00703_c0",
                  "Indoleacetate"
               ]
            ],
            "gpr" : "(PFLU_5658 or PFLU_2536 or PFLU_2173 or PFLU_5051 or PFLU_5596)"
         },
         "rxn00479" : {
            "gfrxn" : 0,
            "cofactor_reactants" : [
               [
                  -1,
                  "cpd00007_c0",
                  "O2"
               ]
            ],
            "flux" : 1,
            "cofactor_products" : [
               [
                  1,
                  "cpd00001_c0",
                  "H2O"
               ],
               [
                  1,
                  "cpd00011_c0",
                  "CO2"
               ]
            ],
            "main_reactants" : [
               [
                  -1,
                  "cpd00065_c0",
                  "L-Tryptophan"
               ]
            ],
            "main_products" : [
               [
                  1,
                  "cpd01747_c0",
                  "Indole-3-acetamide"
               ]
            ],
            "gpr" : "PFLU_2174"
         }
      },
      "main_reactants" : [
         [
            -1,
            "cpd00065_c0",
            "L-Tryptophan"
         ]
      ],
      "totalrxn" : 2,
      "auxotrophic" : 0
   }


In [1978]:
layout2

{'reactions': {'667215': {'name': 'Acetaldehyde reversible transport',
   'bigg_id': 'ACALDt',
   'segments': {'1636': {'to_node_id': '670874',
     'from_node_id': '669549',
     'b2': {'y': 5911.257, 'x': 5201.0},
     'b1': {'y': 5872.757, 'x': 5201.0}},
    '1637': {'to_node_id': '672425',
     'from_node_id': '670874',
     'b2': {'y': 6031.253, 'x': 5200.9965},
     'b1': {'y': 5985.7558, 'x': 5200.99895}}},
   'genes': [],
   'reversibility': True,
   'metabolites': [{'coefficient': 1.0, 'bigg_id': 'acald_c'},
    {'coefficient': -1.0, 'bigg_id': 'acald_e'}],
   'label_x': 5018.5439453125,
   'label_y': 5975.18717578125,
   'gene_reaction_rule': ''},
  '669328': {'name': 'Succinate dehydrogenase',
   'bigg_id': 'SUCD1m',
   'segments': {'5689': {'to_node_id': '669125',
     'from_node_id': '671776',
     'b2': {'y': 8010.204715867223, 'x': 2975.6443495021276},
     'b1': {'y': 8063.434314760167, 'x': 2993.3878448506384}},
    '5688': {'to_node_id': '672373',
     'from_node_id':

In [1812]:
layout['nodes'] = {}

In [1813]:
layout['reactions'] = {}

In [2732]:
stoich = {}

In [2241]:
stoich['0'] = {'udpg_c': -1, 'udp_c':1, 'h_c':1, '13BDglcn_c':1}

In [2242]:
stoich['1'] = {'udp_c':-1, 'udpacgal_c':1, 'h_c': -1, 'chitin_c': -1}

In [2243]:
stoich['2'] = {'udpacgal_c':-1, 'utp_c':1, 'ppi_c': -1, 'acgam1p_c':1, 'h_c': 1}

In [1866]:
stoich

{'0': {'udpg_c': '-1', 'udp_c': '1', 'h_c': '1', '13BDglcn_c': '1'}}

In [1364]:
layout['text_labels'] = {}

In [1851]:
map = [{"map_name": "new_map", "map_id": "6HDPmUKtCDIv", "map_description": "", "homepage": "https://escher.github.io", "schema": "https://escher.github.io/escher/jsonschema/1-0-0#"}, {"reactions": {"0": {"name": "rxn00290", "bigg_id": "rxn00290", "reversibility": false, "label_x": -93, "label_y": 1443.0000457763672, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00078", "coefficient": -1}, {"bigg_id": "cpd00142", "coefficient": -1}, {"bigg_id": "cpd00036", "coefficient": 1}, {"bigg_id": "cpd00279", "coefficient": 1}], "segments": {"0": {"from_node_id": "1", "to_node_id": "2", "b1": null, "b2": null}, "1": {"from_node_id": "3", "to_node_id": "2", "b1": null, "b2": null}, "2": {"from_node_id": "1", "to_node_id": "30", "b1": {"x": -108, "y": 1371.0000457763672}, "b2": {"x": -108, "y": 1316.7500457763672}}, "3": {"from_node_id": "1", "to_node_id": "4", "b1": {"x": -108, "y": 1367.0000457763672}, "b2": {"x": -126, "y": 1361.7500457763672}}, "4": {"from_node_id": "3", "to_node_id": "5", "b1": {"x": -108, "y": 1499.0000457763672}, "b2": {"x": -126, "y": 1504.2500457763672}}, "5": {"from_node_id": "3", "to_node_id": "6", "b1": {"x": -108, "y": 1495.0000457763672}, "b2": {"x": -108, "y": 1549.2500457763672}}}}, "1": {"name": "rxn02483", "bigg_id": "rxn02483", "reversibility": false, "label_x": -90, "label_y": 39.000030517578125, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00067", "coefficient": -1}, {"bigg_id": "cpd00938", "coefficient": -1}, {"bigg_id": "cpd00011", "coefficient": 1}, {"bigg_id": "cpd02255", "coefficient": 1}], "segments": {"6": {"from_node_id": "8", "to_node_id": "9", "b1": null, "b2": null}, "7": {"from_node_id": "10", "to_node_id": "9", "b1": null, "b2": null}, "8": {"from_node_id": "8", "to_node_id": "7", "b1": {"x": -105, "y": -32.999969482421875}, "b2": {"x": -121.25100561192431, "y": -40.656227733638005}}, "9": {"from_node_id": "8", "to_node_id": "42", "b1": {"x": -105, "y": -36.999969482421875}, "b2": {"x": -102.5950435629252, "y": -101.42314185883201}}, "10": {"from_node_id": "10", "to_node_id": "12", "b1": {"x": -105, "y": 95.00003051757812}, "b2": {"x": -123, "y": 100.25003051757812}}, "11": {"from_node_id": "10", "to_node_id": "13", "b1": {"x": -105, "y": 91.00003051757812}, "b2": {"x": -105, "y": 145.25003051757812}}}}, "2": {"name": "rxn02144", "bigg_id": "rxn02144", "reversibility": false, "label_x": -90, "label_y": 389.0000305175781, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd02255", "coefficient": -1}, {"bigg_id": "cpd00067", "coefficient": 1}, {"bigg_id": "cpd00630", "coefficient": 1}], "segments": {"12": {"from_node_id": "14", "to_node_id": "15", "b1": null, "b2": null}, "13": {"from_node_id": "16", "to_node_id": "15", "b1": null, "b2": null}, "14": {"from_node_id": "14", "to_node_id": "17", "b1": {"x": -105, "y": 313.0000305175781}, "b2": {"x": -123, "y": 307.7500305175781}}, "15": {"from_node_id": "14", "to_node_id": "13", "b1": {"x": -105, "y": 317.0000305175781}, "b2": {"x": -105, "y": 262.7500305175781}}, "16": {"from_node_id": "16", "to_node_id": "18", "b1": {"x": -105, "y": 445.0000305175781}, "b2": {"x": -123, "y": 450.2500305175781}}, "17": {"from_node_id": "16", "to_node_id": "19", "b1": {"x": -105, "y": 441.0000305175781}, "b2": {"x": -105, "y": 495.2500305175781}}}}, "3": {"name": "rxn02143", "bigg_id": "rxn02143", "reversibility": false, "label_x": -90, "label_y": 739.0000305175781, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00078", "coefficient": -1}, {"bigg_id": "cpd00630", "coefficient": -1}, {"bigg_id": "cpd00036", "coefficient": 1}, {"bigg_id": "cpd01507", "coefficient": 1}], "segments": {"18": {"from_node_id": "20", "to_node_id": "21", "b1": null, "b2": null}, "19": {"from_node_id": "22", "to_node_id": "21", "b1": null, "b2": null}, "20": {"from_node_id": "20", "to_node_id": "23", "b1": {"x": -105, "y": 663.0000305175781}, "b2": {"x": -123, "y": 657.7500305175781}}, "21": {"from_node_id": "20", "to_node_id": "19", "b1": {"x": -105, "y": 667.0000305175781}, "b2": {"x": -105, "y": 612.7500305175781}}, "22": {"from_node_id": "22", "to_node_id": "24", "b1": {"x": -105, "y": 795.0000305175781}, "b2": {"x": -123, "y": 800.2500305175781}}, "23": {"from_node_id": "22", "to_node_id": "25", "b1": {"x": -105, "y": 791.0000305175781}, "b2": {"x": -105, "y": 845.2500305175781}}}}, "4": {"name": "rxn00598", "bigg_id": "rxn00598", "reversibility": false, "label_x": -90, "label_y": 1089.0000305175781, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00022", "coefficient": -1}, {"bigg_id": "cpd00078", "coefficient": -1}, {"bigg_id": "cpd00010", "coefficient": 1}, {"bigg_id": "cpd01507", "coefficient": 1}], "segments": {"24": {"from_node_id": "26", "to_node_id": "27", "b1": null, "b2": null}, "25": {"from_node_id": "28", "to_node_id": "27", "b1": null, "b2": null}, "26": {"from_node_id": "26", "to_node_id": "29", "b1": {"x": -105, "y": 1145.0000305175781}, "b2": {"x": -123, "y": 1150.2500305175781}}, "27": {"from_node_id": "26", "to_node_id": "30", "b1": {"x": -105, "y": 1141.0000305175781}, "b2": {"x": -105, "y": 1195.2500305175781}}, "28": {"from_node_id": "28", "to_node_id": "31", "b1": {"x": -105, "y": 1013.0000305175781}, "b2": {"x": -123, "y": 1007.7500305175781}}, "29": {"from_node_id": "28", "to_node_id": "25", "b1": {"x": -105, "y": 1017.0000305175781}, "b2": {"x": -105, "y": 962.7500305175781}}}}, "5": {"name": "rxn01192", "bigg_id": "rxn01192", "reversibility": false, "label_x": -90, "label_y": -681.0000762939453, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00197", "coefficient": -1}, {"bigg_id": "cpd00067", "coefficient": 2}, {"bigg_id": "cpd00856", "coefficient": 1}], "segments": {"30": {"from_node_id": "33", "to_node_id": "34", "b1": null, "b2": null}, "31": {"from_node_id": "35", "to_node_id": "34", "b1": null, "b2": null}, "32": {"from_node_id": "33", "to_node_id": "32", "b1": {"x": -105, "y": -753.0000762939453}, "b2": {"x": -130.2513923654197, "y": -749.4045524961871}}, "33": {"from_node_id": "33", "to_node_id": "36", "b1": {"x": -105, "y": -757.0000762939453}, "b2": {"x": -105.13690014515647, "y": -797.4184217507151}}, "34": {"from_node_id": "35", "to_node_id": "37", "b1": {"x": -105, "y": -625.0000762939453}, "b2": {"x": -123, "y": -619.7500762939453}}, "35": {"from_node_id": "35", "to_node_id": "38", "b1": {"x": -105, "y": -629.0000762939453}, "b2": {"x": -105, "y": -574.7500762939453}}}}, "6": {"name": "rxn02369", "bigg_id": "rxn02369", "reversibility": false, "label_x": -90, "label_y": -331.0000762939453, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00938", "coefficient": -1}, {"bigg_id": "cpd00067", "coefficient": 1}, {"bigg_id": "cpd00856", "coefficient": 1}], "segments": {"36": {"from_node_id": "39", "to_node_id": "40", "b1": null, "b2": null}, "37": {"from_node_id": "41", "to_node_id": "40", "b1": null, "b2": null}, "38": {"from_node_id": "39", "to_node_id": "42", "b1": {"x": -105, "y": -279.0000762939453}, "b2": {"x": -105, "y": -224.7500762939453}}, "39": {"from_node_id": "41", "to_node_id": "43", "b1": {"x": -105, "y": -407.0000762939453}, "b2": {"x": -123, "y": -412.2500762939453}}, "40": {"from_node_id": "41", "to_node_id": "38", "b1": {"x": -105, "y": -403.0000762939453}, "b2": {"x": -105, "y": -457.2500762939453}}}}}, "nodes": {"1": {"node_type": "multimarker", "x": -108, "y": 1413.0000457763672}, "2": {"node_type": "midmarker", "x": -108, "y": 1433.0000457763672}, "3": {"node_type": "multimarker", "x": -108, "y": 1453.0000457763672}, "4": {"node_type": "metabolite", "x": -188, "y": 1328.0000457763672, "bigg_id": "cpd00142", "name": "Acetoacetate", "label_x": -260, "label_y": 1358.0000457763672, "node_is_primary": false}, "5": {"node_type": "metabolite", "x": -188, "y": 1538.0000457763672, "bigg_id": "cpd00036", "name": "Succinate", "label_x": -260, "label_y": 1568.0000457763672, "node_is_primary": false}, "6": {"node_type": "metabolite", "x": -108, "y": 1608.0000457763672, "bigg_id": "cpd00279", "name": "Acetoacetyl-CoA", "label_x": -83, "label_y": 1608.0000457763672, "node_is_primary": true}, "7": {"node_type": "metabolite", "x": -180, "y": -40.999969482421875, "bigg_id": "cpd00067", "name": "H+", "label_x": -158, "label_y": -61.999969482421875, "node_is_primary": false}, "8": {"node_type": "multimarker", "x": -105, "y": 9.000030517578125}, "9": {"node_type": "midmarker", "x": -105, "y": 29.000030517578125}, "10": {"node_type": "multimarker", "x": -105, "y": 49.000030517578125}, "12": {"node_type": "metabolite", "x": -185, "y": 134.00003051757812, "bigg_id": "cpd00011", "name": "CO2", "label_x": -257, "label_y": 164.00003051757812, "node_is_primary": false}, "13": {"node_type": "metabolite", "x": -105, "y": 204.00003051757812, "bigg_id": "cpd02255", "name": "3-oxoadipate-enol-lactone", "label_x": -80, "label_y": 204.00003051757812, "node_is_primary": true}, "14": {"node_type": "multimarker", "x": -105, "y": 359.0000305175781}, "15": {"node_type": "midmarker", "x": -105, "y": 379.0000305175781}, "16": {"node_type": "multimarker", "x": -105, "y": 399.0000305175781}, "17": {"node_type": "metabolite", "x": -185, "y": 274.0000305175781, "bigg_id": "cpd00001", "name": "H2O", "label_x": -257, "label_y": 304.0000305175781, "node_is_primary": false}, "18": {"node_type": "metabolite", "x": -185, "y": 484.0000305175781, "bigg_id": "cpd00067", "name": "H+", "label_x": -257, "label_y": 514.0000305175781, "node_is_primary": false}, "19": {"node_type": "metabolite", "x": -105, "y": 554.0000305175781, "bigg_id": "cpd00630", "name": "3-Oxoadipate", "label_x": -80, "label_y": 554.0000305175781, "node_is_primary": true}, "20": {"node_type": "multimarker", "x": -105, "y": 709.0000305175781}, "21": {"node_type": "midmarker", "x": -105, "y": 729.0000305175781}, "22": {"node_type": "multimarker", "x": -105, "y": 749.0000305175781}, "23": {"node_type": "metabolite", "x": -185, "y": 624.0000305175781, "bigg_id": "cpd00078", "name": "Succinyl-CoA", "label_x": -257, "label_y": 654.0000305175781, "node_is_primary": false}, "24": {"node_type": "metabolite", "x": -185, "y": 834.0000305175781, "bigg_id": "cpd00036", "name": "Succinate", "label_x": -257, "label_y": 864.0000305175781, "node_is_primary": false}, "25": {"node_type": "metabolite", "x": -105, "y": 904.0000305175781, "bigg_id": "cpd01507", "name": "3-Oxoadipyl-CoA", "label_x": -80, "label_y": 904.0000305175781, "node_is_primary": true}, "26": {"node_type": "multimarker", "x": -105, "y": 1099.0000305175781}, "27": {"node_type": "midmarker", "x": -105, "y": 1079.0000305175781}, "28": {"node_type": "multimarker", "x": -105, "y": 1059.0000305175781}, "29": {"node_type": "metabolite", "x": -185, "y": 1184.0000305175781, "bigg_id": "cpd00022", "name": "Acetyl-CoA", "label_x": -257, "label_y": 1214.0000305175781, "node_is_primary": false}, "30": {"node_type": "metabolite", "x": -105, "y": 1254.0000305175781, "bigg_id": "cpd00078", "name": "Succinyl-CoA", "label_x": -80, "label_y": 1254.0000305175781, "node_is_primary": true}, "31": {"node_type": "metabolite", "x": -185, "y": 974.0000305175781, "bigg_id": "cpd00010", "name": "CoA", "label_x": -257, "label_y": 1004.0000305175781, "node_is_primary": false}, "32": {"node_type": "metabolite", "x": -189, "y": -749.0000762939453, "bigg_id": "cpd00007", "name": "O2", "label_x": -164, "label_y": -749.0000762939453, "node_is_primary": false}, "33": {"node_type": "multimarker", "x": -105, "y": -711.0000762939453}, "34": {"node_type": "midmarker", "x": -105, "y": -691.0000762939453}, "35": {"node_type": "multimarker", "x": -105, "y": -671.0000762939453}, "36": {"node_type": "metabolite", "x": -104, "y": -868.0000762939453, "bigg_id": "cpd00197", "name": "Protocatechuate", "label_x": -176, "label_y": -838.0000762939453, "node_is_primary": true}, "37": {"node_type": "metabolite", "x": -185, "y": -586.0000762939453, "bigg_id": "cpd00067", "name": "H+", "label_x": -257, "label_y": -556.0000762939453, "node_is_primary": false}, "38": {"node_type": "metabolite", "x": -105, "y": -516.0000762939453, "bigg_id": "cpd00856", "name": "3-Carboxy-cis,cis-muconate", "label_x": -80, "label_y": -516.0000762939453, "node_is_primary": true}, "39": {"node_type": "multimarker", "x": -105, "y": -321.0000762939453}, "40": {"node_type": "midmarker", "x": -105, "y": -341.0000762939453}, "41": {"node_type": "multimarker", "x": -105, "y": -361.0000762939453}, "42": {"node_type": "metabolite", "x": -105, "y": -166.0000762939453, "bigg_id": "cpd00938", "name": "4-Carboxymuconolactone", "label_x": -80, "label_y": -166.0000762939453, "node_is_primary": true}, "43": {"node_type": "metabolite", "x": -185, "y": -446.0000762939453, "bigg_id": "cpd00067", "name": "H+", "label_x": -257, "label_y": -416.0000762939453, "node_is_primary": false}}, "text_labels": {}, "canvas": {"x": -1500, "y": -1165.9999389648438, "width": 4500, "height": 3032.9999389648438}}]

In [2406]:
map2 = [{"map_name": "new_map", "map_id": "koev5uH2oDJI", "map_description": "", "homepage": "https://escher.github.io", "schema": "https://escher.github.io/escher/jsonschema/1-0-0#"}, {"reactions": {"2": {"name": "rxn00529", "bigg_id": "rxn00529", "reversibility": false, "label_x": -281.415096282959, "label_y": 3685.933349609375, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00067", "coefficient": -1}, {"bigg_id": "cpd00069", "coefficient": -1}, {"bigg_id": "cpd00011", "coefficient": 1}, {"bigg_id": "cpd00374", "coefficient": 1}], "segments": {"20": {"from_node_id": "23", "to_node_id": "24", "b1": null, "b2": null}, "21": {"from_node_id": "25", "to_node_id": "24", "b1": null, "b2": null}, "22": {"from_node_id": "23", "to_node_id": "22", "b1": {"x": -296.415096282959, "y": 3613.933349609375}, "b2": {"x": -325.31552332332205, "y": 3594.122337857818}}, "23": {"from_node_id": "23", "to_node_id": "51", "b1": {"x": -296.415096282959, "y": 3609.933349609375}, "b2": {"x": -297.07182759056445, "y": 3577.757838379882}}, "24": {"from_node_id": "25", "to_node_id": "27", "b1": {"x": -296.415096282959, "y": 3741.933349609375}, "b2": {"x": -314.415096282959, "y": 3747.183349609375}}, "25": {"from_node_id": "25", "to_node_id": "28", "b1": {"x": -296.415096282959, "y": 3737.933349609375}, "b2": {"x": -296.415096282959, "y": 3792.183349609375}}}}, "5": {"name": "rxn01315", "bigg_id": "rxn01315", "reversibility": false, "label_x": -429.9999694824219, "label_y": 3362, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00066", "coefficient": -1}, {"bigg_id": "cpd00233", "coefficient": -1}, {"bigg_id": "cpd00001", "coefficient": 1}, {"bigg_id": "cpd00069", "coefficient": 1}, {"bigg_id": "cpd00231", "coefficient": 1}], "segments": {"39": {"from_node_id": "45", "to_node_id": "46", "b1": null, "b2": null}, "40": {"from_node_id": "47", "to_node_id": "46", "b1": null, "b2": null}, "41": {"from_node_id": "45", "to_node_id": "44", "b1": {"x": -444.9999694824219, "y": 3290}, "b2": {"x": -365.62347148247534, "y": 3294.9100112440465}}, "42": {"from_node_id": "45", "to_node_id": "48", "b1": {"x": -444.9999694824219, "y": 3286}, "b2": {"x": -372.9999694824219, "y": 3217.75}}, "43": {"from_node_id": "45", "to_node_id": "58", "b1": {"x": -444.9999694824219, "y": 3286}, "b2": {"x": -380.712284068844, "y": 3257.176536223082}}, "44": {"from_node_id": "47", "to_node_id": "50", "b1": {"x": -444.9999694824219, "y": 3418}, "b2": {"x": -413.8515432467593, "y": 3436.943814712415}}, "45": {"from_node_id": "47", "to_node_id": "51", "b1": {"x": -444.9999694824219, "y": 3414}, "b2": {"x": -347.71351285429307, "y": 3473.5402420932714}}, "46": {"from_node_id": "47", "to_node_id": "52", "b1": {"x": -444.9999694824219, "y": 3418}, "b2": {"x": -466.5417785644531, "y": 3398.45751953125}}}}, "6": {"name": "rxn04989", "bigg_id": "rxn04989", "reversibility": false, "label_x": -138.99998474121094, "label_y": 3365, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00066", "coefficient": -1}, {"bigg_id": "cpd00233", "coefficient": -1}, {"bigg_id": "cpd00069", "coefficient": 1}, {"bigg_id": "cpd03897", "coefficient": 1}], "segments": {"47": {"from_node_id": "54", "to_node_id": "55", "b1": null, "b2": null}, "48": {"from_node_id": "56", "to_node_id": "55", "b1": null, "b2": null}, "49": {"from_node_id": "54", "to_node_id": "44", "b1": {"x": -153.99998474121094, "y": 3293}, "b2": {"x": -250.0269092129114, "y": 3290.3398453564405}}, "50": {"from_node_id": "54", "to_node_id": "48", "b1": {"x": -153.99998474121094, "y": 3289}, "b2": {"x": -235.99063828252656, "y": 3212.0165681119015}}, "51": {"from_node_id": "54", "to_node_id": "58", "b1": {"x": -153.99998474121094, "y": 3289}, "b2": {"x": -246.99998474121094, "y": 3268.75}}, "52": {"from_node_id": "56", "to_node_id": "51", "b1": {"x": -153.9999847412109, "y": 3417}, "b2": {"x": -256.1537730575452, "y": 3462.302488749817}}, "53": {"from_node_id": "56", "to_node_id": "60", "b1": {"x": -153.9999847412109, "y": 3421}, "b2": {"x": -151.52375635452614, "y": 3421.5379626151002}}}}, "11": {"name": "rxn02885", "bigg_id": "rxn02885", "reversibility": false, "label_x": 324.974365234375, "label_y": 1608.721923828125, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd01960", "coefficient": -1}, {"bigg_id": "cpd03326", "coefficient": 1}], "segments": {"77": {"from_node_id": "89", "to_node_id": "90", "b1": null, "b2": null}, "78": {"from_node_id": "91", "to_node_id": "90", "b1": null, "b2": null}, "79": {"from_node_id": "89", "to_node_id": "88", "b1": {"x": 309.974365234375, "y": 1536.721923828125}, "b2": {"x": 309.974365234375, "y": 1482.471923828125}}, "80": {"from_node_id": "91", "to_node_id": "92", "b1": {"x": 309.974365234375, "y": 1660.721923828125}, "b2": {"x": 309.974365234375, "y": 1714.971923828125}}}}, "12": {"name": "rxn01203", "bigg_id": "rxn01203", "reversibility": false, "label_x": 326.5580749511719, "label_y": 909.973876953125, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00020", "coefficient": -1}, {"bigg_id": "cpd00199", "coefficient": -1}, {"bigg_id": "cpd03708", "coefficient": 1}], "segments": {"81": {"from_node_id": "94", "to_node_id": "95", "b1": null, "b2": null}, "82": {"from_node_id": "96", "to_node_id": "95", "b1": null, "b2": null}, "83": {"from_node_id": "94", "to_node_id": "93", "b1": {"x": 311.5580749511719, "y": 837.973876953125}, "b2": {"x": 311.5580749511719, "y": 783.723876953125}}, "84": {"from_node_id": "94", "to_node_id": "97", "b1": {"x": 311.5580749511719, "y": 833.973876953125}, "b2": {"x": 293.5580749511719, "y": 828.723876953125}}, "85": {"from_node_id": "96", "to_node_id": "157", "b1": {"x": 311.5580749511719, "y": 961.973876953125}, "b2": {"x": 311.5580749511719, "y": 1016.223876953125}}}}, "13": {"name": "rxn03040", "bigg_id": "rxn03040", "reversibility": false, "label_x": 322.5995780823941, "label_y": 2307.976552327954, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd02572", "coefficient": -1}, {"bigg_id": "cpd02507", "coefficient": 1}], "segments": {"86": {"from_node_id": "100", "to_node_id": "101", "b1": null, "b2": null}, "87": {"from_node_id": "102", "to_node_id": "101", "b1": null, "b2": null}, "88": {"from_node_id": "100", "to_node_id": "99", "b1": {"x": 307.40008368104947, "y": 2359.9762313753777}, "b2": {"x": 307.22552607987257, "y": 2414.2259505418756}}, "89": {"from_node_id": "102", "to_node_id": "103", "b1": {"x": 307.7990724837407, "y": 2235.976873280527}, "b2": {"x": 307.9736300849175, "y": 2181.7271541140317}}}}, "16": {"name": "rxn00473", "bigg_id": "rxn00473", "reversibility": false, "label_x": -1304.9973599225507, "label_y": 1074.2688804149184, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd00065", "coefficient": -1}, {"bigg_id": "cpd00013", "coefficient": 1}, {"bigg_id": "cpd00020", "coefficient": 1}, {"bigg_id": "cpd00359", "coefficient": 1}], "segments": {"104": {"from_node_id": "121", "to_node_id": "122", "b1": null, "b2": null}, "105": {"from_node_id": "123", "to_node_id": "122", "b1": null, "b2": null}, "106": {"from_node_id": "121", "to_node_id": "120", "b1": {"x": -1188.9996379674265, "y": 1094.80036171738}, "b2": {"x": -1171.0852453698271, "y": 1111.6084345317959}}, "107": {"from_node_id": "121", "to_node_id": "124", "b1": {"x": -1184.9997849380625, "y": 1094.8346508336676}, "b2": {"x": -1147.5491565343336, "y": 1095.757860475716}}, "108": {"from_node_id": "123", "to_node_id": "125", "b1": {"x": -1316.9949349070403, "y": 1093.7031099961705}, "b2": {"x": -1322.3990430313745, "y": 1111.6574441631778}}, "109": {"from_node_id": "123", "to_node_id": "126", "b1": {"x": -1316.9949349070403, "y": 1093.7031099961705}, "b2": {"x": -1322.0904409847844, "y": 1075.658766898911}}, "110": {"from_node_id": "123", "to_node_id": "127", "b1": {"x": -1312.9950818776774, "y": 1093.7373991124582}, "b2": {"x": -1367.2430885884137, "y": 1093.2723529728069}}}}, "17": {"name": "rxn00478", "bigg_id": "rxn00478", "reversibility": false, "label_x": -1061.9866085871188, "label_y": 1278.7605304587546, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00065", "coefficient": -1}, {"bigg_id": "cpd01749", "coefficient": 1}], "segments": {"111": {"from_node_id": "128", "to_node_id": "129", "b1": null, "b2": null}, "112": {"from_node_id": "130", "to_node_id": "129", "b1": null, "b2": null}, "113": {"from_node_id": "128", "to_node_id": "131", "b1": {"x": -1076.9866085871188, "y": 1202.7605304587546}, "b2": {"x": -1069.9717517278964, "y": 1184.730918758541}}, "114": {"from_node_id": "128", "to_node_id": "124", "b1": {"x": -1076.9866085871188, "y": 1206.7605304587546}, "b2": {"x": -1076.9866085871188, "y": 1152.5105304587546}}, "115": {"from_node_id": "130", "to_node_id": "132", "b1": {"x": -1076.9866085871188, "y": 1330.7605304587541}, "b2": {"x": -1076.9866085871188, "y": 1385.0105304587541}}}}, "18": {"name": "rxn00479", "bigg_id": "rxn00479", "reversibility": false, "label_x": -951.986608587119, "label_y": 1053.7605304587546, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00065", "coefficient": -1}, {"bigg_id": "cpd00001", "coefficient": 1}, {"bigg_id": "cpd00011", "coefficient": 1}, {"bigg_id": "cpd01747", "coefficient": 1}], "segments": {"116": {"from_node_id": "133", "to_node_id": "134", "b1": null, "b2": null}, "117": {"from_node_id": "135", "to_node_id": "134", "b1": null, "b2": null}, "118": {"from_node_id": "133", "to_node_id": "136", "b1": {"x": -967.986608587119, "y": 1093.7605304587546}, "b2": {"x": -973.3520070834388, "y": 1078.2114436625643}}, "119": {"from_node_id": "133", "to_node_id": "124", "b1": {"x": -963.986608587119, "y": 1093.7605304587546}, "b2": {"x": -1018.236608587119, "y": 1093.7605304587546}}, "120": {"from_node_id": "135", "to_node_id": "137", "b1": {"x": -835.986608587119, "y": 1093.7605304587546}, "b2": {"x": -830.736608587119, "y": 1075.7605304587546}}, "121": {"from_node_id": "135", "to_node_id": "138", "b1": {"x": -835.986608587119, "y": 1093.7605304587546}, "b2": {"x": -830.736608587119, "y": 1111.7605304587546}}, "122": {"from_node_id": "135", "to_node_id": "139", "b1": {"x": -839.986608587119, "y": 1093.7605304587546}, "b2": {"x": -785.736608587119, "y": 1093.7605304587546}}}}, "19": {"name": "rxn01437", "bigg_id": "rxn01437", "reversibility": false, "label_x": -1061.9866085871188, "label_y": 1628.7605304587541, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd01749", "coefficient": -1}, {"bigg_id": "cpd00047", "coefficient": 1}, {"bigg_id": "cpd00067", "coefficient": 1}, {"bigg_id": "cpd00275", "coefficient": 1}], "segments": {"123": {"from_node_id": "140", "to_node_id": "141", "b1": null, "b2": null}, "124": {"from_node_id": "142", "to_node_id": "141", "b1": null, "b2": null}, "125": {"from_node_id": "140", "to_node_id": "143", "b1": {"x": -1076.9866085871188, "y": 1552.7605304587541}, "b2": {"x": -1094.9866085871188, "y": 1547.5105304587541}}, "126": {"from_node_id": "140", "to_node_id": "132", "b1": {"x": -1076.9866085871188, "y": 1556.7605304587541}, "b2": {"x": -1076.9866085871188, "y": 1502.5105304587541}}, "127": {"from_node_id": "142", "to_node_id": "144", "b1": {"x": -1076.9866085871188, "y": 1684.7605304587541}, "b2": {"x": -1058.9866085871188, "y": 1690.0105304587541}}, "128": {"from_node_id": "142", "to_node_id": "145", "b1": {"x": -1076.9866085871188, "y": 1684.7605304587541}, "b2": {"x": -1094.9866085871188, "y": 1690.0105304587541}}, "129": {"from_node_id": "142", "to_node_id": "146", "b1": {"x": -1076.9866085871188, "y": 1680.7605304587541}, "b2": {"x": -1076.9866085871188, "y": 1735.0105304587541}}}}, "20": {"name": "rxn02222", "bigg_id": "rxn02222", "reversibility": false, "label_x": -601.986608587119, "label_y": 1053.7605304587546, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd01747", "coefficient": -1}, {"bigg_id": "cpd00013", "coefficient": 1}, {"bigg_id": "cpd00703", "coefficient": 1}], "segments": {"130": {"from_node_id": "147", "to_node_id": "148", "b1": null, "b2": null}, "131": {"from_node_id": "149", "to_node_id": "148", "b1": null, "b2": null}, "132": {"from_node_id": "147", "to_node_id": "150", "b1": {"x": -617.986608587119, "y": 1093.7605304587546}, "b2": {"x": -623.236608587119, "y": 1111.7605304587546}}, "133": {"from_node_id": "147", "to_node_id": "139", "b1": {"x": -613.986608587119, "y": 1093.7605304587546}, "b2": {"x": -668.236608587119, "y": 1093.7605304587546}}, "134": {"from_node_id": "149", "to_node_id": "151", "b1": {"x": -485.986608587119, "y": 1093.7605304587546}, "b2": {"x": -480.736608587119, "y": 1111.7605304587546}}, "135": {"from_node_id": "149", "to_node_id": "152", "b1": {"x": -489.986608587119, "y": 1093.7605304587546}, "b2": {"x": -435.736608587119, "y": 1093.7605304587546}}}}, "21": {"name": "rxn02883", "bigg_id": "rxn02883", "reversibility": false, "label_x": 324.974365234375, "label_y": 1258.721923828125, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd01960", "coefficient": -1}, {"bigg_id": "cpd03708", "coefficient": 1}], "segments": {"136": {"from_node_id": "153", "to_node_id": "154", "b1": null, "b2": null}, "137": {"from_node_id": "155", "to_node_id": "154", "b1": null, "b2": null}, "138": {"from_node_id": "153", "to_node_id": "156", "b1": {"x": 309.974365234375, "y": 1314.721923828125}, "b2": {"x": 327.974365234375, "y": 1319.971923828125}}, "139": {"from_node_id": "153", "to_node_id": "88", "b1": {"x": 309.974365234375, "y": 1310.721923828125}, "b2": {"x": 309.974365234375, "y": 1364.971923828125}}, "140": {"from_node_id": "155", "to_node_id": "157", "b1": {"x": 309.974365234375, "y": 1186.721923828125}, "b2": {"x": 309.974365234375, "y": 1132.471923828125}}}}, "22": {"name": "rxn03041", "bigg_id": "rxn03041", "reversibility": false, "label_x": 323.72575615450455, "label_y": 1957.9783641570011, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00067", "coefficient": -1}, {"bigg_id": "cpd02507", "coefficient": -1}, {"bigg_id": "cpd00011", "coefficient": 1}, {"bigg_id": "cpd03326", "coefficient": 1}], "segments": {"141": {"from_node_id": "158", "to_node_id": "159", "b1": null, "b2": null}, "142": {"from_node_id": "160", "to_node_id": "159", "b1": null, "b2": null}, "143": {"from_node_id": "158", "to_node_id": "161", "b1": {"x": 308.51339114662073, "y": 2013.9780224978067}, "b2": {"x": 326.49640529576027, "y": 2019.285913049794}}, "144": {"from_node_id": "158", "to_node_id": "103", "b1": {"x": 308.52626175315777, "y": 2009.9780432044263}, "b2": {"x": 308.35170415198377, "y": 2064.227762370921}}, "145": {"from_node_id": "160", "to_node_id": "162", "b1": {"x": 308.9381211623884, "y": 1881.9787058161915}, "b2": {"x": 326.9549206536907, "y": 1876.7866507230478}}, "146": {"from_node_id": "160", "to_node_id": "92", "b1": {"x": 308.92525055584855, "y": 1885.9786851095737}, "b2": {"x": 309.09980815702767, "y": 1831.728965943074}}}}, "23": {"name": "rxn03055", "bigg_id": "rxn03055", "reversibility": false, "label_x": 322.0364890463395, "label_y": 2657.9756464134284, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd00003", "coefficient": -1}, {"bigg_id": "cpd02828", "coefficient": -1}, {"bigg_id": "cpd00004", "coefficient": 1}, {"bigg_id": "cpd00067", "coefficient": 2}, {"bigg_id": "cpd02572", "coefficient": 1}], "segments": {"147": {"from_node_id": "164", "to_node_id": "165", "b1": null, "b2": null}, "148": {"from_node_id": "166", "to_node_id": "165", "b1": null, "b2": null}, "149": {"from_node_id": "164", "to_node_id": "167", "b1": {"x": 307.0364890463395, "y": 2713.9756464134284}, "b2": {"x": 288.2847638209299, "y": 2722.7944144559838}}, "150": {"from_node_id": "164", "to_node_id": "168", "b1": {"x": 307.0364890463395, "y": 2709.9756464134284}, "b2": {"x": 332.13065155317975, "y": 2731.3857926041355}}, "151": {"from_node_id": "164", "to_node_id": "169", "b1": {"x": 307.0364890463395, "y": 2713.9756464134284}, "b2": {"x": 307.37944521633926, "y": 2747.0328895338043}}, "152": {"from_node_id": "166", "to_node_id": "170", "b1": {"x": 307.0364890463395, "y": 2581.9756464134284}, "b2": {"x": 325.0364890463395, "y": 2576.7256464134284}}, "153": {"from_node_id": "166", "to_node_id": "171", "b1": {"x": 307.0364890463395, "y": 2581.9756464134284}, "b2": {"x": 289.0364890463395, "y": 2576.7256464134284}}, "154": {"from_node_id": "166", "to_node_id": "99", "b1": {"x": 307.0364890463395, "y": 2585.9756464134284}, "b2": {"x": 307.0364890463395, "y": 2531.7256464134284}}}}, "24": {"name": "rxn02366", "bigg_id": "rxn02366", "reversibility": false, "label_x": 321.0352378256364, "label_y": 3002.6109003196784, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00854", "coefficient": -1}, {"bigg_id": "cpd00067", "coefficient": 1}, {"bigg_id": "cpd02828", "coefficient": 1}], "segments": {"155": {"from_node_id": "172", "to_node_id": "173", "b1": null, "b2": null}, "156": {"from_node_id": "174", "to_node_id": "173", "b1": null, "b2": null}, "157": {"from_node_id": "172", "to_node_id": "175", "b1": {"x": 306.0352378256364, "y": 3058.6109003196784}, "b2": {"x": 288.0352378256364, "y": 3063.8609003196784}}, "158": {"from_node_id": "172", "to_node_id": "176", "b1": {"x": 306.0352378256364, "y": 3054.6109003196784}, "b2": {"x": 306.0352378256364, "y": 3108.8609003196784}}, "159": {"from_node_id": "174", "to_node_id": "177", "b1": {"x": 306.0352378256364, "y": 2926.6109003196784}, "b2": {"x": 288.0352378256364, "y": 2921.3609003196784}}, "160": {"from_node_id": "174", "to_node_id": "169", "b1": {"x": 306.0352378256364, "y": 2930.6109003196784}, "b2": {"x": 306.0352378256364, "y": 2876.3609003196784}}}}, "26": {"name": "rxn01945", "bigg_id": "rxn01945", "reversibility": false, "label_x": 321.0352378256364, "label_y": 3352.6109003196784, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00004", "coefficient": -1}, {"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00067", "coefficient": -1}, {"bigg_id": "cpd00489", "coefficient": -1}, {"bigg_id": "cpd00001", "coefficient": 1}, {"bigg_id": "cpd00003", "coefficient": 1}, {"bigg_id": "cpd00854", "coefficient": 1}], "segments": {"170": {"from_node_id": "187", "to_node_id": "188", "b1": null, "b2": null}, "171": {"from_node_id": "189", "to_node_id": "188", "b1": null, "b2": null}, "172": {"from_node_id": "187", "to_node_id": "190", "b1": {"x": 306.0352378256364, "y": 3404.6109003196784}, "b2": {"x": 274.0745093967104, "y": 3419.580608855168}}, "173": {"from_node_id": "187", "to_node_id": "191", "b1": {"x": 306.0352378256364, "y": 3408.6109003196784}, "b2": {"x": 334.80779031587076, "y": 3415.6563104759284}}, "174": {"from_node_id": "187", "to_node_id": "192", "b1": {"x": 306.0352378256364, "y": 3408.6109003196784}, "b2": {"x": 335.1984705980981, "y": 3391.879956347313}}, "175": {"from_node_id": "187", "to_node_id": "193", "b1": {"x": 306.0352378256364, "y": 3408.6109003196784}, "b2": {"x": 304.9610332749151, "y": 3388.3003976261084}}, "176": {"from_node_id": "189", "to_node_id": "194", "b1": {"x": 306.0352378256364, "y": 3276.6109003196784}, "b2": {"x": 324.0352378256364, "y": 3271.3609003196784}}, "177": {"from_node_id": "189", "to_node_id": "195", "b1": {"x": 306.0352378256364, "y": 3276.6109003196784}, "b2": {"x": 288.0352378256364, "y": 3271.3609003196784}}, "178": {"from_node_id": "189", "to_node_id": "176", "b1": {"x": 306.0352378256364, "y": 3280.6109003196784}, "b2": {"x": 306.0352378256364, "y": 3226.3609003196784}}}}, "27": {"name": "rxn01943", "bigg_id": "rxn01943", "reversibility": false, "label_x": 320.8282065756364, "label_y": 3700.5867303978034, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd00003", "coefficient": -1}, {"bigg_id": "cpd02361", "coefficient": -1}, {"bigg_id": "cpd00004", "coefficient": 1}, {"bigg_id": "cpd00067", "coefficient": 2}, {"bigg_id": "cpd00489", "coefficient": 1}], "segments": {"179": {"from_node_id": "196", "to_node_id": "197", "b1": null, "b2": null}, "180": {"from_node_id": "198", "to_node_id": "197", "b1": null, "b2": null}, "181": {"from_node_id": "196", "to_node_id": "199", "b1": {"x": 305.8282065756364, "y": 3756.5867303978034}, "b2": {"x": 323.8282065756364, "y": 3761.8367303978034}}, "182": {"from_node_id": "196", "to_node_id": "200", "b1": {"x": 305.8282065756364, "y": 3752.5867303978034}, "b2": {"x": 277.2955844173661, "y": 3770.5223872560846}}, "183": {"from_node_id": "196", "to_node_id": "201", "b1": {"x": 305.8282065756364, "y": 3756.5867303978034}, "b2": {"x": 308.54694639029884, "y": 3773.498722425746}}, "184": {"from_node_id": "198", "to_node_id": "202", "b1": {"x": 305.8282065756364, "y": 3624.5867303978034}, "b2": {"x": 323.8282065756364, "y": 3619.3367303978034}}, "185": {"from_node_id": "198", "to_node_id": "203", "b1": {"x": 305.8282065756364, "y": 3624.5867303978034}, "b2": {"x": 287.8282065756364, "y": 3619.3367303978034}}, "186": {"from_node_id": "198", "to_node_id": "193", "b1": {"x": 305.8282065756364, "y": 3628.5867303978034}, "b2": {"x": 305.8282065756364, "y": 3574.3367303978034}}}}, "28": {"name": "rxn01715", "bigg_id": "rxn01715", "reversibility": false, "label_x": -76.25748825073242, "label_y": 3812.728759765625, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00374", "coefficient": -1}, {"bigg_id": "cpd00013", "coefficient": 1}, {"bigg_id": "cpd00025", "coefficient": 1}, {"bigg_id": "cpd02361", "coefficient": 1}], "segments": {"187": {"from_node_id": "204", "to_node_id": "205", "b1": null, "b2": null}, "188": {"from_node_id": "206", "to_node_id": "205", "b1": null, "b2": null}, "189": {"from_node_id": "204", "to_node_id": "207", "b1": {"x": -79.68950176239014, "y": 3850.933349609375}, "b2": {"x": -84.93950176239014, "y": 3832.933349609375}}, "190": {"from_node_id": "204", "to_node_id": "208", "b1": {"x": -79.68950176239014, "y": 3850.933349609375}, "b2": {"x": -84.93950176239014, "y": 3868.933349609375}}, "191": {"from_node_id": "204", "to_node_id": "28", "b1": {"x": -75.68950176239014, "y": 3850.933349609375}, "b2": {"x": -237.66509628295898, "y": 3850.933349609375}}, "192": {"from_node_id": "206", "to_node_id": "209", "b1": {"x": 52.31049823760986, "y": 3850.933349609375}, "b2": {"x": 57.56049823760986, "y": 3832.933349609375}}, "193": {"from_node_id": "206", "to_node_id": "210", "b1": {"x": 52.31049823760986, "y": 3850.933349609375}, "b2": {"x": 57.56049823760986, "y": 3868.933349609375}}, "194": {"from_node_id": "206", "to_node_id": "201", "b1": {"x": 48.31049823760986, "y": 3850.933349609375}, "b2": {"x": 249.78547382354736, "y": 3847.342529296875}}}}, "29": {"name": "rxn00728", "bigg_id": "rxn00728", "reversibility": false, "label_x": -1061.9866085871188, "label_y": 1978.7605304587541, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd00275", "coefficient": -1}, {"bigg_id": "cpd00035", "coefficient": 1}, {"bigg_id": "cpd00067", "coefficient": 1}, {"bigg_id": "cpd00093", "coefficient": 1}], "segments": {"195": {"from_node_id": "212", "to_node_id": "213", "b1": null, "b2": null}, "196": {"from_node_id": "214", "to_node_id": "213", "b1": null, "b2": null}, "197": {"from_node_id": "212", "to_node_id": "215", "b1": {"x": -1076.9866085871188, "y": 1902.7605304587541}, "b2": {"x": -1094.9866085871188, "y": 1897.5105304587541}}, "198": {"from_node_id": "212", "to_node_id": "146", "b1": {"x": -1076.9866085871188, "y": 1906.7605304587541}, "b2": {"x": -1076.9866085871188, "y": 1852.5105304587541}}, "199": {"from_node_id": "214", "to_node_id": "216", "b1": {"x": -1076.9866085871188, "y": 2034.7605304587541}, "b2": {"x": -1058.9866085871188, "y": 2040.0105304587541}}, "200": {"from_node_id": "214", "to_node_id": "217", "b1": {"x": -1076.9866085871188, "y": 2034.7605304587541}, "b2": {"x": -1094.9866085871188, "y": 2040.0105304587541}}, "201": {"from_node_id": "214", "to_node_id": "218", "b1": {"x": -1076.9866085871188, "y": 2030.7605304587541}, "b2": {"x": -1076.9866085871188, "y": 2085.010530458754}}}}, "30": {"name": "rxn00594", "bigg_id": "rxn00594", "reversibility": false, "label_x": -1300.9879513605565, "label_y": 2373.225862490004, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00004", "coefficient": -1}, {"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00067", "coefficient": -3}, {"bigg_id": "cpd00093", "coefficient": -1}, {"bigg_id": "cpd00003", "coefficient": 1}, {"bigg_id": "cpd00011", "coefficient": 1}, {"bigg_id": "cpd00013", "coefficient": 1}, {"bigg_id": "cpd00077", "coefficient": 1}], "segments": {"202": {"from_node_id": "219", "to_node_id": "220", "b1": null, "b2": null}, "203": {"from_node_id": "221", "to_node_id": "220", "b1": null, "b2": null}, "204": {"from_node_id": "219", "to_node_id": "222", "b1": {"x": -1315.9879513605563, "y": 2297.225862490004}, "b2": {"x": -1297.9879513605563, "y": 2291.975862490004}}, "205": {"from_node_id": "219", "to_node_id": "223", "b1": {"x": -1315.9879513605563, "y": 2297.225862490004}, "b2": {"x": -1333.9879513605563, "y": 2291.975862490004}}, "206": {"from_node_id": "219", "to_node_id": "224", "b1": {"x": -1315.9879513605563, "y": 2297.225862490004}, "b2": {"x": -1351.9879513605563, "y": 2306.975862490004}}, "207": {"from_node_id": "219", "to_node_id": "218", "b1": {"x": -1315.9879513605563, "y": 2301.225862490004}, "b2": {"x": -1076.9866085871188, "y": 2202.510530458754}}, "208": {"from_node_id": "221", "to_node_id": "225", "b1": {"x": -1315.9879513605563, "y": 2425.225862490004}, "b2": {"x": -1315.9878292902438, "y": 2498.929475771254}}, "209": {"from_node_id": "221", "to_node_id": "226", "b1": {"x": -1315.9879513605563, "y": 2429.225862490004}, "b2": {"x": -1297.9879513605563, "y": 2434.475862490004}}, "210": {"from_node_id": "221", "to_node_id": "227", "b1": {"x": -1315.9879513605563, "y": 2429.225862490004}, "b2": {"x": -1333.9879513605563, "y": 2434.475862490004}}, "211": {"from_node_id": "221", "to_node_id": "228", "b1": {"x": -1315.9879513605563, "y": 2429.225862490004}, "b2": {"x": -1066.3245137256345, "y": 2418.264923610896}}}}, "32": {"name": "rxn00595", "bigg_id": "rxn00595", "reversibility": false, "label_x": -812.2493432021806, "label_y": 2366.9565832167896, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00005", "coefficient": -1}, {"bigg_id": "cpd00007", "coefficient": -1}, {"bigg_id": "cpd00067", "coefficient": -3}, {"bigg_id": "cpd00093", "coefficient": -1}, {"bigg_id": "cpd00006", "coefficient": 1}, {"bigg_id": "cpd00011", "coefficient": 1}, {"bigg_id": "cpd00013", "coefficient": 1}, {"bigg_id": "cpd00077", "coefficient": 1}], "segments": {"222": {"from_node_id": "239", "to_node_id": "240", "b1": null, "b2": null}, "223": {"from_node_id": "241", "to_node_id": "240", "b1": null, "b2": null}, "224": {"from_node_id": "239", "to_node_id": "242", "b1": {"x": -829.8266137580108, "y": 2291.0069230472177}, "b2": {"x": -812.0453529605238, "y": 2285.0580353821397}}, "225": {"from_node_id": "239", "to_node_id": "243", "b1": {"x": -829.8266137580108, "y": 2291.0069230472177}, "b2": {"x": -848.0178948711996, "y": 2286.4638193216824}}, "226": {"from_node_id": "239", "to_node_id": "244", "b1": {"x": -829.8266137580108, "y": 2291.0069230472177}, "b2": {"x": -865.4184225183944, "y": 2302.1552704209034}}, "227": {"from_node_id": "239", "to_node_id": "218", "b1": {"x": -829.6704155425068, "y": 2295.0038721484043}, "b2": {"x": -1076.9866085871188, "y": 2202.510530458754}}, "228": {"from_node_id": "241", "to_node_id": "245", "b1": {"x": -824.8282708618563, "y": 2418.9092942851753}, "b2": {"x": -828.4807402674264, "y": 2467.7810918445603}}, "229": {"from_node_id": "241", "to_node_id": "246", "b1": {"x": -824.67207264635, "y": 2422.906243386362}, "b2": {"x": -806.4807915331626, "y": 2427.4493471118963}}, "230": {"from_node_id": "241", "to_node_id": "247", "b1": {"x": -824.67207264635, "y": 2422.906243386362}, "b2": {"x": -842.4533334438379, "y": 2428.8551310514404}}, "231": {"from_node_id": "241", "to_node_id": "228", "b1": {"x": -824.67207264635, "y": 2422.906243386362}, "b2": {"x": -1068.6387556173797, "y": 2407.098521944588}}}}, "33": {"name": "rxn00487", "bigg_id": "rxn00487", "reversibility": false, "label_x": -295.9999694824219, "label_y": 3019, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00001", "coefficient": -1}, {"bigg_id": "cpd00003", "coefficient": -1}, {"bigg_id": "cpd00066", "coefficient": -1}, {"bigg_id": "cpd00004", "coefficient": 1}, {"bigg_id": "cpd00013", "coefficient": 1}, {"bigg_id": "cpd00067", "coefficient": 1}, {"bigg_id": "cpd00143", "coefficient": 1}], "segments": {"232": {"from_node_id": "249", "to_node_id": "250", "b1": null, "b2": null}, "233": {"from_node_id": "251", "to_node_id": "250", "b1": null, "b2": null}, "234": {"from_node_id": "249", "to_node_id": "252", "b1": {"x": -310.9999694824219, "y": 3075}, "b2": {"x": -328.9999694824219, "y": 3080.25}}, "235": {"from_node_id": "249", "to_node_id": "253", "b1": {"x": -310.9999694824219, "y": 3075}, "b2": {"x": -292.9999694824219, "y": 3080.25}}, "236": {"from_node_id": "249", "to_node_id": "48", "b1": {"x": -310.9999694824219, "y": 3071}, "b2": {"x": -310.9999694824219, "y": 3125.25}}, "237": {"from_node_id": "251", "to_node_id": "254", "b1": {"x": -310.9999694824219, "y": 2947}, "b2": {"x": -258.5519839553635, "y": 2945.926778038805}}, "238": {"from_node_id": "251", "to_node_id": "255", "b1": {"x": -310.9999694824219, "y": 2943}, "b2": {"x": -328.9999694824219, "y": 2937.75}}, "239": {"from_node_id": "251", "to_node_id": "256", "b1": {"x": -310.9999694824219, "y": 2943}, "b2": {"x": -292.9999694824219, "y": 2937.75}}, "240": {"from_node_id": "251", "to_node_id": "257", "b1": {"x": -310.9999694824219, "y": 2943}, "b2": {"x": -310.8750394305852, "y": 2948.4549024295716}}}}, "34": {"name": "rxn01000", "bigg_id": "rxn01000", "reversibility": false, "label_x": -294.4078063964844, "label_y": 2656.174072265625, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00067", "coefficient": -1}, {"bigg_id": "cpd00219", "coefficient": -1}, {"bigg_id": "cpd00001", "coefficient": 1}, {"bigg_id": "cpd00011", "coefficient": 1}, {"bigg_id": "cpd00143", "coefficient": 1}], "segments": {"241": {"from_node_id": "258", "to_node_id": "259", "b1": null, "b2": null}, "242": {"from_node_id": "260", "to_node_id": "259", "b1": null, "b2": null}, "243": {"from_node_id": "258", "to_node_id": "261", "b1": {"x": -309.4078063964844, "y": 2580.174072265625}, "b2": {"x": -291.4078063964844, "y": 2574.924072265625}}, "244": {"from_node_id": "258", "to_node_id": "262", "b1": {"x": -309.4078063964844, "y": 2584.174072265625}, "b2": {"x": -309.4078063964844, "y": 2529.924072265625}}, "245": {"from_node_id": "260", "to_node_id": "263", "b1": {"x": -309.4078063964844, "y": 2712.174072265625}, "b2": {"x": -327.4078063964844, "y": 2717.424072265625}}, "246": {"from_node_id": "260", "to_node_id": "264", "b1": {"x": -309.4078063964844, "y": 2712.174072265625}, "b2": {"x": -291.4078063964844, "y": 2717.424072265625}}, "247": {"from_node_id": "260", "to_node_id": "257", "b1": {"x": -309.4078063964844, "y": 2708.174072265625}, "b2": {"x": -309.4078063964844, "y": 2762.424072265625}}}}, "35": {"name": "rxn01256", "bigg_id": "rxn01256", "reversibility": false, "label_x": -294.4078063964844, "label_y": 2306.174072265625, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00216", "coefficient": -1}, {"bigg_id": "cpd00219", "coefficient": 1}], "segments": {"248": {"from_node_id": "265", "to_node_id": "266", "b1": null, "b2": null}, "249": {"from_node_id": "267", "to_node_id": "266", "b1": null, "b2": null}, "250": {"from_node_id": "265", "to_node_id": "268", "b1": {"x": -309.4078063964844, "y": 2234.174072265625}, "b2": {"x": -311.2748107910156, "y": 2192.9931640625}}, "251": {"from_node_id": "267", "to_node_id": "262", "b1": {"x": -309.4078063964844, "y": 2358.174072265625}, "b2": {"x": -309.4078063964844, "y": 2412.424072265625}}}}, "37": {"name": "rxn00726", "bigg_id": "rxn00726", "reversibility": false, "label_x": -722.976318359375, "label_y": 2088.642333984375, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00013", "coefficient": -1}, {"bigg_id": "cpd00216", "coefficient": -1}, {"bigg_id": "cpd00001", "coefficient": 1}, {"bigg_id": "cpd00020", "coefficient": 1}, {"bigg_id": "cpd00067", "coefficient": 1}, {"bigg_id": "cpd00093", "coefficient": 1}], "segments": {"260": {"from_node_id": "277", "to_node_id": "278", "b1": null, "b2": null}, "261": {"from_node_id": "279", "to_node_id": "278", "b1": null, "b2": null}, "262": {"from_node_id": "277", "to_node_id": "280", "b1": {"x": -606.976318359375, "y": 2128.642333984375}, "b2": {"x": -601.726318359375, "y": 2110.642333984375}}, "263": {"from_node_id": "277", "to_node_id": "268", "b1": {"x": -610.976318359375, "y": 2128.642333984375}, "b2": {"x": -370.0248107910156, "y": 2134.2431640625}}, "264": {"from_node_id": "279", "to_node_id": "281", "b1": {"x": -738.976318359375, "y": 2128.642333984375}, "b2": {"x": -742.3592529296875, "y": 2126.105224609375}}, "265": {"from_node_id": "279", "to_node_id": "282", "b1": {"x": -738.976318359375, "y": 2128.642333984375}, "b2": {"x": -744.226318359375, "y": 2110.642333984375}}, "266": {"from_node_id": "279", "to_node_id": "283", "b1": {"x": -738.976318359375, "y": 2128.642333984375}, "b2": {"x": -729.226318359375, "y": 2092.642333984375}}, "267": {"from_node_id": "279", "to_node_id": "218", "b1": {"x": -734.976318359375, "y": 2128.642333984375}, "b2": {"x": -1013.26806640625, "y": 2145.445556640625}}}}}, "nodes": {"22": {"node_type": "metabolite", "x": -379.64684677124023, "y": 3571.76904296875, "bigg_id": "cpd00067", "name": "H+", "label_x": -432.56594467163086, "label_y": 3589.477783203125, "node_is_primary": false}, "23": {"node_type": "multimarker", "x": -296.415096282959, "y": 3655.933349609375}, "24": {"node_type": "midmarker", "x": -296.415096282959, "y": 3675.933349609375}, "25": {"node_type": "multimarker", "x": -296.415096282959, "y": 3695.933349609375}, "27": {"node_type": "metabolite", "x": -376.415096282959, "y": 3780.933349609375, "bigg_id": "cpd00011", "name": "CO2", "label_x": -439.56066513061523, "label_y": 3807.391357421875, "node_is_primary": false}, "28": {"node_type": "metabolite", "x": -296.415096282959, "y": 3850.933349609375, "bigg_id": "cpd00374", "name": "Tyramine", "label_x": -276.80138778686523, "label_y": 3831.183837890625, "node_is_primary": true}, "44": {"node_type": "metabolite", "x": -309.9999694824219, "y": 3276, "bigg_id": "cpd00007", "name": "O2", "label_x": -284.9999694824219, "label_y": 3276, "node_is_primary": false}, "45": {"node_type": "multimarker", "x": -444.9999694824219, "y": 3332}, "46": {"node_type": "midmarker", "x": -444.9999694824219, "y": 3352}, "47": {"node_type": "multimarker", "x": -444.9999694824219, "y": 3372}, "48": {"node_type": "metabolite", "x": -310.9999694824219, "y": 3184, "bigg_id": "cpd00066", "name": "L-Phenylalanine", "label_x": -373.9999694824219, "label_y": 3160, "node_is_primary": true}, "50": {"node_type": "metabolite", "x": -442.9190368652344, "y": 3501.272216796875, "bigg_id": "cpd00001", "name": "H2O", "label_x": -504.0672302246094, "label_y": 3510.126708984375, "node_is_primary": false}, "51": {"node_type": "metabolite", "x": -299.7871398925781, "y": 3507.52001953125, "bigg_id": "cpd00069", "name": "L-Tyrosine", "label_x": -274.7871398925781, "label_y": 3507.52001953125, "node_is_primary": true}, "52": {"node_type": "metabolite", "x": -528.5417785644531, "y": 3432.20751953125, "bigg_id": "cpd00231", "name": "Dihydrobiopterin", "label_x": -600.5417785644531, "label_y": 3462.20751953125, "node_is_primary": true}, "54": {"node_type": "multimarker", "x": -153.99998474121094, "y": 3335}, "55": {"node_type": "midmarker", "x": -153.99998474121094, "y": 3355}, "56": {"node_type": "multimarker", "x": -153.99998474121094, "y": 3375}, "58": {"node_type": "metabolite", "x": -308.99998474121094, "y": 3235, "bigg_id": "cpd00233", "name": "Tetrahydrobiopterin", "label_x": -386.99998474121094, "label_y": 3229, "node_is_primary": true}, "60": {"node_type": "metabolite", "x": -85.24536514282227, "y": 3445.832763671875, "bigg_id": "cpd03897", "name": "4a-Hydroxytetrahydrobiopterin", "label_x": -205.7218723297119, "label_y": 3474.03759765625, "node_is_primary": true}, "88": {"node_type": "metabolite", "x": 309.974365234375, "y": 1423.721923828125, "bigg_id": "cpd01960", "name": "2-Oxohept-3-enedioate", "label_x": 334.974365234375, "label_y": 1423.721923828125, "node_is_primary": true}, "89": {"node_type": "multimarker", "x": 309.974365234375, "y": 1578.721923828125}, "90": {"node_type": "midmarker", "x": 309.974365234375, "y": 1598.721923828125}, "91": {"node_type": "multimarker", "x": 309.974365234375, "y": 1618.721923828125}, "92": {"node_type": "metabolite", "x": 309.974365234375, "y": 1773.721923828125, "bigg_id": "cpd03326", "name": "2-Hydroxyhepta-2,4-dienedioate", "label_x": 334.974365234375, "label_y": 1773.721923828125, "node_is_primary": true}, "93": {"node_type": "metabolite", "x": 311.5580749511719, "y": 724.973876953125, "bigg_id": "cpd00020", "name": "Pyruvate", "label_x": 336.5580749511719, "label_y": 724.973876953125, "node_is_primary": true}, "94": {"node_type": "multimarker", "x": 311.5580749511719, "y": 879.973876953125}, "95": {"node_type": "midmarker", "x": 311.5580749511719, "y": 899.973876953125}, "96": {"node_type": "multimarker", "x": 311.5580749511719, "y": 919.973876953125}, "97": {"node_type": "metabolite", "x": 231.55807495117188, "y": 794.973876953125, "bigg_id": "cpd00199", "name": "4-Oxobutanoate", "label_x": 159.55807495117188, "label_y": 824.973876953125, "node_is_primary": true}, "99": {"node_type": "metabolite", "x": 307.0364890463395, "y": 2472.9756464134284, "bigg_id": "cpd02572", "name": "5-Carboxymethyl-2-hydroxymuconate", "label_x": 332.0364890463395, "label_y": 2472.9756464134284, "node_is_primary": true}, "100": {"node_type": "multimarker", "x": 307.53522504970323, "y": 2317.976448794865}, "101": {"node_type": "midmarker", "x": 307.59957808239415, "y": 2297.976552327954}, "102": {"node_type": "multimarker", "x": 307.6639311150883, "y": 2277.9766558610395}, "103": {"node_type": "metabolite", "x": 308.16266711845077, "y": 2122.9774582424725, "bigg_id": "cpd02507", "name": "5-Carboxy-2-oxohept-3-enedioate", "label_x": 333.1626671184508, "label_y": 2122.9774582424725, "node_is_primary": true}, "120": {"node_type": "metabolite", "x": -1166.772869988884, "y": 1170.199951994428, "bigg_id": "cpd00001", "name": "H2O", "label_x": -1141.772869988884, "label_y": 1170.199951994428, "node_is_primary": false}, "121": {"node_type": "multimarker", "x": -1230.9980947757372, "y": 1094.4403259963578}, "122": {"node_type": "midmarker", "x": -1250.9973599225507, "y": 1094.2688804149184}, "123": {"node_type": "multimarker", "x": -1270.996625069366, "y": 1094.0974348334798}, "124": {"node_type": "metabolite", "x": -1076.9866085871188, "y": 1093.7605304587546, "bigg_id": "cpd00065", "name": "L-Tryptophan", "label_x": -1142.1515255793063, "label_y": 1075.9790363181296, "node_is_primary": true}, "125": {"node_type": "metabolite", "x": -1356.679284269084, "y": 1173.3658516996256, "bigg_id": "cpd00013", "name": "NH3", "label_x": -1341.679284269084, "label_y": 1173.3658516996256, "node_is_primary": false}, "126": {"node_type": "metabolite", "x": -1355.3077196175755, "y": 1013.3717305251066, "bigg_id": "cpd00020", "name": "Pyruvate", "label_x": -1388.3077196175755, "label_y": 989.3717305251066, "node_is_primary": false}, "127": {"node_type": "metabolite", "x": -1425.9909299571823, "y": 1092.76873157733, "bigg_id": "cpd00359", "name": "indol", "label_x": -1403.9909299571823, "label_y": 1071.76873157733, "node_is_primary": true}, "128": {"node_type": "multimarker", "x": -1076.9866085871188, "y": 1248.7605304587546}, "129": {"node_type": "midmarker", "x": -1076.9866085871188, "y": 1268.7605304587546}, "130": {"node_type": "multimarker", "x": -1076.9866085871188, "y": 1288.7605304587541}, "131": {"node_type": "metabolite", "x": -1000.986608587119, "y": 1169.7605304587546, "bigg_id": "cpd00007", "name": "O2", "label_x": -976.986608587119, "label_y": 1169.7605304587546, "node_is_primary": false}, "132": {"node_type": "metabolite", "x": -1076.9866085871188, "y": 1443.7605304587541, "bigg_id": "cpd01749", "name": "N-Formylkynurenine", "label_x": -1051.9866085871188, "label_y": 1443.7605304587541, "node_is_primary": true}, "133": {"node_type": "multimarker", "x": -921.986608587119, "y": 1093.7605304587546}, "134": {"node_type": "midmarker", "x": -901.986608587119, "y": 1093.7605304587546}, "135": {"node_type": "multimarker", "x": -881.986608587119, "y": 1093.7605304587546}, "136": {"node_type": "metabolite", "x": -985.986608587119, "y": 1008.7605304587546, "bigg_id": "cpd00007", "name": "O2", "label_x": -1035.1866085871188, "label_y": 1018.7605304587546, "node_is_primary": false}, "137": {"node_type": "metabolite", "x": -796.986608587119, "y": 1013.7605304587546, "bigg_id": "cpd00001", "name": "H2O", "label_x": -840.1866085871188, "label_y": 993.7605304587546, "node_is_primary": false}, "138": {"node_type": "metabolite", "x": -796.986608587119, "y": 1173.7605304587546, "bigg_id": "cpd00011", "name": "CO2", "label_x": -840.1866085871188, "label_y": 1213.7605304587546, "node_is_primary": false}, "139": {"node_type": "metabolite", "x": -726.986608587119, "y": 1093.7605304587546, "bigg_id": "cpd01747", "name": "Indole-3-acetamide", "label_x": -798.3301632746188, "label_y": 1066.6494464743796, "node_is_primary": true}, "140": {"node_type": "multimarker", "x": -1076.9866085871188, "y": 1598.7605304587541}, "141": {"node_type": "midmarker", "x": -1076.9866085871188, "y": 1618.7605304587541}, "142": {"node_type": "multimarker", "x": -1076.9866085871188, "y": 1638.7605304587541}, "143": {"node_type": "metabolite", "x": -1156.9866085871188, "y": 1513.7605304587541, "bigg_id": "cpd00001", "name": "H2O", "label_x": -1228.9866085871188, "label_y": 1543.7605304587541, "node_is_primary": false}, "144": {"node_type": "metabolite", "x": -996.9866085871188, "y": 1723.7605304587541, "bigg_id": "cpd00047", "name": "Formate", "label_x": -981.9866085871188, "label_y": 1723.7605304587541, "node_is_primary": false}, "145": {"node_type": "metabolite", "x": -1156.9866085871188, "y": 1723.7605304587541, "bigg_id": "cpd00067", "name": "H+", "label_x": -1228.9866085871188, "label_y": 1753.7605304587541, "node_is_primary": false}, "146": {"node_type": "metabolite", "x": -1076.9866085871188, "y": 1793.7605304587541, "bigg_id": "cpd00275", "name": "L-Kynurenine", "label_x": -1051.9866085871188, "label_y": 1793.7605304587541, "node_is_primary": true}, "147": {"node_type": "multimarker", "x": -571.986608587119, "y": 1093.7605304587546}, "148": {"node_type": "midmarker", "x": -551.986608587119, "y": 1093.7605304587546}, "149": {"node_type": "multimarker", "x": -531.986608587119, "y": 1093.7605304587546}, "150": {"node_type": "metabolite", "x": -656.986608587119, "y": 1173.7605304587546, "bigg_id": "cpd00001", "name": "H2O", "label_x": -700.1866085871188, "label_y": 1213.7605304587546, "node_is_primary": false}, "151": {"node_type": "metabolite", "x": -446.986608587119, "y": 1173.7605304587546, "bigg_id": "cpd00013", "name": "NH3", "label_x": -490.1866085871188, "label_y": 1213.7605304587546, "node_is_primary": false}, "152": {"node_type": "metabolite", "x": -376.986608587119, "y": 1093.7605304587546, "bigg_id": "cpd00703", "name": "Indoleacetate", "label_x": -439.6707394464938, "label_y": 1064.4844074118796, "node_is_primary": true}, "153": {"node_type": "multimarker", "x": 309.974365234375, "y": 1268.721923828125}, "154": {"node_type": "midmarker", "x": 309.974365234375, "y": 1248.721923828125}, "155": {"node_type": "multimarker", "x": 309.974365234375, "y": 1228.721923828125}, "156": {"node_type": "metabolite", "x": 389.974365234375, "y": 1353.721923828125, "bigg_id": "cpd00001", "name": "H2O", "label_x": 404.974365234375, "label_y": 1353.721923828125, "node_is_primary": false}, "157": {"node_type": "metabolite", "x": 309.974365234375, "y": 1073.721923828125, "bigg_id": "cpd03708", "name": "2,4-Dihydroxyhept-2-enedioate", "label_x": 334.974365234375, "label_y": 1073.721923828125, "node_is_primary": true}, "158": {"node_type": "multimarker", "x": 308.6614031218121, "y": 1967.9782606239105}, "159": {"node_type": "midmarker", "x": 308.72575615450467, "y": 1947.9783641570011}, "160": {"node_type": "multimarker", "x": 308.7901091871963, "y": 1927.9784676900867}, "161": {"node_type": "metabolite", "x": 388.3874886005184, "y": 2053.2352327390513, "bigg_id": "cpd00067", "name": "H+", "label_x": 400.0154182880184, "label_y": 2059.6011995359263, "node_is_primary": false}, "162": {"node_type": "metabolite", "x": 389.06319544378437, "y": 1843.2363198364787, "bigg_id": "cpd00011", "name": "CO2", "label_x": 409.781121469175, "label_y": 1849.6024087036662, "node_is_primary": false}, "164": {"node_type": "multimarker", "x": 307.0364890463395, "y": 2667.9756464134284}, "165": {"node_type": "midmarker", "x": 307.0364890463395, "y": 2647.9756464134284}, "166": {"node_type": "multimarker", "x": 307.0364890463395, "y": 2627.9756464134284}, "167": {"node_type": "metabolite", "x": 223.6526694662614, "y": 2751.1802362571784, "bigg_id": "cpd00001", "name": "H2O", "label_x": 163.24475625825357, "label_y": 2742.2029413353034, "node_is_primary": false}, "168": {"node_type": "metabolite", "x": 386.0352683432145, "y": 2754.7493280540534, "bigg_id": "cpd00003", "name": "NAD", "label_x": 411.0352683432145, "label_y": 2754.7493280540534, "node_is_primary": false}, "169": {"node_type": "metabolite", "x": 306.0352378256364, "y": 2817.6109003196784, "bigg_id": "cpd02828", "name": "5-Carboxymethyl-2-hydroxymuconic semialdehyde", "label_x": 234.0352378256364, "label_y": 2847.6109003196784, "node_is_primary": true}, "170": {"node_type": "metabolite", "x": 387.0364890463395, "y": 2542.9756464134284, "bigg_id": "cpd00004", "name": "NADH", "label_x": 402.0364890463395, "label_y": 2542.9756464134284, "node_is_primary": false}, "171": {"node_type": "metabolite", "x": 227.0364890463395, "y": 2542.9756464134284, "bigg_id": "cpd00067", "name": "H+", "label_x": 190.9450278646989, "label_y": 2562.2031854759284, "node_is_primary": false}, "172": {"node_type": "multimarker", "x": 306.0352378256364, "y": 3012.6109003196784}, "173": {"node_type": "midmarker", "x": 306.0352378256364, "y": 2992.6109003196784}, "174": {"node_type": "multimarker", "x": 306.0352378256364, "y": 2972.6109003196784}, "175": {"node_type": "metabolite", "x": 226.0352378256364, "y": 3097.6109003196784, "bigg_id": "cpd00007", "name": "O2", "label_x": 154.0352378256364, "label_y": 3127.6109003196784, "node_is_primary": false}, "176": {"node_type": "metabolite", "x": 306.0352378256364, "y": 3167.6109003196784, "bigg_id": "cpd00854", "name": "Homoprotocatechuate", "label_x": 331.0352378256364, "label_y": 3167.6109003196784, "node_is_primary": true}, "177": {"node_type": "metabolite", "x": 226.0352378256364, "y": 2887.6109003196784, "bigg_id": "cpd00067", "name": "H+", "label_x": 154.0352378256364, "label_y": 2917.6109003196784, "node_is_primary": false}, "187": {"node_type": "multimarker", "x": 306.0352378256364, "y": 3362.6109003196784}, "188": {"node_type": "midmarker", "x": 306.0352378256364, "y": 3342.6109003196784}, "189": {"node_type": "multimarker", "x": 306.0352378256364, "y": 3322.6109003196784}, "190": {"node_type": "metabolite", "x": 223.44560769868326, "y": 3449.3845819603034, "bigg_id": "cpd00004", "name": "NADH", "label_x": 155.08342508149576, "label_y": 3426.0440057884284, "node_is_primary": false}, "191": {"node_type": "metabolite", "x": 396.80779031587076, "y": 3449.4063104759284, "bigg_id": "cpd00007", "name": "O2", "label_x": 411.80779031587076, "label_y": 3449.4063104759284, "node_is_primary": false}, "192": {"node_type": "metabolite", "x": 405.5778861410661, "y": 3397.3389276634284, "bigg_id": "cpd00067", "name": "H+", "label_x": 419.75839761567545, "label_y": 3400.4075311790534, "node_is_primary": false}, "193": {"node_type": "metabolite", "x": 305.8282065756364, "y": 3515.5867303978034, "bigg_id": "cpd00489", "name": "4-Hydroxyphenylacetate", "label_x": 233.8282065756364, "label_y": 3545.5867303978034, "node_is_primary": true}, "194": {"node_type": "metabolite", "x": 386.0352378256364, "y": 3237.6109003196784, "bigg_id": "cpd00001", "name": "H2O", "label_x": 401.0352378256364, "label_y": 3237.6109003196784, "node_is_primary": false}, "195": {"node_type": "metabolite", "x": 226.0352378256364, "y": 3237.6109003196784, "bigg_id": "cpd00003", "name": "NAD", "label_x": 154.0352378256364, "label_y": 3267.6109003196784, "node_is_primary": false}, "196": {"node_type": "multimarker", "x": 305.8282065756364, "y": 3710.5867303978034}, "197": {"node_type": "midmarker", "x": 305.8282065756364, "y": 3690.5867303978034}, "198": {"node_type": "multimarker", "x": 305.8282065756364, "y": 3670.5867303978034}, "199": {"node_type": "metabolite", "x": 385.8282065756364, "y": 3795.5867303978034, "bigg_id": "cpd00001", "name": "H2O", "label_x": 400.8282065756364, "label_y": 3795.5867303978034, "node_is_primary": false}, "200": {"node_type": "metabolite", "x": 225.03403238130045, "y": 3797.3606561790534, "bigg_id": "cpd00003", "name": "NAD", "label_x": 174.62611917329264, "label_y": 3775.8154901634284, "node_is_primary": false}, "201": {"node_type": "metabolite", "x": 306.6223960287614, "y": 3844.0632928978034, "bigg_id": "cpd02361", "name": "4-Hydroxyphenylacetaldehyde", "label_x": 234.6223960287614, "label_y": 3874.0632928978034, "node_is_primary": true}, "202": {"node_type": "metabolite", "x": 385.8282065756364, "y": 3585.5867303978034, "bigg_id": "cpd00004", "name": "NADH", "label_x": 400.8282065756364, "label_y": 3585.5867303978034, "node_is_primary": false}, "203": {"node_type": "metabolite", "x": 225.8282065756364, "y": 3585.5867303978034, "bigg_id": "cpd00067", "name": "H+", "label_x": 180.7596030600114, "label_y": 3601.2232050071784, "node_is_primary": false}, "204": {"node_type": "multimarker", "x": -33.68950176239014, "y": 3850.933349609375}, "205": {"node_type": "midmarker", "x": -13.689501762390137, "y": 3850.933349609375}, "206": {"node_type": "multimarker", "x": 6.310498237609863, "y": 3850.933349609375}, "207": {"node_type": "metabolite", "x": -118.68950176239014, "y": 3770.933349609375, "bigg_id": "cpd00001", "name": "H2O", "label_x": -161.88950176239013, "label_y": 3750.933349609375, "node_is_primary": false}, "208": {"node_type": "metabolite", "x": -118.68950176239014, "y": 3930.933349609375, "bigg_id": "cpd00007", "name": "O2", "label_x": -161.88950176239013, "label_y": 3970.933349609375, "node_is_primary": false}, "209": {"node_type": "metabolite", "x": 91.31049823760986, "y": 3770.933349609375, "bigg_id": "cpd00013", "name": "NH3", "label_x": 48.11049823760987, "label_y": 3750.933349609375, "node_is_primary": false}, "210": {"node_type": "metabolite", "x": 91.31049823760986, "y": 3930.933349609375, "bigg_id": "cpd00025", "name": "H2O2", "label_x": 48.11049823760987, "label_y": 3970.933349609375, "node_is_primary": false}, "212": {"node_type": "multimarker", "x": -1076.9866085871188, "y": 1948.7605304587541}, "213": {"node_type": "midmarker", "x": -1076.9866085871188, "y": 1968.7605304587541}, "214": {"node_type": "multimarker", "x": -1076.9866085871188, "y": 1988.7605304587541}, "215": {"node_type": "metabolite", "x": -1156.9866085871188, "y": 1863.7605304587541, "bigg_id": "cpd00001", "name": "H2O", "label_x": -1228.9866085871188, "label_y": 1893.7605304587541, "node_is_primary": false}, "216": {"node_type": "metabolite", "x": -996.9866085871188, "y": 2073.760530458754, "bigg_id": "cpd00035", "name": "L-Alanine", "label_x": -981.9866085871188, "label_y": 2073.760530458754, "node_is_primary": false}, "217": {"node_type": "metabolite", "x": -1156.9866085871188, "y": 2073.760530458754, "bigg_id": "cpd00067", "name": "H+", "label_x": -1228.9866085871188, "label_y": 2103.760530458754, "node_is_primary": false}, "218": {"node_type": "metabolite", "x": -1076.9866085871188, "y": 2143.760530458754, "bigg_id": "cpd00093", "name": "Anthranilate", "label_x": -1063.1886959894625, "label_y": 2125.090608583754, "node_is_primary": true}, "219": {"node_type": "multimarker", "x": -1315.9879513605563, "y": 2343.225862490004}, "220": {"node_type": "midmarker", "x": -1315.9879513605563, "y": 2363.225862490004}, "221": {"node_type": "multimarker", "x": -1315.9879513605563, "y": 2383.225862490004}, "222": {"node_type": "metabolite", "x": -1235.9879513605563, "y": 2258.225862490004, "bigg_id": "cpd00004", "name": "NADH", "label_x": -1257.1160031183688, "label_y": 2244.330354677504, "node_is_primary": false}, "223": {"node_type": "metabolite", "x": -1395.9879513605563, "y": 2258.225862490004, "bigg_id": "cpd00007", "name": "O2", "label_x": -1442.9762326105563, "label_y": 2249.318635927504, "node_is_primary": false}, "224": {"node_type": "metabolite", "x": -1475.9879513605563, "y": 2278.225862490004, "bigg_id": "cpd00067", "name": "H+", "label_x": -1520.1971798761813, "label_y": 2294.330354677504, "node_is_primary": false}, "225": {"node_type": "metabolite", "x": -1315.9878292902438, "y": 2557.679475771254, "bigg_id": "cpd00003", "name": "NAD", "label_x": -1290.9878292902438, "label_y": 2557.679475771254, "node_is_primary": false}, "226": {"node_type": "metabolite", "x": -1235.9879513605563, "y": 2468.225862490004, "bigg_id": "cpd00011", "name": "CO2", "label_x": -1220.9879513605563, "label_y": 2468.225862490004, "node_is_primary": false}, "227": {"node_type": "metabolite", "x": -1395.9879513605563, "y": 2468.225862490004, "bigg_id": "cpd00013", "name": "NH3", "label_x": -1467.9879513605563, "label_y": 2498.225862490004, "node_is_primary": false}, "228": {"node_type": "metabolite", "x": -1070.2414913996188, "y": 2545.493928896254, "bigg_id": "cpd00077", "name": "Catechol", "label_x": -1142.2414913996188, "label_y": 2575.493928896254, "node_is_primary": true}, "239": {"node_type": "multimarker", "x": -828.0303342797051, "y": 2336.9718377108584}, "240": {"node_type": "midmarker", "x": -827.2493432021806, "y": 2356.9565832167896}, "241": {"node_type": "multimarker", "x": -826.4683521246566, "y": 2376.9413287227203}, "242": {"node_type": "metabolite", "x": -751.4105643354601, "y": 2248.912705000554, "bigg_id": "cpd00005", "name": "NADPH", "label_x": -736.4105643354601, "label_y": 2248.912705000554, "node_is_primary": false}, "243": {"node_type": "metabolite", "x": -911.2885283829082, "y": 2255.1606336207496, "bigg_id": "cpd00007", "name": "O2", "label_x": -952.7185820938455, "label_y": 2240.6953015894996, "node_is_primary": false}, "244": {"node_type": "metabolite", "x": -990.4465193291069, "y": 2278.2693434367784, "bigg_id": "cpd00067", "name": "H+", "label_x": -1017.9811872978569, "label_y": 2305.4902907024034, "node_is_primary": false}, "245": {"node_type": "metabolite", "x": -826.1865789771987, "y": 2526.486281768232, "bigg_id": "cpd00006", "name": "NADP", "label_x": -801.1865789771987, "label_y": 2526.486281768232, "node_is_primary": false}, "246": {"node_type": "metabolite", "x": -743.2101580214539, "y": 2458.752532812829, "bigg_id": "cpd00011", "name": "CO2", "label_x": -728.2101580214539, "label_y": 2458.752532812829, "node_is_primary": false}, "247": {"node_type": "metabolite", "x": -903.0881220689016, "y": 2465.0004614330246, "bigg_id": "cpd00013", "name": "NH3", "label_x": -975.0881220689012, "label_y": 2495.0004614330246, "node_is_primary": false}, "249": {"node_type": "multimarker", "x": -310.9999694824219, "y": 3029}, "250": {"node_type": "midmarker", "x": -310.9999694824219, "y": 3009}, "251": {"node_type": "multimarker", "x": -310.9999694824219, "y": 2989}, "252": {"node_type": "metabolite", "x": -390.9999694824219, "y": 3114, "bigg_id": "cpd00001", "name": "H2O", "label_x": -462.9999694824219, "label_y": 3144, "node_is_primary": false}, "253": {"node_type": "metabolite", "x": -230.99996948242188, "y": 3114, "bigg_id": "cpd00003", "name": "NAD", "label_x": -215.99996948242188, "label_y": 3114, "node_is_primary": false}, "254": {"node_type": "metabolite", "x": -199.83657836914062, "y": 2947.942626953125, "bigg_id": "cpd00004", "name": "NADH", "label_x": -174.83657836914062, "label_y": 2947.942626953125, "node_is_primary": false}, "255": {"node_type": "metabolite", "x": -390.9999694824219, "y": 2904, "bigg_id": "cpd00013", "name": "NH3", "label_x": -462.9999694824219, "label_y": 2934, "node_is_primary": false}, "256": {"node_type": "metabolite", "x": -230.99996948242188, "y": 2904, "bigg_id": "cpd00067", "name": "H+", "label_x": -215.99996948242188, "label_y": 2904, "node_is_primary": false}, "257": {"node_type": "metabolite", "x": -309.4078063964844, "y": 2821.174072265625, "bigg_id": "cpd00143", "name": "Phenylpyruvate", "label_x": -294.4078063964844, "label_y": 2821.174072265625, "node_is_primary": true}, "258": {"node_type": "multimarker", "x": -309.4078063964844, "y": 2626.174072265625}, "259": {"node_type": "midmarker", "x": -309.4078063964844, "y": 2646.174072265625}, "260": {"node_type": "multimarker", "x": -309.4078063964844, "y": 2666.174072265625}, "261": {"node_type": "metabolite", "x": -229.40780639648438, "y": 2541.174072265625, "bigg_id": "cpd00067", "name": "H+", "label_x": -214.40780639648438, "label_y": 2541.174072265625, "node_is_primary": false}, "262": {"node_type": "metabolite", "x": -309.4078063964844, "y": 2471.174072265625, "bigg_id": "cpd00219", "name": "Prephenate", "label_x": -284.4078063964844, "label_y": 2471.174072265625, "node_is_primary": true}, "263": {"node_type": "metabolite", "x": -389.4078063964844, "y": 2751.174072265625, "bigg_id": "cpd00001", "name": "H2O", "label_x": -461.4078063964844, "label_y": 2781.174072265625, "node_is_primary": false}, "264": {"node_type": "metabolite", "x": -229.40780639648438, "y": 2751.174072265625, "bigg_id": "cpd00011", "name": "CO2", "label_x": -214.40780639648438, "label_y": 2751.174072265625, "node_is_primary": false}, "265": {"node_type": "multimarker", "x": -309.4078063964844, "y": 2276.174072265625}, "266": {"node_type": "midmarker", "x": -309.4078063964844, "y": 2296.174072265625}, "267": {"node_type": "multimarker", "x": -309.4078063964844, "y": 2316.174072265625}, "268": {"node_type": "metabolite", "x": -311.2748107910156, "y": 2134.2431640625, "bigg_id": "cpd00216", "name": "Chorismate", "label_x": -286.2748107910156, "label_y": 2134.2431640625, "node_is_primary": true}, "277": {"node_type": "multimarker", "x": -652.976318359375, "y": 2128.642333984375}, "278": {"node_type": "midmarker", "x": -672.976318359375, "y": 2128.642333984375}, "279": {"node_type": "multimarker", "x": -692.976318359375, "y": 2128.642333984375}, "280": {"node_type": "metabolite", "x": -567.976318359375, "y": 2048.642333984375, "bigg_id": "cpd00013", "name": "NH3", "label_x": -611.176318359375, "label_y": 2028.642333984375, "node_is_primary": false}, "281": {"node_type": "metabolite", "x": -776.1092529296875, "y": 2188.105224609375, "bigg_id": "cpd00001", "name": "H2O", "label_x": -757.697802734375, "label_y": 2192.632080078125, "node_is_primary": false}, "282": {"node_type": "metabolite", "x": -777.976318359375, "y": 2048.642333984375, "bigg_id": "cpd00020", "name": "Pyruvate", "label_x": -821.176318359375, "label_y": 2028.642333984375, "node_is_primary": false}, "283": {"node_type": "metabolite", "x": -757.976318359375, "y": 1968.642333984375, "bigg_id": "cpd00067", "name": "H+", "label_x": -801.176318359375, "label_y": 1948.642333984375, "node_is_primary": false}}, "text_labels": {"0": {"x": -854.3882141113281, "y": 554.7875366210938, "text": "Phenylalanine, Tyrosine, and Tryptophan Degradation"}, "3": {"x": -1219.9794921875, "y": 964.8426513671875, "text": "Tryptophan"}, "4": {"x": -824.7456665039062, "y": 3178.15185546875, "text": "Phenylalanine"}, "5": {"x": -730.877685546875, "y": 3543.7431640625, "text": "Tyrosine"}}, "canvas": {"x": -2035.5486450195312, "y": 317.6473083496094, "width": 3646.8306274414062, "height": 3996.8019104003906}}]

In [1410]:
false = False

In [1412]:
null = None

In [2445]:
def parse_map_for_reactions(map, stoich):
    n = 0
    for r in map[1]['reactions']:
        for m in map[1]['reactions'][r]['metabolites']:
            if str(n) not in stoich.keys():
                stoich[str(n)] = {}
            stoich[str(n)][m['bigg_id']] = m['coefficient']
        n += 1

In [4419]:
parse_map_for_reactions(map2, stoich4)

In [4418]:
stoich4 = {}

In [2692]:
stoich = {'0': {'cpd00035': -1, 'cpd00117': 1},
 '1': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1},
 '2': {'cpd00156': 1, 'cpd00123': -1, 'cpd00024': -1, 'cpd00023': 1},
 '3': {'cpd00084': -1, 'cpd00035': 1, 'cpd14547': -1, 'cpd14548': 1},
'4': {'cpd00035': -1, 'cpd00024': -1, 'cpd00020': 1, 'cpd00023': 1}}

In [2484]:
def order_stoich(stoich, layout):
    stoich2 = {}
    x = 0
    for r in stoich:
        stoich2[str(x)] = {}
        for m in stoich[r]:
            for n in layout[1]['nodes']:
                if layout[1]['nodes'][n]['node_type'] == 'metabolite':
                    if layout[1]['nodes'][n]['bigg_id'] == m:
                        if layout[1]['nodes'][n]['node_is_primary']:
                            stoich2[str(x)][m] = copy.deepcopy(stoich[str(x)][m])
        for m in stoich[r]: 
            if m not in stoich2:
                stoich2[str(x)][m] = copy.deepcopy(stoich[str(x)][m])
        x += 1
    
    return stoich2

In [4420]:
stoich4 = order_stoich(stoich4, map2)

In [4269]:
stoich6 = {'0': {'cpd01716': -1, 'cpd00197': 1, 'cpd00001': 1},
 '1': {'cpd00699': -1, 'cpd01716': 1, 'cpd00001': 1},
 '2': {'cpd00248': -1, 'cpd00699': 1, 'cpd00097': -1, 'cpd00986': 1},
 '3': {'cpd00383': 1,
  'cpd01716': -1,
  'cpd00006': -1,
  'cpd00005': 1,
  'cpd00067': 1},
 '4': {'cpd00383': 1, 'cpd01716': -1, 'cpd00097': -1, 'cpd00986': 1}}

In [2548]:
stoich = {'0': {'cpd00035': -1, 'cpd00117': 1},
 '1': {'cpd00156': -1, 'cpd00035': 1, 'cpd00123': 1, 'cpd00020': -1},
 '2': {'cpd00156': -1, 'cpd00123': 1, 'cpd00024': -1, 'cpd00023': 1}}

In [4217]:
map2 = [{"map_name":"new_map","map_id":"9YGMK6mZMDvw","map_description":"","homepage":"https://escher.github.io","schema":"https://escher.github.io/escher/jsonschema/1-0-0#"},{"reactions":{"0":{"name":"rxn00293","bigg_id":"rxn00293","reversibility":false,"label_x":-289.6517333984375,"label_y":-145.3863983154297,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00062","coefficient":-1},{"bigg_id":"cpd02611","coefficient":-1},{"bigg_id":"cpd00012","coefficient":1},{"bigg_id":"cpd00037","coefficient":1}],"segments":{"0":{"from_node_id":"1","to_node_id":"2","b1":null,"b2":null},"1":{"from_node_id":"3","to_node_id":"2","b1":null,"b2":null},"2":{"from_node_id":"1","to_node_id":"0","b1":{"x":-304.6517333984375,"y":-217.3863983154297},"b2":{"x":-329.9708557128906,"y":-217.00038146972656}},"3":{"from_node_id":"1","to_node_id":"4","b1":{"x":-304.6517333984375,"y":-221.3863983154297},"b2":{"x":-306.6606750488281,"y":-254.62074279785156}},"4":{"from_node_id":"3","to_node_id":"5","b1":{"x":-304.6517333984375,"y":-89.38639831542969},"b2":{"x":-322.6517333984375,"y":-84.13639831542969}},"5":{"from_node_id":"3","to_node_id":"31","b1":{"x":-304.6517333984375,"y":-93.38639831542969},"b2":{"x":-304.6517333984375,"y":-39.13639831542969}}}},"1":{"name":"rxn01483","bigg_id":"rxn01483","reversibility":false,"label_x":-428.72580384817286,"label_y":-1192.1574273735353,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00022","coefficient":-1},{"bigg_id":"cpd00288","coefficient":-1},{"bigg_id":"cpd00010","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd00293","coefficient":1}],"segments":{"6":{"from_node_id":"8","to_node_id":"9","b1":null,"b2":null},"7":{"from_node_id":"10","to_node_id":"9","b1":null,"b2":null},"8":{"from_node_id":"8","to_node_id":"7","b1":{"x":-444.250888296424,"y":-1140.1596509108094},"b2":{"x":-461.92627957145487,"y":-1138.8836424043616}},"9":{"from_node_id":"8","to_node_id":"42","b1":{"x":-441.636174624548,"y":-1072.5932660445155},"b2":{"x":-442.2214746140397,"y":-1068.51522920584}},"10":{"from_node_id":"10","to_node_id":"12","b1":{"x":-443.2007193999229,"y":-1264.155203836262},"b2":{"x":-479.8217758787986,"y":-1276.0253285536478}},"11":{"from_node_id":"10","to_node_id":"13","b1":{"x":-443.1668429839065,"y":-1268.1550603822438},"b2":{"x":-414.7711487073065,"y":-1273.5573159709193}},"12":{"from_node_id":"10","to_node_id":"14","b1":{"x":-443.1668429839065,"y":-1268.1550603822438},"b2":{"x":-443.66330893409054,"y":-1285.1708852580234}}}},"2":{"name":"rxn02285","bigg_id":"rxn02285","reversibility":false,"label_x":-285.48439025878906,"label_y":567.8349151611328,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00006","coefficient":-1},{"bigg_id":"cpd00773","coefficient":-1},{"bigg_id":"cpd00005","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd02820","coefficient":1}],"segments":{"13":{"from_node_id":"16","to_node_id":"17","b1":null,"b2":null},"14":{"from_node_id":"18","to_node_id":"17","b1":null,"b2":null},"15":{"from_node_id":"16","to_node_id":"15","b1":{"x":-299.0311919399217,"y":619.8178822944193},"b2":{"x":-283.1011961452879,"y":622.0820312704195}},"16":{"from_node_id":"16","to_node_id":"19","b1":{"x":-298.9374372096721,"y":623.8167833997921},"b2":{"x":-300.80812920086777,"y":651.2974365136906}},"17":{"from_node_id":"18","to_node_id":"20","b1":{"x":-301.9375885776564,"y":495.85194802784633},"b2":{"x":-279.2225800998294,"y":492.25509641512735}},"18":{"from_node_id":"18","to_node_id":"21","b1":{"x":-302.031343307906,"y":491.8530469224735},"b2":{"x":-320.14945136553837,"y":487.02638550779375}},"19":{"from_node_id":"18","to_node_id":"37","b1":{"x":-302.031343307906,"y":491.8530469224735},"b2":{"x":-302.4933319091797,"y":523.8105010986328}}}},"3":{"name":"rxn03638","bigg_id":"rxn03638","reversibility":false,"label_x":-291.2892875671387,"label_y":-531.5687103271484,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00022","coefficient":-1},{"bigg_id":"cpd03671","coefficient":-1},{"bigg_id":"cpd00010","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd02611","coefficient":1}],"segments":{"20":{"from_node_id":"24","to_node_id":"25","b1":null,"b2":null},"21":{"from_node_id":"26","to_node_id":"25","b1":null,"b2":null},"22":{"from_node_id":"24","to_node_id":"23","b1":{"x":-306.2892875671387,"y":-603.5687103271484},"b2":{"x":-340.9365043640137,"y":-613.8433685302734}},"23":{"from_node_id":"24","to_node_id":"27","b1":{"x":-306.2892875671387,"y":-607.5687103271484},"b2":{"x":-306.9656791687012,"y":-631.4748992919922}},"24":{"from_node_id":"26","to_node_id":"28","b1":{"x":-306.2892875671387,"y":-479.56871032714844},"b2":{"x":-335.6061820983887,"y":-473.29176330566406}},"25":{"from_node_id":"26","to_node_id":"29","b1":{"x":-306.2892875671387,"y":-475.56871032714844},"b2":{"x":-288.2892875671387,"y":-470.31871032714844}},"26":{"from_node_id":"26","to_node_id":"4","b1":{"x":-306.2892875671387,"y":-475.56871032714844},"b2":{"x":-308.2982597351074,"y":-438.3367462158203}}}},"4":{"name":"rxn00461","bigg_id":"rxn00461","reversibility":false,"label_x":-290,"label_y":203.99998474121094,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00037","coefficient":-1},{"bigg_id":"cpd00061","coefficient":-1},{"bigg_id":"cpd00009","coefficient":1},{"bigg_id":"cpd02820","coefficient":1}],"segments":{"27":{"from_node_id":"32","to_node_id":"33","b1":null,"b2":null},"28":{"from_node_id":"34","to_node_id":"33","b1":null,"b2":null},"29":{"from_node_id":"32","to_node_id":"31","b1":{"x":-305,"y":131.99998474121094},"b2":{"x":-305,"y":77.74998474121094}},"30":{"from_node_id":"32","to_node_id":"35","b1":{"x":-305,"y":127.99998474121094},"b2":{"x":-323,"y":122.74998474121094}},"31":{"from_node_id":"34","to_node_id":"36","b1":{"x":-305,"y":259.99998474121094},"b2":{"x":-323,"y":265.24998474121094}},"32":{"from_node_id":"34","to_node_id":"37","b1":{"x":-305,"y":255.99998474121094},"b2":{"x":-305,"y":310.24998474121094}}}},"5":{"name":"rxn01485","bigg_id":"rxn01485","reversibility":false,"label_x":-288.06358337402344,"label_y":-881.0799713134766,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd03671","coefficient":-1},{"bigg_id":"cpd00288","coefficient":1}],"segments":{"33":{"from_node_id":"39","to_node_id":"40","b1":null,"b2":null},"34":{"from_node_id":"41","to_node_id":"40","b1":null,"b2":null},"35":{"from_node_id":"39","to_node_id":"27","b1":{"x":-303.8578542832081,"y":-829.0850591535029},"b2":{"x":-303.22026076233817,"y":-818.8148528104018}},"36":{"from_node_id":"41","to_node_id":"42","b1":{"x":-302.26931246483866,"y":-953.0748834734504},"b2":{"x":-301.5743254193022,"y":-1007.3204316134265}}}},"7":{"name":"rxn15064","bigg_id":"rxn15064","reversibility":false,"label_x":-130.4826784133911,"label_y":-1192.4988059997559,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00053","coefficient":-1},{"bigg_id":"cpd19035","coefficient":-1},{"bigg_id":"cpd00023","coefficient":1},{"bigg_id":"cpd00288","coefficient":1}],"segments":{"43":{"from_node_id":"51","to_node_id":"52","b1":null,"b2":null},"44":{"from_node_id":"53","to_node_id":"52","b1":null,"b2":null},"45":{"from_node_id":"51","to_node_id":"50","b1":{"x":-145.4826784133911,"y":-1264.4988059997559},"b2":{"x":-117.6723165512085,"y":-1268.4253196716309}},"46":{"from_node_id":"51","to_node_id":"54","b1":{"x":-145.4826784133911,"y":-1268.4988059997559},"b2":{"x":-146.26674365997314,"y":-1298.9105491638184}},"47":{"from_node_id":"53","to_node_id":"55","b1":{"x":-145.4826784133911,"y":-1136.4988059997559},"b2":{"x":-125.07787799835205,"y":-1140.5190696716309}},"48":{"from_node_id":"53","to_node_id":"42","b1":{"x":-146.80698108673096,"y":-1067.6620140075684},"b2":{"x":-148.13128185272217,"y":-1069.0329856872559}}}}},"nodes":{"0":{"node_type":"metabolite","x":-392.6024169921875,"y":-245.10089111328125,"bigg_id":"cpd00062","name":"UTP","label_x":-367.6024169921875,"label_y":-245.10089111328125,"node_is_primary":false},"1":{"node_type":"multimarker","x":-304.6517333984375,"y":-175.3863983154297},"2":{"node_type":"midmarker","x":-304.6517333984375,"y":-155.3863983154297},"3":{"node_type":"multimarker","x":-304.6517333984375,"y":-135.3863983154297},"4":{"node_type":"metabolite","x":-307.3617248535156,"y":-331.0134735107422,"bigg_id":"cpd02611","name":"N-Acetyl-D-glucosamine1-phosphate","label_x":-477.9730987548828,"label_y":-321.0022430419922,"node_is_primary":true},"5":{"node_type":"metabolite","x":-384.6517333984375,"y":-50.38639831542969,"bigg_id":"cpd00012","name":"PPi","label_x":-456.6517333984375,"label_y":-20.386398315429688,"node_is_primary":false},"7":{"node_type":"metabolite","x":-528.638988357472,"y":-1114.5676341508704,"bigg_id":"cpd00022","name":"Acetyl-CoA","label_x":-604.2860220488783,"label_y":-1146.350959346183,"node_is_primary":false},"8":{"node_type":"multimarker","x":-443.89518592825254,"y":-1182.1581446436237},"9":{"node_type":"midmarker","x":-443.72580384817286,"y":-1202.1574273735353},"10":{"node_type":"multimarker","x":-443.55642176809226,"y":-1222.1567101034475},"12":{"node_type":"metabolite","x":-533.6206637724674,"y":-1285.7740760649517,"bigg_id":"cpd00010","name":"CoA","label_x":-573.5115329130924,"label_y":-1250.0178504790142,"node_is_primary":false},"13":{"node_type":"metabolite","x":-358.6200863180999,"y":-1290.6152476430836,"bigg_id":"cpd00067","name":"H+","label_x":-343.6200863180999,"label_y":-1290.6152476430836,"node_is_primary":false},"14":{"node_type":"metabolite","x":-443.62191822880584,"y":-1362.0967388540003,"bigg_id":"cpd00293","name":"N-Acetyl-D-glucosamine 6-phosphate","label_x":-629.5119633948214,"label_y":-1396.9876079946253,"node_is_primary":true},"15":{"node_type":"metabolite","x":-224.42298030744348,"y":644.836154927465,"bigg_id":"cpd00006","name":"NADP","label_x":-199.42298030744348,"label_y":644.836154927465,"node_is_primary":false},"16":{"node_type":"multimarker","x":-300.0156166075416,"y":577.8294206879995},"17":{"node_type":"midmarker","x":-300.48439025878906,"y":557.8349151611328},"18":{"node_type":"multimarker","x":-300.95316391003655,"y":537.8404096342662},"19":{"node_type":"metabolite","x":-299.3330567264138,"y":706.2388969354731,"bigg_id":"cpd00773","name":"UDP-MurNAc","label_x":-366.0027039432108,"label_y":748.2321830682856,"node_is_primary":true},"20":{"node_type":"metabolite","x":-215.3028955470487,"y":456.17516709401934,"bigg_id":"cpd00005","name":"NADPH","label_x":-190.3028955470487,"label_y":456.17516709401934,"node_is_primary":false},"21":{"node_type":"metabolite","x":-382.92347403530493,"y":454.738855750074,"bigg_id":"cpd00067","name":"H+","label_x":-367.92347403530493,"label_y":454.738855750074,"node_is_primary":false},"23":{"node_type":"metabolite","x":-412.8961601257324,"y":-625.9528045654297,"bigg_id":"cpd00022","name":"Acetyl-CoA","label_x":-469.1838798522949,"label_y":-641.9438934326172,"node_is_primary":false},"24":{"node_type":"multimarker","x":-306.2892875671387,"y":-561.5687103271484},"25":{"node_type":"midmarker","x":-306.2892875671387,"y":-541.5687103271484},"26":{"node_type":"multimarker","x":-306.2892875671387,"y":-521.5687103271484},"27":{"node_type":"metabolite","x":-305.0015525817871,"y":-710.5328216552734,"bigg_id":"cpd03671","name":"D-Glucosamine1-phosphate","label_x":-442.2982597351074,"label_y":-703.1867523193359,"node_is_primary":true},"28":{"node_type":"metabolite","x":-418.22651290893555,"y":-459.8496856689453,"bigg_id":"cpd00010","name":"CoA","label_x":-443.8647880554199,"label_y":-477.1732940673828,"node_is_primary":false},"29":{"node_type":"metabolite","x":-226.28928756713867,"y":-436.56871032714844,"bigg_id":"cpd00067","name":"H+","label_x":-211.28928756713867,"label_y":-436.56871032714844,"node_is_primary":false},"31":{"node_type":"metabolite","x":-305,"y":18.999984741210938,"bigg_id":"cpd00037","name":"UDP-N-acetylglucosamine","label_x":-280,"label_y":18.999984741210938,"node_is_primary":true},"32":{"node_type":"multimarker","x":-305,"y":173.99998474121094},"33":{"node_type":"midmarker","x":-305,"y":193.99998474121094},"34":{"node_type":"multimarker","x":-305,"y":213.99998474121094},"35":{"node_type":"metabolite","x":-385,"y":88.99998474121094,"bigg_id":"cpd00061","name":"Phosphoenolpyruvate","label_x":-457,"label_y":118.99998474121094,"node_is_primary":false},"36":{"node_type":"metabolite","x":-385,"y":298.99998474121094,"bigg_id":"cpd00009","name":"Phosphate","label_x":-457,"label_y":328.99998474121094,"node_is_primary":false},"37":{"node_type":"metabolite","x":-305,"y":368.99998474121094,"bigg_id":"cpd02820","name":"UDP-N-acetylglucosamine enolpyruvate","label_x":-280,"label_y":368.99998474121094,"node_is_primary":true},"39":{"node_type":"multimarker","x":-303.31979979634116,"y":-871.0816125521947},"40":{"node_type":"midmarker","x":-303.06358337402344,"y":-891.0799713134766},"41":{"node_type":"multimarker","x":-302.8073669517057,"y":-911.0783300747582},"42":{"node_type":"metabolite","x":-300.8216896787453,"y":-1066.0656104746922,"bigg_id":"cpd00288","name":"D-Glucosamine phosphate","label_x":-416.197834087925,"label_y":-1095.2003760996922,"node_is_primary":true},"50":{"node_type":"metabolite","x":-48.80854892730713,"y":-1288.7705345153809,"bigg_id":"cpd00053","name":"L-Glutamine","label_x":-23.80854892730713,"label_y":-1288.7705345153809,"node_is_primary":false},"51":{"node_type":"multimarker","x":-145.4826784133911,"y":-1222.4988059997559},"52":{"node_type":"midmarker","x":-145.4826784133911,"y":-1202.4988059997559},"53":{"node_type":"multimarker","x":-145.4826784133911,"y":-1182.4988059997559},"54":{"node_type":"metabolite","x":-147.34879779815674,"y":-1367.0924339294434,"bigg_id":"cpd19035","name":"beta-D-Fructose 6-phosphate","label_x":-235.2404375076294,"label_y":-1395.3617210388184,"node_is_primary":true},"55":{"node_type":"metabolite","x":-42.7288293838501,"y":-1122.6606712341309,"bigg_id":"cpd00023","name":"L-Glutamate","label_x":-114.7288293838501,"label_y":-1092.6606712341309,"node_is_primary":false}},"text_labels":{"0":{"x":-858.7408981323242,"y":-1487.5926513671875,"text":"UDP-N-Acetylmuramate from Fructose-6-Phosphate"}},"canvas":{"x":-957.4156494140625,"y":-1657.0641479492188,"width":1475.9495544433594,"height":2596.5421142578125}}]

In [1599]:
import cobrakbase
modelseed_local = cobrakbase.modelseed.from_local('/Users/toyosiakins/Downloads/ModelSEEDDatabase-dev')  #change to your repo path

load: /Users/toyosiakins/Downloads/ModelSEEDDatabase-dev/Biochemistry/reactions.tsv
load: /Users/toyosiakins/Downloads/ModelSEEDDatabase-dev/Biochemistry/compounds.tsv
load: /Users/toyosiakins/Downloads/ModelSEEDDatabase-dev/Biochemistry/Structures/Unique_ModelSEED_Structures.txt
load: /Users/toyosiakins/Downloads/ModelSEEDDatabase-dev/Biochemistry/Aliases/Unique_ModelSEED_Compound_Aliases.txt
load: /Users/toyosiakins/Downloads/ModelSEEDDatabase-dev/Biochemistry/Aliases/Unique_ModelSEED_Reaction_Aliases.txt
load: /Users/toyosiakins/Downloads/ModelSEEDDatabase-dev/Biochemistry/Aliases/Unique_ModelSEED_Reaction_ECs.txt


In [1605]:
rxn = modelseed_local.get_seed_reaction('rxn00003')

In [1551]:
display(rxn.stoichiometry) #this should return a dict {'cpd00001': -1.0, 'cpd00012': -1.0, 'cpd00009': 2.0, 'cpd00067': 1.0}

{'cpd00001': -1.0, 'cpd00012': -1.0, 'cpd00009': 2.0, 'cpd00067': 1.0}

In [1606]:
rxn.stoichiometry

{'cpd00011': -1.0, 'cpd00668': -1.0, 'cpd00020': 2.0, 'cpd00067': 1.0}

In [2498]:
modelseed_local

In [4299]:
stoich3 = {}

In [2401]:
stoich['0'] = rxn.stoichiometry

In [2195]:
stoich['0'] = {'t1': -1, 't2': 1}

In [2196]:
stoich['1'] = {'t2': -1, 't3': 1}

In [2210]:
stoich['2'] = {'t2': -1 , 't4': 1}

In [2436]:
map2

[{'map_name': 'new_map',
  'map_id': 'koev5uH2oDJI',
  'map_description': '',
  'homepage': 'https://escher.github.io',
  'schema': 'https://escher.github.io/escher/jsonschema/1-0-0#'},
 {'reactions': {'2': {'name': 'rxn00529',
    'bigg_id': 'rxn00529',
    'reversibility': False,
    'label_x': -281.415096282959,
    'label_y': 3685.933349609375,
    'gene_reaction_rule': '',
    'genes': [],
    'metabolites': [{'bigg_id': 'cpd00067', 'coefficient': -1},
     {'bigg_id': 'cpd00069', 'coefficient': -1},
     {'bigg_id': 'cpd00011', 'coefficient': 1},
     {'bigg_id': 'cpd00374', 'coefficient': 1}],
    'segments': {'20': {'from_node_id': '23',
      'to_node_id': '24',
      'b1': None,
      'b2': None},
     '21': {'from_node_id': '25', 'to_node_id': '24', 'b1': None, 'b2': None},
     '22': {'from_node_id': '23',
      'to_node_id': '22',
      'b1': {'x': -296.415096282959, 'y': 3613.933349609375},
      'b2': {'x': -325.31552332332205, 'y': 3594.122337857818}},
     '23': {'from

In [2019]:
escher_map2

[{'map_name': 'iMM904.Central carbon metabolism',
  'map_id': '78086bfdab8ac8a8150cf4cd5dada037',
  'map_description': 'Yeast central carbon metabolism\nLast Modified Fri Jan 09 2015 19:50:57 GMT-0800 (PST)',
  'homepage': 'https://escher.github.io',
  'schema': 'https://escher.github.io/escher/jsonschema/1-0-0#'},
 {'reactions': {'667215': {'name': 'Acetaldehyde reversible transport',
    'bigg_id': 'ACALDt',
    'segments': {'1636': {'to_node_id': '670874',
      'from_node_id': '669549',
      'b2': {'y': 5911.257, 'x': 5201.0},
      'b1': {'y': 5872.757, 'x': 5201.0}},
     '1637': {'to_node_id': '672425',
      'from_node_id': '670874',
      'b2': {'y': 6031.253, 'x': 5200.9965},
      'b1': {'y': 5985.7558, 'x': 5200.99895}}},
    'genes': [],
    'reversibility': True,
    'metabolites': [{'coefficient': 1.0, 'bigg_id': 'acald_c'},
     {'coefficient': -1.0, 'bigg_id': 'acald_e'}],
    'label_x': 5018.5439453125,
    'label_y': 5975.18717578125,
    'gene_reaction_rule': ''},


In [2497]:
cc

['cpd00035']

In [2120]:
cc = ['t2']

In [2247]:
layout

{'reactions': {'1': {'name': '0',
   'bigg_id': '0',
   'segments': {'1': {'to_node_id': '4',
     'from_node_id': '6',
     'b2': {'x': 5141.42135623731, 'y': 4858.57864376269},
     'b1': {'y': 4929.289321881345, 'x': 5070.710678118655}},
    '2': {'to_node_id': '4',
     'from_node_id': '7',
     'b2': {'x': 5212.132034355964, 'y': 4787.867965644036},
     'b1': {'y': 4717.157287525381, 'x': 5282.842712474619}},
    '3': {'to_node_id': '4',
     'from_node_id': '1',
     'b2': {'x': 5141.42135623731, 'y': 4858.57864376269},
     'b1': {'y': 4929.289321881345, 'x': 5070.710678118655}},
    '4': {'to_node_id': '2',
     'from_node_id': '5',
     'b2': {'x': 5212.132034355964, 'y': 4787.867965644036},
     'b1': {'y': 4717.157287525381, 'x': 5282.842712474619}},
    '5': {'to_node_id': '3', 'from_node_id': '4', 'b2': None, 'b1': None},
    '6': {'to_node_id': '5', 'from_node_id': '3', 'b2': None, 'b1': None}},
   'genes': [],
   'reversibility': True,
   'metabolites': [{'coefficient':

In [3664]:
layout

{'reactions': {'1': {'name': '0',
   'bigg_id': '0',
   'segments': {'2': {'to_node_id': 4,
     'from_node_id': 1,
     'b2': {'y': 5000.0, 'x': 5200.0},
     'b1': {'y': 5000.0, 'x': 5100.0}},
    '3': {'to_node_id': 3, 'from_node_id': 4, 'b2': None, 'b1': None},
    '4': {'to_node_id': 5, 'from_node_id': 3, 'b2': None, 'b1': None},
    '5': {'to_node_id': 2,
     'from_node_id': 5,
     'b2': {'y': 5000.0, 'x': 5200.0},
     'b1': {'y': 5000.0, 'x': 5100.0}},
    '6': {'to_node_id': '6',
     'from_node_id': 4,
     'b2': {'y': 5000.0, 'x': 5200.0},
     'b1': {'y': 5000.0, 'x': 5100.0}}},
   'genes': [],
   'reversibility': False,
   'metabolites': [{'coefficient': -1, 'bigg_id': 'cpd00024'}],
   'label_x': 5250.0,
   'label_y': 5050.0,
   'gene_reaction_rule': ''},
  '2': {'name': '1',
   'bigg_id': '1',
   'segments': {'8': {'to_node_id': 10,
     'from_node_id': 7,
     'b2': {'y': 5000.0, 'x': 5700.0},
     'b1': {'y': 5000.0, 'x': 5600.0}},
    '9': {'to_node_id': 9, 'from_nod

In [4421]:
escher_map2 = map2
with open('escher/maps/Saccharomyces cerevisiae/iMM904.Central carbon metabolism2.json', 'w') as fh:
    fh.write(json.dumps(escher_map2))

In [4367]:
map2 = [{"map_name":"new_map","map_id":"ChUc93PJNuFV","map_description":"","homepage":"https://escher.github.io","schema":"https://escher.github.io/escher/jsonschema/1-0-0#"},{"reactions":{"0":{"name":"rxn03397","bigg_id":"rxn03397","reversibility":false,"label_x":312.852783203125,"label_y":316.0582580566406,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00017","coefficient":-1},{"bigg_id":"cpd03447","coefficient":-1},{"bigg_id":"cpd00019","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd03448","coefficient":1}],"segments":{"0":{"from_node_id":"1","to_node_id":"2","b1":null,"b2":null},"1":{"from_node_id":"3","to_node_id":"2","b1":null,"b2":null},"2":{"from_node_id":"1","to_node_id":"0","b1":{"x":297.852783203125,"y":244.05825805664062},"b2":{"x":313.7633972167969,"y":221.62940979003906}},"3":{"from_node_id":"1","to_node_id":"43","b1":{"x":297.852783203125,"y":240.05825805664062},"b2":{"x":292.77984619140625,"y":138.90822458267212}},"4":{"from_node_id":"3","to_node_id":"5","b1":{"x":297.852783203125,"y":372.0582580566406},"b2":{"x":315.852783203125,"y":377.3082580566406}},"5":{"from_node_id":"3","to_node_id":"6","b1":{"x":297.852783203125,"y":372.0582580566406},"b2":{"x":279.852783203125,"y":377.3082580566406}},"6":{"from_node_id":"3","to_node_id":"7","b1":{"x":297.852783203125,"y":368.0582580566406},"b2":{"x":297.852783203125,"y":424.5812072753906}}}},"1":{"name":"rxn03394","bigg_id":"rxn03394","reversibility":false,"label_x":-40.90336036682129,"label_y":-982.0036315917969,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00005","coefficient":-1},{"bigg_id":"cpd00007","coefficient":-1},{"bigg_id":"cpd00067","coefficient":-1},{"bigg_id":"cpd03444","coefficient":-1},{"bigg_id":"cpd00001","coefficient":1},{"bigg_id":"cpd00006","coefficient":1},{"bigg_id":"cpd03445","coefficient":1}],"segments":{"7":{"from_node_id":"9","to_node_id":"10","b1":null,"b2":null},"8":{"from_node_id":"11","to_node_id":"10","b1":null,"b2":null},"9":{"from_node_id":"9","to_node_id":"8","b1":{"x":-55.90336036682129,"y":-1054.0036315917969},"b2":{"x":-80.59824562072754,"y":-1070.2615661621094}},"10":{"from_node_id":"9","to_node_id":"12","b1":{"x":-55.90336036682129,"y":-1058.0036315917969},"b2":{"x":-37.90336036682129,"y":-1063.2536315917969}},"11":{"from_node_id":"9","to_node_id":"13","b1":{"x":-55.90336036682129,"y":-1058.0036315917969},"b2":{"x":-77.20563697814941,"y":-1098.3184509277344}},"12":{"from_node_id":"9","to_node_id":"14","b1":{"x":-55.90336036682129,"y":-1058.0036315917969},"b2":{"x":-57.70627403259277,"y":-1138.5627136230469}},"13":{"from_node_id":"11","to_node_id":"15","b1":{"x":-55.90336036682129,"y":-926.0036315917969},"b2":{"x":-37.90336036682129,"y":-920.7536315917969}},"14":{"from_node_id":"11","to_node_id":"16","b1":{"x":-55.90336036682129,"y":-926.0036315917969},"b2":{"x":-73.90336036682129,"y":-920.7536315917969}},"15":{"from_node_id":"11","to_node_id":"17","b1":{"x":-55.90336036682129,"y":-930.0036315917969},"b2":{"x":-55.90336036682129,"y":-875.7536315917969}}}},"2":{"name":"rxn00966","bigg_id":"rxn00966","reversibility":false,"label_x":281.8619000909378,"label_y":-2258.8630498732473,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00020","coefficient":-1},{"bigg_id":"cpd00136","coefficient":-1},{"bigg_id":"cpd00216","coefficient":1}],"segments":{"16":{"from_node_id":"19","to_node_id":"20","b1":null,"b2":null},"17":{"from_node_id":"21","to_node_id":"20","b1":null,"b2":null},"18":{"from_node_id":"19","to_node_id":"18","b1":{"x":267.65057306437643,"y":-2206.868066246013},"b2":{"x":223.5065983861675,"y":-2171.3036079159338}},"19":{"from_node_id":"19","to_node_id":"82","b1":{"x":267.7014551916937,"y":-2202.8683898829627},"b2":{"x":263.349737931602,"y":-2173.562750049713}},"20":{"from_node_id":"21","to_node_id":"23","b1":{"x":266.073227117497,"y":-2330.8580335004804},"b2":{"x":267.25122603429344,"y":-2381.3673160493086}}}},"3":{"name":"rxn04139","bigg_id":"rxn04139","reversibility":false,"label_x":313.6166687011719,"label_y":684.8319396972656,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00005","coefficient":-1},{"bigg_id":"cpd00007","coefficient":-1},{"bigg_id":"cpd00067","coefficient":-3},{"bigg_id":"cpd03448","coefficient":-1},{"bigg_id":"cpd00001","coefficient":1},{"bigg_id":"cpd00006","coefficient":1},{"bigg_id":"cpd03449","coefficient":1}],"segments":{"21":{"from_node_id":"25","to_node_id":"26","b1":null,"b2":null},"22":{"from_node_id":"27","to_node_id":"26","b1":null,"b2":null},"23":{"from_node_id":"25","to_node_id":"24","b1":{"x":298.6166687011719,"y":612.8319396972656},"b2":{"x":248.6120147705078,"y":569.9466552734375}},"24":{"from_node_id":"25","to_node_id":"28","b1":{"x":298.6166687011719,"y":608.8319396972656},"b2":{"x":316.6166687011719,"y":603.5819396972656}},"25":{"from_node_id":"25","to_node_id":"29","b1":{"x":298.6166687011719,"y":608.8319396972656},"b2":{"x":253.3413848876953,"y":605.8548278808594}},"26":{"from_node_id":"25","to_node_id":"7","b1":{"x":298.6166687011719,"y":608.8319396972656},"b2":{"x":297.2087097167969,"y":545.6097717285156}},"27":{"from_node_id":"27","to_node_id":"31","b1":{"x":298.6166687011719,"y":740.8319396972656},"b2":{"x":316.6166687011719,"y":746.0819396972656}},"28":{"from_node_id":"27","to_node_id":"32","b1":{"x":298.6166687011719,"y":740.8319396972656},"b2":{"x":280.6166687011719,"y":746.0819396972656}},"29":{"from_node_id":"27","to_node_id":"33","b1":{"x":298.6166687011719,"y":736.8319396972656},"b2":{"x":298.6166687011719,"y":836.5406494140625}}}},"4":{"name":"rxn03396","bigg_id":"rxn03396","reversibility":false,"label_x":309.2174987792969,"label_y":-169.66554641723633,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00005","coefficient":-1},{"bigg_id":"cpd00007","coefficient":-1},{"bigg_id":"cpd03446","coefficient":-1},{"bigg_id":"cpd00001","coefficient":1},{"bigg_id":"cpd00006","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd03447","coefficient":1}],"segments":{"30":{"from_node_id":"35","to_node_id":"36","b1":null,"b2":null},"31":{"from_node_id":"37","to_node_id":"36","b1":null,"b2":null},"32":{"from_node_id":"35","to_node_id":"34","b1":{"x":294.2174987792969,"y":-241.66554641723633},"b2":{"x":241.93991088867188,"y":-250.45676040649414}},"33":{"from_node_id":"35","to_node_id":"38","b1":{"x":294.2174987792969,"y":-245.66554641723633},"b2":{"x":312.2174987792969,"y":-250.91554641723633}},"34":{"from_node_id":"35","to_node_id":"51","b1":{"x":294.2174987792969,"y":-245.66554641723633},"b2":{"x":295.86663818359375,"y":-306.6664047241211}},"35":{"from_node_id":"37","to_node_id":"40","b1":{"x":294.2174987792969,"y":-113.66554641723633},"b2":{"x":312.2174987792969,"y":-108.41554641723633}},"36":{"from_node_id":"37","to_node_id":"41","b1":{"x":294.2174987792969,"y":-113.66554641723633},"b2":{"x":276.2174987792969,"y":-108.41554641723633}},"37":{"from_node_id":"37","to_node_id":"42","b1":{"x":294.2174987792969,"y":-113.66554641723633},"b2":{"x":258.2174987792969,"y":-123.41554641723633}},"38":{"from_node_id":"37","to_node_id":"43","b1":{"x":294.2174987792969,"y":-117.66554641723633},"b2":{"x":294.217529296875,"y":4.772623538970947}}}},"5":{"name":"rxn03395","bigg_id":"rxn03395","reversibility":false,"label_x":-39.80930519104004,"label_y":-605.4647369384766,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00017","coefficient":-1},{"bigg_id":"cpd03445","coefficient":-1},{"bigg_id":"cpd00019","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd03446","coefficient":1}],"segments":{"39":{"from_node_id":"45","to_node_id":"46","b1":null,"b2":null},"40":{"from_node_id":"47","to_node_id":"46","b1":null,"b2":null},"41":{"from_node_id":"45","to_node_id":"44","b1":{"x":-54.80930519104004,"y":-677.4647369384766},"b2":{"x":-20.71525001525879,"y":-706.7124176025391}},"42":{"from_node_id":"45","to_node_id":"17","b1":{"x":-54.80930519104004,"y":-681.4647369384766},"b2":{"x":-54.29471778869629,"y":-744.5754451751709}},"43":{"from_node_id":"47","to_node_id":"49","b1":{"x":-54.80930519104004,"y":-549.4647369384766},"b2":{"x":-36.80930519104004,"y":-544.2147369384766}},"44":{"from_node_id":"47","to_node_id":"50","b1":{"x":-54.80930519104004,"y":-549.4647369384766},"b2":{"x":-72.80930519104004,"y":-544.2147369384766}},"45":{"from_node_id":"47","to_node_id":"51","b1":{"x":-55.82783317565918,"y":-378.3721160888672},"b2":{"x":-52.68364715576172,"y":-376.6711883544922}}}},"6":{"name":"rxn11946","bigg_id":"rxn11946","reversibility":false,"label_x":312.02095794677734,"label_y":1086.8248443603516,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00017","coefficient":-1},{"bigg_id":"cpd03449","coefficient":-1},{"bigg_id":"cpd00019","coefficient":1},{"bigg_id":"cpd00067","coefficient":3},{"bigg_id":"cpd15560","coefficient":1}],"segments":{"46":{"from_node_id":"53","to_node_id":"54","b1":null,"b2":null},"47":{"from_node_id":"55","to_node_id":"54","b1":null,"b2":null},"48":{"from_node_id":"53","to_node_id":"52","b1":{"x":297.02095794677734,"y":1014.8248443603516},"b2":{"x":315.20446014404297,"y":1015.1254425048828}},"49":{"from_node_id":"53","to_node_id":"33","b1":{"x":297.02095794677734,"y":1010.8248443603516},"b2":{"x":295.69531059265137,"y":985.2374725341797}},"50":{"from_node_id":"55","to_node_id":"57","b1":{"x":297.02095794677734,"y":1142.8248443603516},"b2":{"x":315.02095794677734,"y":1148.0748443603516}},"51":{"from_node_id":"55","to_node_id":"58","b1":{"x":297.02095794677734,"y":1142.8248443603516},"b2":{"x":279.02095794677734,"y":1148.0748443603516}},"52":{"from_node_id":"55","to_node_id":"59","b1":{"x":297.02095794677734,"y":1138.8248443603516},"b2":{"x":297.02095794677734,"y":1193.0748443603516}}}},"9":{"name":"rxn03393","bigg_id":"rxn03393","reversibility":false,"label_x":-41.57929229736328,"label_y":-1424.8287715911865,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00067","coefficient":-1},{"bigg_id":"cpd03443","coefficient":-1},{"bigg_id":"cpd00011","coefficient":1},{"bigg_id":"cpd03444","coefficient":1}],"segments":{"66":{"from_node_id":"76","to_node_id":"77","b1":null,"b2":null},"67":{"from_node_id":"78","to_node_id":"77","b1":null,"b2":null},"68":{"from_node_id":"76","to_node_id":"75","b1":{"x":-56.57929229736328,"y":-1496.8287715911865},"b2":{"x":-95.80910110473633,"y":-1500.640417098999}},"69":{"from_node_id":"76","to_node_id":"89","b1":{"x":-52.6359977722168,"y":-1694.2403926849365},"b2":{"x":-52.205817222595215,"y":-1692.677282333374}},"70":{"from_node_id":"78","to_node_id":"80","b1":{"x":-56.57929229736328,"y":-1368.8287715911865},"b2":{"x":-74.57929229736328,"y":-1363.5787715911865}},"71":{"from_node_id":"78","to_node_id":"14","b1":{"x":-56.57929229736328,"y":-1372.8287715911865},"b2":{"x":-57.46287727355957,"y":-1332.9615173339844}}}},"10":{"name":"rxn03893","bigg_id":"rxn03893","reversibility":false,"label_x":281.0681953430176,"label_y":-1861.7272090911865,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00136","coefficient":-1},{"bigg_id":"cpd02557","coefficient":-1},{"bigg_id":"cpd00012","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd03443","coefficient":1}],"segments":{"72":{"from_node_id":"83","to_node_id":"84","b1":null,"b2":null},"73":{"from_node_id":"85","to_node_id":"84","b1":null,"b2":null},"74":{"from_node_id":"83","to_node_id":"82","b1":{"x":266.0681953430176,"y":-1933.7272090911865},"b2":{"x":266.0681953430176,"y":-1987.9772090911865}},"75":{"from_node_id":"83","to_node_id":"86","b1":{"x":266.0681953430176,"y":-1937.7272090911865},"b2":{"x":248.06819534301758,"y":-1942.9772090911865}},"76":{"from_node_id":"85","to_node_id":"87","b1":{"x":266.0681953430176,"y":-1805.7272090911865},"b2":{"x":284.0681953430176,"y":-1806.0919551849365}},"77":{"from_node_id":"85","to_node_id":"88","b1":{"x":266.0681953430176,"y":-1805.7272090911865},"b2":{"x":248.06819534301758,"y":-1800.4772090911865}},"78":{"from_node_id":"85","to_node_id":"89","b1":{"x":266.0681953430176,"y":-1809.7272090911865},"b2":{"x":266.0681953430176,"y":-1755.4772090911865}}}},"11":{"name":"rxn25474","bigg_id":"rxn25474","reversibility":false,"label_x":631.4448011835274,"label_y":-1415.3229294067353,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00005","coefficient":-1},{"bigg_id":"cpd00007","coefficient":-1},{"bigg_id":"cpd00067","coefficient":-1},{"bigg_id":"cpd03443","coefficient":-1},{"bigg_id":"cpd00001","coefficient":1},{"bigg_id":"cpd00006","coefficient":1},{"bigg_id":"cpd25893","coefficient":1}],"segments":{"79":{"from_node_id":"90","to_node_id":"91","b1":null,"b2":null},"80":{"from_node_id":"92","to_node_id":"91","b1":null,"b2":null},"81":{"from_node_id":"90","to_node_id":"93","b1":{"x":605.8332578995798,"y":-1486.2362825569653},"b2":{"x":638.9769841895375,"y":-1497.6034741097026}},"82":{"from_node_id":"90","to_node_id":"94","b1":{"x":605.8332578995798,"y":-1486.2362825569653},"b2":{"x":582.1601575158414,"y":-1508.1255620384388}},"83":{"from_node_id":"90","to_node_id":"95","b1":{"x":605.8332578995798,"y":-1486.2362825569653},"b2":{"x":538.1296242842551,"y":-1487.3076492159187}},"84":{"from_node_id":"90","to_node_id":"89","b1":{"x":607.750168799956,"y":-1695.5946943925571},"b2":{"x":609.2958869934082,"y":-1696.727331161499}},"85":{"from_node_id":"92","to_node_id":"96","b1":{"x":607.3311247409122,"y":-1354.2447813346298},"b2":{"x":621.604263603799,"y":-1352.8998229041142}},"86":{"from_node_id":"92","to_node_id":"97","b1":{"x":607.3311247409122,"y":-1354.2447813346298},"b2":{"x":589.3918579145109,"y":-1348.7908647849242}},"87":{"from_node_id":"92","to_node_id":"98","b1":{"x":607.2857348366299,"y":-1358.2445237959137},"b2":{"x":607.9013354134659,"y":-1303.9980166647636}}}},"12":{"name":"rxn22114","bigg_id":"rxn22114","reversibility":false,"label_x":623.5679996326196,"label_y":-1060.2517992646722,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00017","coefficient":-1},{"bigg_id":"cpd25893","coefficient":-1},{"bigg_id":"cpd00019","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd25898","coefficient":1}],"segments":{"88":{"from_node_id":"99","to_node_id":"100","b1":null,"b2":null},"89":{"from_node_id":"101","to_node_id":"100","b1":null,"b2":null},"90":{"from_node_id":"99","to_node_id":"102","b1":{"x":608.5679996326196,"y":-1136.2517992646722},"b2":{"x":590.5679996326196,"y":-1141.5017992646722}},"91":{"from_node_id":"99","to_node_id":"98","b1":{"x":608.5679996326196,"y":-1132.2517992646722},"b2":{"x":608.5679996326196,"y":-1186.5017992646722}},"92":{"from_node_id":"101","to_node_id":"103","b1":{"x":608.5679996326196,"y":-1004.2517992646722},"b2":{"x":626.5679996326196,"y":-999.0017992646722}},"93":{"from_node_id":"101","to_node_id":"104","b1":{"x":608.5679996326196,"y":-1004.2517992646722},"b2":{"x":590.5679996326196,"y":-999.0017992646722}},"94":{"from_node_id":"101","to_node_id":"105","b1":{"x":608.5679996326196,"y":-1008.2517992646722},"b2":{"x":608.5679385974634,"y":-888.4970080049065}}}},"13":{"name":"rxn25479","bigg_id":"rxn25479","reversibility":false,"label_x":625.4395206287134,"label_y":-648.4901720674065,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00067","coefficient":-1},{"bigg_id":"cpd25898","coefficient":-1},{"bigg_id":"cpd00011","coefficient":1},{"bigg_id":"cpd03446","coefficient":1}],"segments":{"95":{"from_node_id":"106","to_node_id":"107","b1":null,"b2":null},"96":{"from_node_id":"108","to_node_id":"107","b1":null,"b2":null},"97":{"from_node_id":"106","to_node_id":"109","b1":{"x":610.4395206287134,"y":-724.4901720674065},"b2":{"x":592.4395206287134,"y":-729.7401720674065}},"98":{"from_node_id":"106","to_node_id":"105","b1":{"x":610.4395206287134,"y":-720.4901720674065},"b2":{"x":608.5679385974634,"y":-770.9970080049065}},"99":{"from_node_id":"108","to_node_id":"110","b1":{"x":610.4395206287134,"y":-592.4901720674065},"b2":{"x":592.4395206287134,"y":-587.2401720674065}},"100":{"from_node_id":"108","to_node_id":"51","b1":{"x":610.4395206287134,"y":-379.38873163771905},"b2":{"x":610.4395206287134,"y":-377.54254023146905}}}}},"nodes":{"0":{"node_type":"metabolite","x":384.2244873046875,"y":199.24642944335938,"bigg_id":"cpd00017","name":"S-Adenosyl-L-methionine","label_x":409.2244873046875,"label_y":199.24642944335938,"node_is_primary":false},"1":{"node_type":"multimarker","x":297.852783203125,"y":286.0582580566406},"2":{"node_type":"midmarker","x":297.852783203125,"y":306.0582580566406},"3":{"node_type":"multimarker","x":297.852783203125,"y":326.0582580566406},"5":{"node_type":"metabolite","x":377.852783203125,"y":411.0582580566406,"bigg_id":"cpd00019","name":"S-Adenosyl-homocysteine","label_x":392.852783203125,"label_y":411.0582580566406,"node_is_primary":false},"6":{"node_type":"metabolite","x":217.852783203125,"y":411.0582580566406,"bigg_id":"cpd00067","name":"H+","label_x":145.852783203125,"label_y":441.0582580566406,"node_is_primary":false},"7":{"node_type":"metabolite","x":297.852783203125,"y":483.3312072753906,"bigg_id":"cpd03448","name":"2-Octaprenyl-3-methyl-6-methoxy-1,4-benzoquinone","label_x":322.852783203125,"label_y":483.3312072753906,"node_is_primary":true},"8":{"node_type":"metabolite","x":-167.98015022277832,"y":-1073.9229431152344,"bigg_id":"cpd00005","name":"NADPH","label_x":-272.153413772583,"label_y":-1056.8263854980469,"node_is_primary":false},"9":{"node_type":"multimarker","x":-55.90336036682129,"y":-1012.0036315917969},"10":{"node_type":"midmarker","x":-55.90336036682129,"y":-992.0036315917969},"11":{"node_type":"multimarker","x":-55.90336036682129,"y":-972.0036315917969},"12":{"node_type":"metabolite","x":24.09663963317871,"y":-1097.0036315917969,"bigg_id":"cpd00007","name":"O2","label_x":39.09663963317871,"label_y":-1097.0036315917969,"node_is_primary":false},"13":{"node_type":"metabolite","x":-139.2056369781494,"y":-1132.0684509277344,"bigg_id":"cpd00067","name":"H+","label_x":-184.0803928375244,"label_y":-1115.6310729980469,"node_is_primary":false},"14":{"node_type":"metabolite","x":-58.23185157775879,"y":-1235.6985778808594,"bigg_id":"cpd03444","name":"2-Octaprenylphenol","label_x":-150.8190574645996,"label_y":-1230.0289001464844,"node_is_primary":true},"15":{"node_type":"metabolite","x":24.09663963317871,"y":-887.0036315917969,"bigg_id":"cpd00001","name":"H2O","label_x":39.09663963317871,"label_y":-887.0036315917969,"node_is_primary":false},"16":{"node_type":"metabolite","x":-135.9033603668213,"y":-887.0036315917969,"bigg_id":"cpd00006","name":"NADP","label_x":-207.9033603668213,"label_y":-857.0036315917969,"node_is_primary":false},"17":{"node_type":"metabolite","x":-55.90336036682129,"y":-817.0036315917969,"bigg_id":"cpd03445","name":"2-Octaprenyl-6-hydroxyphenol","label_x":-30.90336036682129,"label_y":-817.0036315917969,"node_is_primary":true},"18":{"node_type":"metabolite","x":164.47516681377772,"y":-2155.5241816461757,"bigg_id":"cpd00020","name":"Pyruvate","label_x":107.27938129131678,"label_y":-2181.6772578180507,"node_is_primary":false},"19":{"node_type":"multimarker","x":267.11631072752925,"y":-2248.86466805801},"20":{"node_type":"midmarker","x":266.8619000909378,"y":-2268.8630498732473},"21":{"node_type":"multimarker","x":266.6074894543447,"y":-2288.8614316884837},"23":{"node_type":"metabolite","x":266.50389478930083,"y":-2440.112562631567,"bigg_id":"cpd00216","name":"Chorismate","label_x":291.50389478930083,"label_y":-2440.112562631567,"node_is_primary":true},"24":{"node_type":"metabolite","x":178.15090942382812,"y":538.471923828125,"bigg_id":"cpd00005","name":"NADPH","label_x":82.68514251708984,"label_y":545.290771484375,"node_is_primary":false},"25":{"node_type":"multimarker","x":298.6166687011719,"y":654.8319396972656},"26":{"node_type":"midmarker","x":298.6166687011719,"y":674.8319396972656},"27":{"node_type":"multimarker","x":298.6166687011719,"y":694.8319396972656},"28":{"node_type":"metabolite","x":378.6166687011719,"y":569.8319396972656,"bigg_id":"cpd00007","name":"O2","label_x":393.6166687011719,"label_y":569.8319396972656,"node_is_primary":false},"29":{"node_type":"metabolite","x":173.1578826904297,"y":590.2883605957031,"bigg_id":"cpd00067","name":"H+","label_x":121.61433410644531,"label_y":606.6507263183594,"node_is_primary":false},"31":{"node_type":"metabolite","x":378.6166687011719,"y":779.8319396972656,"bigg_id":"cpd00001","name":"H2O","label_x":393.6166687011719,"label_y":779.8319396972656,"node_is_primary":false},"32":{"node_type":"metabolite","x":218.61666870117188,"y":779.8319396972656,"bigg_id":"cpd00006","name":"NADP","label_x":146.61666870117188,"label_y":809.8319396972656,"node_is_primary":false},"33":{"node_type":"metabolite","x":298.6166687011719,"y":895.2906494140625,"bigg_id":"cpd03449","name":"2-Octaprenyl-3-methyl-5-hydroxy-6-methoxy-1,4-benzoquinone","label_x":323.6166687011719,"label_y":895.2906494140625,"node_is_primary":true},"34":{"node_type":"metabolite","x":144.2035369873047,"y":-266.02092361450195,"bigg_id":"cpd00005","name":"NADPH","label_x":110.10713195800781,"label_y":-218.28918838500977,"node_is_primary":false},"35":{"node_type":"multimarker","x":294.2174987792969,"y":-199.66554641723633},"36":{"node_type":"midmarker","x":294.2174987792969,"y":-179.66554641723633},"37":{"node_type":"multimarker","x":294.2174987792969,"y":-159.66554641723633},"38":{"node_type":"metabolite","x":374.2174987792969,"y":-284.6655464172363,"bigg_id":"cpd00007","name":"O2","label_x":389.2174987792969,"label_y":-284.6655464172363,"node_is_primary":false},"40":{"node_type":"metabolite","x":374.2174987792969,"y":-74.66554641723633,"bigg_id":"cpd00001","name":"H2O","label_x":389.2174987792969,"label_y":-74.66554641723633,"node_is_primary":false},"41":{"node_type":"metabolite","x":214.21749877929688,"y":-74.66554641723633,"bigg_id":"cpd00006","name":"NADP","label_x":142.21749877929688,"label_y":-44.66554641723633,"node_is_primary":false},"42":{"node_type":"metabolite","x":134.21749877929688,"y":-94.66554641723633,"bigg_id":"cpd00067","name":"H+","label_x":80.40100860595703,"label_y":-98.7596321105957,"node_is_primary":false},"43":{"node_type":"metabolite","x":294.217529296875,"y":63.52262353897095,"bigg_id":"cpd03447","name":"2-Octaprenyl-6-methoxy-1,4-benzoquinone","label_x":319.217529296875,"label_y":63.52262353897095,"node_is_primary":true},"44":{"node_type":"metabolite","x":38.381155014038086,"y":-722.2765655517578,"bigg_id":"cpd00017","name":"S-Adenosyl-L-methionine","label_x":63.381155014038086,"label_y":-722.2765655517578,"node_is_primary":false},"45":{"node_type":"multimarker","x":-54.80930519104004,"y":-635.4647369384766},"46":{"node_type":"midmarker","x":-54.80930519104004,"y":-615.4647369384766},"47":{"node_type":"multimarker","x":-54.80930519104004,"y":-595.4647369384766},"49":{"node_type":"metabolite","x":25.19069480895996,"y":-510.46473693847656,"bigg_id":"cpd00019","name":"S-Adenosyl-homocysteine","label_x":40.19069480895996,"label_y":-510.46473693847656,"node_is_primary":false},"50":{"node_type":"metabolite","x":-134.80930519104004,"y":-510.46473693847656,"bigg_id":"cpd00067","name":"H+","label_x":-206.80930519104004,"label_y":-480.46473693847656,"node_is_primary":false},"51":{"node_type":"metabolite","x":293.4345397949219,"y":-373.8769989013672,"bigg_id":"cpd03446","name":"2-Octaprenyl-6-methoxyphenol","label_x":149.99378967285156,"label_y":-400.0789031982422,"node_is_primary":true},"52":{"node_type":"metabolite","x":378.84676361083984,"y":981.3777618408203,"bigg_id":"cpd00017","name":"S-Adenosyl-L-methionine","label_x":403.84676361083984,"label_y":981.3777618408203,"node_is_primary":false},"53":{"node_type":"multimarker","x":297.02095794677734,"y":1056.8248443603516},"54":{"node_type":"midmarker","x":297.02095794677734,"y":1076.8248443603516},"55":{"node_type":"multimarker","x":297.02095794677734,"y":1096.8248443603516},"57":{"node_type":"metabolite","x":377.02095794677734,"y":1181.8248443603516,"bigg_id":"cpd00019","name":"S-Adenosyl-homocysteine","label_x":392.02095794677734,"label_y":1181.8248443603516,"node_is_primary":false},"58":{"node_type":"metabolite","x":217.02095794677734,"y":1181.8248443603516,"bigg_id":"cpd00067","name":"H+","label_x":145.02095794677734,"label_y":1211.8248443603516,"node_is_primary":false},"59":{"node_type":"metabolite","x":297.02095794677734,"y":1251.8248443603516,"bigg_id":"cpd15560","name":"Ubiquinone-8","label_x":322.02095794677734,"label_y":1251.8248443603516,"node_is_primary":true},"75":{"node_type":"metabolite","x":-142.51125717163086,"y":-1516.424352645874,"bigg_id":"cpd00067","name":"H+","label_x":-117.51125717163086,"label_y":-1516.424352645874,"node_is_primary":false},"76":{"node_type":"multimarker","x":-56.57929229736328,"y":-1454.8287715911865},"77":{"node_type":"midmarker","x":-56.57929229736328,"y":-1434.8287715911865},"78":{"node_type":"multimarker","x":-56.57929229736328,"y":-1414.8287715911865},"80":{"node_type":"metabolite","x":-136.57929229736328,"y":-1329.8287715911865,"bigg_id":"cpd00011","name":"CO2","label_x":-208.57929229736328,"label_y":-1299.8287715911865,"node_is_primary":false},"82":{"node_type":"metabolite","x":266.0681953430176,"y":-2046.7272090911865,"bigg_id":"cpd00136","name":"4-Hydroxybenzoate","label_x":291.0681953430176,"label_y":-2046.7272090911865,"node_is_primary":true},"83":{"node_type":"multimarker","x":266.0681953430176,"y":-1891.7272090911865},"84":{"node_type":"midmarker","x":266.0681953430176,"y":-1871.7272090911865},"85":{"node_type":"multimarker","x":266.0681953430176,"y":-1851.7272090911865},"86":{"node_type":"metabolite","x":186.06819534301758,"y":-1976.7272090911865,"bigg_id":"cpd02557","name":"Farnesylfarnesylgeraniol","label_x":114.06819534301758,"label_y":-1946.7272090911865,"node_is_primary":false},"87":{"node_type":"metabolite","x":346.0681953430176,"y":-1772.3419551849365,"bigg_id":"cpd00012","name":"PPi","label_x":361.0681953430176,"label_y":-1772.3419551849365,"node_is_primary":false},"88":{"node_type":"metabolite","x":186.06819534301758,"y":-1766.7272090911865,"bigg_id":"cpd00067","name":"H+","label_x":114.06819534301758,"label_y":-1736.7272090911865,"node_is_primary":false},"89":{"node_type":"metabolite","x":266.0681953430176,"y":-1696.7272090911865,"bigg_id":"cpd03443","name":"3-Octaprenyl-4-hydroxybenzoate","label_x":135.72836685180664,"label_y":-1659.295934677124,"node_is_primary":true},"90":{"node_type":"multimarker","x":606.3552417988327,"y":-1440.2392442522125},"91":{"node_type":"midmarker","x":606.5821913202461,"y":-1420.2405319457978},"92":{"node_type":"multimarker","x":606.8091408416602,"y":-1400.2418196393837},"93":{"node_type":"metabolite","x":694.9753299634424,"y":-1509.5961044087865,"bigg_id":"cpd00005","name":"NADPH","label_x":651.7753299634423,"label_y":-1529.5961044087865,"node_is_primary":false},"94":{"node_type":"metabolite","x":531.0105421657582,"y":-1526.197433429755,"bigg_id":"cpd00007","name":"O2","label_x":487.81054216575825,"label_y":-1523.6287078438174,"node_is_primary":false},"95":{"node_type":"metabolite","x":464.3435946099529,"y":-1494.061552920496,"bigg_id":"cpd00067","name":"H+","label_x":426.75827966854666,"label_y":-1470.9056691314336,"node_is_primary":false},"96":{"node_type":"metabolite","x":683.983249071071,"y":-1319.855539403422,"bigg_id":"cpd00001","name":"H2O","label_x":642.6547700671647,"label_y":-1358.571115575297,"node_is_primary":false},"97":{"node_type":"metabolite","x":527.7788270820126,"y":-1314.339494251465,"bigg_id":"cpd00006","name":"NADP","label_x":484.5788270820126,"label_y":-1274.339494251465,"node_is_primary":false},"98":{"node_type":"metabolite","x":608.5679996326196,"y":-1245.2517992646722,"bigg_id":"cpd25893","name":"3,4-dihydroxy-5-all-trans-octaprenylbenzoate","label_x":408.15657385136956,"label_y":-1235.1967455537347,"node_is_primary":true},"99":{"node_type":"multimarker","x":608.5679996326196,"y":-1090.2517992646722},"100":{"node_type":"midmarker","x":608.5679996326196,"y":-1070.2517992646722},"101":{"node_type":"multimarker","x":608.5679996326196,"y":-1050.2517992646722},"102":{"node_type":"metabolite","x":528.5679996326196,"y":-1175.2517992646722,"bigg_id":"cpd00017","name":"S-Adenosyl-L-methionine","label_x":409.7789066150415,"label_y":-1145.2517992646722,"node_is_primary":false},"103":{"node_type":"metabolite","x":688.5679996326196,"y":-965.2517992646722,"bigg_id":"cpd00019","name":"S-Adenosyl-homocysteine","label_x":703.5679996326196,"label_y":-965.2517992646722,"node_is_primary":false},"104":{"node_type":"metabolite","x":528.5679996326196,"y":-965.2517992646722,"bigg_id":"cpd00067","name":"H+","label_x":456.5679996326196,"label_y":-935.2517992646722,"node_is_primary":false},"105":{"node_type":"metabolite","x":608.5679385974634,"y":-829.7470080049065,"bigg_id":"cpd25898","name":"3-(3,7,11,15,19,23-octamethyltetracosa-2,6,10,14,18,22-hexaenyl) -4-hydroxy-5-methoxy-benzoic acid","label_x":328.5030582263696,"label_y":-826.0038439424065,"node_is_primary":true},"106":{"node_type":"multimarker","x":610.4395206287134,"y":-678.4901720674065},"107":{"node_type":"midmarker","x":610.4395206287134,"y":-658.4901720674065},"108":{"node_type":"multimarker","x":610.4395206287134,"y":-638.4901720674065},"109":{"node_type":"metabolite","x":530.4395206287134,"y":-763.4901720674065,"bigg_id":"cpd00067","name":"H+","label_x":458.43952062871335,"label_y":-733.4901720674065,"node_is_primary":false},"110":{"node_type":"metabolite","x":530.4395206287134,"y":-553.4901720674065,"bigg_id":"cpd00011","name":"CO2","label_x":458.43952062871335,"label_y":-523.4901720674065,"node_is_primary":false}},"text_labels":{"0":{"x":-31.314170837402344,"y":-2651.60888671875,"text":"Ubiquinone Biosynthesis"}},"canvas":{"x":-726.8074951171875,"y":-2955.438720703125,"width":2063.7171630859375,"height":4424.18896484375}}]

In [4298]:
map2 = [{"map_name": "new_map", "map_id": "lLGzTsO3Cyot", "map_description": "\nLast Modified Tue Jun 11 2019 14:05:27 GMT-0500 (Central Daylight Time)", "homepage": "https://escher.github.io", "schema": "https://escher.github.io/escher/jsonschema/1-0-0#"}, {"reactions": {"2": {"name": "rxn00283", "bigg_id": "rxn00283", "reversibility": false, "label_x": -378.9999694824219, "label_y": 767, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00035", "coefficient": -1}, {"bigg_id": "cpd00117", "coefficient": 1}], "segments": {"12": {"from_node_id": "15", "to_node_id": "16", "b1": null, "b2": null}, "13": {"from_node_id": "17", "to_node_id": "16", "b1": null, "b2": null}, "14": {"from_node_id": "15", "to_node_id": "14", "b1": {"x": -393.9999694824219, "y": 695}, "b2": {"x": -393.9999694824219, "y": 640.75}}, "15": {"from_node_id": "17", "to_node_id": "18", "b1": {"x": -393.9999694824219, "y": 819}, "b2": {"x": -393.9999694824219, "y": 873.25}}}}, "4": {"name": "rxn00904", "bigg_id": "rxn00904", "reversibility": false, "label_x": -378.9999694824219, "label_y": 417, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00020", "coefficient": -1}, {"bigg_id": "cpd00156", "coefficient": -1}, {"bigg_id": "cpd00035", "coefficient": 1}, {"bigg_id": "cpd00123", "coefficient": 1}], "segments": {"22": {"from_node_id": "26", "to_node_id": "27", "b1": null, "b2": null}, "23": {"from_node_id": "28", "to_node_id": "27", "b1": null, "b2": null}, "24": {"from_node_id": "26", "to_node_id": "29", "b1": {"x": -393.9999694824219, "y": 341}, "b2": {"x": -375.9999694824219, "y": 335.75}}, "25": {"from_node_id": "26", "to_node_id": "30", "b1": {"x": -393.9999694824219, "y": 345}, "b2": {"x": -393.9999694824219, "y": 290.75}}, "26": {"from_node_id": "28", "to_node_id": "14", "b1": {"x": -393.9999694824219, "y": 469}, "b2": {"x": -393.9999694824219, "y": 523.25}}, "27": {"from_node_id": "28", "to_node_id": "31", "b1": {"x": -393.9999694824219, "y": 473}, "b2": {"x": -375.9999694824219, "y": 478.25}}}}, "5": {"name": "rxn00903", "bigg_id": "rxn00903", "reversibility": false, "label_x": -378.9999694824219, "label_y": 67, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00024", "coefficient": -1}, {"bigg_id": "cpd00156", "coefficient": -1}, {"bigg_id": "cpd00023", "coefficient": 1}, {"bigg_id": "cpd00123", "coefficient": 1}], "segments": {"28": {"from_node_id": "32", "to_node_id": "33", "b1": null, "b2": null}, "29": {"from_node_id": "34", "to_node_id": "33", "b1": null, "b2": null}, "30": {"from_node_id": "32", "to_node_id": "35", "b1": {"x": -393.9999694824219, "y": 123}, "b2": {"x": -375.9999694824219, "y": 128.25}}, "31": {"from_node_id": "32", "to_node_id": "30", "b1": {"x": -393.9999694824219, "y": 119}, "b2": {"x": -393.9999694824219, "y": 173.25}}, "32": {"from_node_id": "34", "to_node_id": "36", "b1": {"x": -393.9999694824219, "y": -5}, "b2": {"x": -365.0718028977366, "y": -28.538923188751095}}, "33": {"from_node_id": "34", "to_node_id": "37", "b1": {"x": -393.9999694824219, "y": -9}, "b2": {"x": -394.8191047060389, "y": -16.465505058936202}}}}, "6": {"name": "rxn07292", "bigg_id": "rxn07292", "reversibility": false, "label_x": -618.9999694824219, "label_y": 542, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00084", "coefficient": -1}, {"bigg_id": "cpd14547", "coefficient": -1}, {"bigg_id": "cpd00035", "coefficient": 1}, {"bigg_id": "cpd14548", "coefficient": 1}], "segments": {"34": {"from_node_id": "38", "to_node_id": "39", "b1": null, "b2": null}, "35": {"from_node_id": "40", "to_node_id": "39", "b1": null, "b2": null}, "36": {"from_node_id": "38", "to_node_id": "41", "b1": {"x": -634.9999694824219, "y": 582}, "b2": {"x": -672.8771600962205, "y": 583.5044735311532}}, "37": {"from_node_id": "38", "to_node_id": "42", "b1": {"x": -630.9999694824219, "y": 582}, "b2": {"x": -649.119464412386, "y": 551.1994883450482}}, "38": {"from_node_id": "40", "to_node_id": "14", "b1": {"x": -506.9999694824219, "y": 582}, "b2": {"x": -452.7499694824219, "y": 582}}, "39": {"from_node_id": "40", "to_node_id": "43", "b1": {"x": -502.9999694824219, "y": 582}, "b2": {"x": -497.7499694824219, "y": 564}}}}, "7": {"name": "rxn00191", "bigg_id": "rxn00191", "reversibility": false, "label_x": -268.9999694824219, "label_y": 542, "gene_reaction_rule": "", "genes": [], "metabolites": [{"bigg_id": "cpd00024", "coefficient": -1}, {"bigg_id": "cpd00035", "coefficient": -1}, {"bigg_id": "cpd00020", "coefficient": 1}, {"bigg_id": "cpd00023", "coefficient": 1}], "segments": {"40": {"from_node_id": "44", "to_node_id": "45", "b1": null, "b2": null}, "41": {"from_node_id": "46", "to_node_id": "45", "b1": null, "b2": null}, "42": {"from_node_id": "44", "to_node_id": "47", "b1": {"x": -284.9999694824219, "y": 582}, "b2": {"x": -290.2499694824219, "y": 600}}, "43": {"from_node_id": "44", "to_node_id": "14", "b1": {"x": -280.9999694824219, "y": 582}, "b2": {"x": -335.2499694824219, "y": 582}}, "44": {"from_node_id": "46", "to_node_id": "48", "b1": {"x": -152.99996948242188, "y": 582}, "b2": {"x": -141.66637696200453, "y": 580.9015238972845}}, "45": {"from_node_id": "46", "to_node_id": "49", "b1": {"x": -156.99996948242188, "y": 582}, "b2": {"x": -152.25396188800758, "y": 613.3914219455767}}}}}, "nodes": {"14": {"node_type": "metabolite", "x": -393.9999694824219, "y": 582, "bigg_id": "cpd00035", "name": "L-Alanine", "label_x": -436.0650939941406, "label_y": 608.8260498046875, "node_is_primary": true}, "15": {"node_type": "multimarker", "x": -393.9999694824219, "y": 737}, "16": {"node_type": "midmarker", "x": -393.9999694824219, "y": 757}, "17": {"node_type": "multimarker", "x": -393.9999694824219, "y": 777}, "18": {"node_type": "metabolite", "x": -393.9999694824219, "y": 932, "bigg_id": "cpd00117", "name": "D-Alanine", "label_x": -368.9999694824219, "label_y": 932, "node_is_primary": true}, "26": {"node_type": "multimarker", "x": -393.9999694824219, "y": 387}, "27": {"node_type": "midmarker", "x": -393.9999694824219, "y": 407}, "28": {"node_type": "multimarker", "x": -393.9999694824219, "y": 427}, "29": {"node_type": "metabolite", "x": -313.9999694824219, "y": 302, "bigg_id": "cpd00020", "name": "Pyruvate", "label_x": -298.9999694824219, "label_y": 302, "node_is_primary": false}, "30": {"node_type": "metabolite", "x": -393.9999694824219, "y": 232, "bigg_id": "cpd00156", "name": "L-Valine", "label_x": -368.9999694824219, "label_y": 232, "node_is_primary": true}, "31": {"node_type": "metabolite", "x": -313.9999694824219, "y": 512, "bigg_id": "cpd00123", "name": "3-Methyl-2-oxobutanoate", "label_x": -298.9999694824219, "label_y": 512, "node_is_primary": false}, "32": {"node_type": "multimarker", "x": -393.9999694824219, "y": 77}, "33": {"node_type": "midmarker", "x": -393.9999694824219, "y": 57}, "34": {"node_type": "multimarker", "x": -393.9999694824219, "y": 37}, "35": {"node_type": "metabolite", "x": -313.9999694824219, "y": 162, "bigg_id": "cpd00024", "name": "2-Oxoglutarate", "label_x": -298.9999694824219, "label_y": 162, "node_is_primary": false}, "36": {"node_type": "metabolite", "x": -309.9999694824219, "y": -49, "bigg_id": "cpd00023", "name": "L-Glutamate", "label_x": -284.9999694824219, "label_y": -49, "node_is_primary": false}, "37": {"node_type": "metabolite", "x": -391.9999694824219, "y": -87, "bigg_id": "cpd00123", "name": "3-Methyl-2-oxobutanoate", "label_x": -376.9999694824219, "label_y": -87, "node_is_primary": true}, "38": {"node_type": "multimarker", "x": -588.9999694824219, "y": 582}, "39": {"node_type": "midmarker", "x": -568.9999694824219, "y": 582}, "40": {"node_type": "multimarker", "x": -548.9999694824219, "y": 582}, "41": {"node_type": "metabolite", "x": -743.4676208496094, "y": 583.7266235351562, "bigg_id": "cpd00084", "name": "L-Cysteine", "label_x": -786.6676208496094, "label_y": 563.7266235351562, "node_is_primary": true}, "42": {"node_type": "metabolite", "x": -674.5323181152344, "y": 498.23016357421875, "bigg_id": "cpd14547", "name": "C15811", "label_x": -717.7323181152344, "label_y": 538.2301635742188, "node_is_primary": false}, "43": {"node_type": "metabolite", "x": -463.9999694824219, "y": 502, "bigg_id": "cpd14548", "name": "C15812", "label_x": -507.19996948242186, "label_y": 482, "node_is_primary": false}, "44": {"node_type": "multimarker", "x": -238.99996948242188, "y": 582}, "45": {"node_type": "midmarker", "x": -218.99996948242188, "y": 582}, "46": {"node_type": "multimarker", "x": -198.99996948242188, "y": 582}, "47": {"node_type": "metabolite", "x": -323.9999694824219, "y": 662, "bigg_id": "cpd00024", "name": "2-Oxoglutarate", "label_x": -367.19996948242186, "label_y": 702, "node_is_primary": false}, "48": {"node_type": "metabolite", "x": -71.07829284667969, "y": 581.5219116210938, "bigg_id": "cpd00020", "name": "Pyruvate", "label_x": -114.27829284667968, "label_y": 621.5219116210938, "node_is_primary": false}, "49": {"node_type": "metabolite", "x": -124.47811889648438, "y": 665.1607666015625, "bigg_id": "cpd00023", "name": "L-Glutamate", "label_x": -167.67811889648436, "label_y": 705.1607666015625, "node_is_primary": false}}, "text_labels": {"0": {"x": -625.5147705078125, "y": -252.08319091796875, "text": "Alanine Biosynthesis"}}, "canvas": {"x": -1048, "y": -476.61676025390625, "width": 1343.9468383789062, "height": 1652.9218139648438}}]

In [4416]:
map2 = [{"map_name":"new_map","map_id":"9YGMK6mZMDvw","map_description":"","homepage":"https://escher.github.io","schema":"https://escher.github.io/escher/jsonschema/1-0-0#"},{"reactions":{"0":{"name":"rxn00293","bigg_id":"rxn00293","reversibility":false,"label_x":-289.6517333984375,"label_y":-145.3863983154297,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00062","coefficient":-1},{"bigg_id":"cpd02611","coefficient":-1},{"bigg_id":"cpd00012","coefficient":1},{"bigg_id":"cpd00037","coefficient":1}],"segments":{"0":{"from_node_id":"1","to_node_id":"2","b1":null,"b2":null},"1":{"from_node_id":"3","to_node_id":"2","b1":null,"b2":null},"2":{"from_node_id":"1","to_node_id":"0","b1":{"x":-304.6517333984375,"y":-217.3863983154297},"b2":{"x":-329.9708557128906,"y":-217.00038146972656}},"3":{"from_node_id":"1","to_node_id":"4","b1":{"x":-304.6517333984375,"y":-221.3863983154297},"b2":{"x":-306.6606750488281,"y":-254.62074279785156}},"4":{"from_node_id":"3","to_node_id":"5","b1":{"x":-304.6517333984375,"y":-89.38639831542969},"b2":{"x":-322.6517333984375,"y":-84.13639831542969}},"5":{"from_node_id":"3","to_node_id":"31","b1":{"x":-304.6517333984375,"y":-93.38639831542969},"b2":{"x":-304.6517333984375,"y":-39.13639831542969}}}},"1":{"name":"rxn01483","bigg_id":"rxn01483","reversibility":false,"label_x":-428.72580384817286,"label_y":-1192.1574273735353,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00022","coefficient":-1},{"bigg_id":"cpd00288","coefficient":-1},{"bigg_id":"cpd00010","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd00293","coefficient":1}],"segments":{"6":{"from_node_id":"8","to_node_id":"9","b1":null,"b2":null},"7":{"from_node_id":"10","to_node_id":"9","b1":null,"b2":null},"8":{"from_node_id":"8","to_node_id":"7","b1":{"x":-444.250888296424,"y":-1140.1596509108094},"b2":{"x":-461.92627957145487,"y":-1138.8836424043616}},"9":{"from_node_id":"8","to_node_id":"42","b1":{"x":-441.636174624548,"y":-1072.5932660445155},"b2":{"x":-442.2214746140397,"y":-1068.51522920584}},"10":{"from_node_id":"10","to_node_id":"12","b1":{"x":-443.2007193999229,"y":-1264.155203836262},"b2":{"x":-479.8217758787986,"y":-1276.0253285536478}},"11":{"from_node_id":"10","to_node_id":"13","b1":{"x":-443.1668429839065,"y":-1268.1550603822438},"b2":{"x":-414.7711487073065,"y":-1273.5573159709193}},"12":{"from_node_id":"10","to_node_id":"14","b1":{"x":-443.1668429839065,"y":-1268.1550603822438},"b2":{"x":-443.66330893409054,"y":-1285.1708852580234}}}},"2":{"name":"rxn02285","bigg_id":"rxn02285","reversibility":false,"label_x":-285.48439025878906,"label_y":567.8349151611328,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00006","coefficient":-1},{"bigg_id":"cpd00773","coefficient":-1},{"bigg_id":"cpd00005","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd02820","coefficient":1}],"segments":{"13":{"from_node_id":"16","to_node_id":"17","b1":null,"b2":null},"14":{"from_node_id":"18","to_node_id":"17","b1":null,"b2":null},"15":{"from_node_id":"16","to_node_id":"15","b1":{"x":-299.0311919399217,"y":619.8178822944193},"b2":{"x":-283.1011961452879,"y":622.0820312704195}},"16":{"from_node_id":"16","to_node_id":"19","b1":{"x":-298.9374372096721,"y":623.8167833997921},"b2":{"x":-300.80812920086777,"y":651.2974365136906}},"17":{"from_node_id":"18","to_node_id":"20","b1":{"x":-301.9375885776564,"y":495.85194802784633},"b2":{"x":-279.2225800998294,"y":492.25509641512735}},"18":{"from_node_id":"18","to_node_id":"21","b1":{"x":-302.031343307906,"y":491.8530469224735},"b2":{"x":-320.14945136553837,"y":487.02638550779375}},"19":{"from_node_id":"18","to_node_id":"37","b1":{"x":-302.031343307906,"y":491.8530469224735},"b2":{"x":-302.4933319091797,"y":523.8105010986328}}}},"3":{"name":"rxn03638","bigg_id":"rxn03638","reversibility":false,"label_x":-291.2892875671387,"label_y":-531.5687103271484,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00022","coefficient":-1},{"bigg_id":"cpd03671","coefficient":-1},{"bigg_id":"cpd00010","coefficient":1},{"bigg_id":"cpd00067","coefficient":1},{"bigg_id":"cpd02611","coefficient":1}],"segments":{"20":{"from_node_id":"24","to_node_id":"25","b1":null,"b2":null},"21":{"from_node_id":"26","to_node_id":"25","b1":null,"b2":null},"22":{"from_node_id":"24","to_node_id":"23","b1":{"x":-306.2892875671387,"y":-603.5687103271484},"b2":{"x":-340.9365043640137,"y":-613.8433685302734}},"23":{"from_node_id":"24","to_node_id":"27","b1":{"x":-306.2892875671387,"y":-607.5687103271484},"b2":{"x":-306.9656791687012,"y":-631.4748992919922}},"24":{"from_node_id":"26","to_node_id":"28","b1":{"x":-306.2892875671387,"y":-479.56871032714844},"b2":{"x":-335.6061820983887,"y":-473.29176330566406}},"25":{"from_node_id":"26","to_node_id":"29","b1":{"x":-306.2892875671387,"y":-475.56871032714844},"b2":{"x":-288.2892875671387,"y":-470.31871032714844}},"26":{"from_node_id":"26","to_node_id":"4","b1":{"x":-306.2892875671387,"y":-475.56871032714844},"b2":{"x":-308.2982597351074,"y":-438.3367462158203}}}},"4":{"name":"rxn00461","bigg_id":"rxn00461","reversibility":false,"label_x":-290,"label_y":203.99998474121094,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00037","coefficient":-1},{"bigg_id":"cpd00061","coefficient":-1},{"bigg_id":"cpd00009","coefficient":1},{"bigg_id":"cpd02820","coefficient":1}],"segments":{"27":{"from_node_id":"32","to_node_id":"33","b1":null,"b2":null},"28":{"from_node_id":"34","to_node_id":"33","b1":null,"b2":null},"29":{"from_node_id":"32","to_node_id":"31","b1":{"x":-305,"y":131.99998474121094},"b2":{"x":-305,"y":77.74998474121094}},"30":{"from_node_id":"32","to_node_id":"35","b1":{"x":-305,"y":127.99998474121094},"b2":{"x":-323,"y":122.74998474121094}},"31":{"from_node_id":"34","to_node_id":"36","b1":{"x":-305,"y":259.99998474121094},"b2":{"x":-323,"y":265.24998474121094}},"32":{"from_node_id":"34","to_node_id":"37","b1":{"x":-305,"y":255.99998474121094},"b2":{"x":-305,"y":310.24998474121094}}}},"5":{"name":"rxn01485","bigg_id":"rxn01485","reversibility":false,"label_x":-288.06358337402344,"label_y":-881.0799713134766,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd03671","coefficient":-1},{"bigg_id":"cpd00288","coefficient":1}],"segments":{"33":{"from_node_id":"39","to_node_id":"40","b1":null,"b2":null},"34":{"from_node_id":"41","to_node_id":"40","b1":null,"b2":null},"35":{"from_node_id":"39","to_node_id":"27","b1":{"x":-303.8578542832081,"y":-829.0850591535029},"b2":{"x":-303.22026076233817,"y":-818.8148528104018}},"36":{"from_node_id":"41","to_node_id":"42","b1":{"x":-302.26931246483866,"y":-953.0748834734504},"b2":{"x":-301.5743254193022,"y":-1007.3204316134265}}}},"7":{"name":"rxn15064","bigg_id":"rxn15064","reversibility":false,"label_x":-130.4826784133911,"label_y":-1192.4988059997559,"gene_reaction_rule":"","genes":[],"metabolites":[{"bigg_id":"cpd00053","coefficient":-1},{"bigg_id":"cpd19035","coefficient":-1},{"bigg_id":"cpd00023","coefficient":1},{"bigg_id":"cpd00288","coefficient":1}],"segments":{"43":{"from_node_id":"51","to_node_id":"52","b1":null,"b2":null},"44":{"from_node_id":"53","to_node_id":"52","b1":null,"b2":null},"45":{"from_node_id":"51","to_node_id":"50","b1":{"x":-145.4826784133911,"y":-1264.4988059997559},"b2":{"x":-117.6723165512085,"y":-1268.4253196716309}},"46":{"from_node_id":"51","to_node_id":"54","b1":{"x":-145.4826784133911,"y":-1268.4988059997559},"b2":{"x":-146.26674365997314,"y":-1298.9105491638184}},"47":{"from_node_id":"53","to_node_id":"55","b1":{"x":-145.4826784133911,"y":-1136.4988059997559},"b2":{"x":-125.07787799835205,"y":-1140.5190696716309}},"48":{"from_node_id":"53","to_node_id":"42","b1":{"x":-146.80698108673096,"y":-1067.6620140075684},"b2":{"x":-148.13128185272217,"y":-1069.0329856872559}}}}},"nodes":{"0":{"node_type":"metabolite","x":-392.6024169921875,"y":-245.10089111328125,"bigg_id":"cpd00062","name":"UTP","label_x":-367.6024169921875,"label_y":-245.10089111328125,"node_is_primary":false},"1":{"node_type":"multimarker","x":-304.6517333984375,"y":-175.3863983154297},"2":{"node_type":"midmarker","x":-304.6517333984375,"y":-155.3863983154297},"3":{"node_type":"multimarker","x":-304.6517333984375,"y":-135.3863983154297},"4":{"node_type":"metabolite","x":-307.3617248535156,"y":-331.0134735107422,"bigg_id":"cpd02611","name":"N-Acetyl-D-glucosamine1-phosphate","label_x":-477.9730987548828,"label_y":-321.0022430419922,"node_is_primary":true},"5":{"node_type":"metabolite","x":-384.6517333984375,"y":-50.38639831542969,"bigg_id":"cpd00012","name":"PPi","label_x":-456.6517333984375,"label_y":-20.386398315429688,"node_is_primary":false},"7":{"node_type":"metabolite","x":-528.638988357472,"y":-1114.5676341508704,"bigg_id":"cpd00022","name":"Acetyl-CoA","label_x":-604.2860220488783,"label_y":-1146.350959346183,"node_is_primary":false},"8":{"node_type":"multimarker","x":-443.89518592825254,"y":-1182.1581446436237},"9":{"node_type":"midmarker","x":-443.72580384817286,"y":-1202.1574273735353},"10":{"node_type":"multimarker","x":-443.55642176809226,"y":-1222.1567101034475},"12":{"node_type":"metabolite","x":-533.6206637724674,"y":-1285.7740760649517,"bigg_id":"cpd00010","name":"CoA","label_x":-573.5115329130924,"label_y":-1250.0178504790142,"node_is_primary":false},"13":{"node_type":"metabolite","x":-358.6200863180999,"y":-1290.6152476430836,"bigg_id":"cpd00067","name":"H+","label_x":-343.6200863180999,"label_y":-1290.6152476430836,"node_is_primary":false},"14":{"node_type":"metabolite","x":-443.62191822880584,"y":-1362.0967388540003,"bigg_id":"cpd00293","name":"N-Acetyl-D-glucosamine 6-phosphate","label_x":-629.5119633948214,"label_y":-1396.9876079946253,"node_is_primary":true},"15":{"node_type":"metabolite","x":-224.42298030744348,"y":644.836154927465,"bigg_id":"cpd00006","name":"NADP","label_x":-199.42298030744348,"label_y":644.836154927465,"node_is_primary":false},"16":{"node_type":"multimarker","x":-300.0156166075416,"y":577.8294206879995},"17":{"node_type":"midmarker","x":-300.48439025878906,"y":557.8349151611328},"18":{"node_type":"multimarker","x":-300.95316391003655,"y":537.8404096342662},"19":{"node_type":"metabolite","x":-299.3330567264138,"y":706.2388969354731,"bigg_id":"cpd00773","name":"UDP-MurNAc","label_x":-366.0027039432108,"label_y":748.2321830682856,"node_is_primary":true},"20":{"node_type":"metabolite","x":-215.3028955470487,"y":456.17516709401934,"bigg_id":"cpd00005","name":"NADPH","label_x":-190.3028955470487,"label_y":456.17516709401934,"node_is_primary":false},"21":{"node_type":"metabolite","x":-382.92347403530493,"y":454.738855750074,"bigg_id":"cpd00067","name":"H+","label_x":-367.92347403530493,"label_y":454.738855750074,"node_is_primary":false},"23":{"node_type":"metabolite","x":-412.8961601257324,"y":-625.9528045654297,"bigg_id":"cpd00022","name":"Acetyl-CoA","label_x":-469.1838798522949,"label_y":-641.9438934326172,"node_is_primary":false},"24":{"node_type":"multimarker","x":-306.2892875671387,"y":-561.5687103271484},"25":{"node_type":"midmarker","x":-306.2892875671387,"y":-541.5687103271484},"26":{"node_type":"multimarker","x":-306.2892875671387,"y":-521.5687103271484},"27":{"node_type":"metabolite","x":-305.0015525817871,"y":-710.5328216552734,"bigg_id":"cpd03671","name":"D-Glucosamine1-phosphate","label_x":-442.2982597351074,"label_y":-703.1867523193359,"node_is_primary":true},"28":{"node_type":"metabolite","x":-418.22651290893555,"y":-459.8496856689453,"bigg_id":"cpd00010","name":"CoA","label_x":-443.8647880554199,"label_y":-477.1732940673828,"node_is_primary":false},"29":{"node_type":"metabolite","x":-226.28928756713867,"y":-436.56871032714844,"bigg_id":"cpd00067","name":"H+","label_x":-211.28928756713867,"label_y":-436.56871032714844,"node_is_primary":false},"31":{"node_type":"metabolite","x":-305,"y":18.999984741210938,"bigg_id":"cpd00037","name":"UDP-N-acetylglucosamine","label_x":-280,"label_y":18.999984741210938,"node_is_primary":true},"32":{"node_type":"multimarker","x":-305,"y":173.99998474121094},"33":{"node_type":"midmarker","x":-305,"y":193.99998474121094},"34":{"node_type":"multimarker","x":-305,"y":213.99998474121094},"35":{"node_type":"metabolite","x":-385,"y":88.99998474121094,"bigg_id":"cpd00061","name":"Phosphoenolpyruvate","label_x":-457,"label_y":118.99998474121094,"node_is_primary":false},"36":{"node_type":"metabolite","x":-385,"y":298.99998474121094,"bigg_id":"cpd00009","name":"Phosphate","label_x":-457,"label_y":328.99998474121094,"node_is_primary":false},"37":{"node_type":"metabolite","x":-305,"y":368.99998474121094,"bigg_id":"cpd02820","name":"UDP-N-acetylglucosamine enolpyruvate","label_x":-280,"label_y":368.99998474121094,"node_is_primary":true},"39":{"node_type":"multimarker","x":-303.31979979634116,"y":-871.0816125521947},"40":{"node_type":"midmarker","x":-303.06358337402344,"y":-891.0799713134766},"41":{"node_type":"multimarker","x":-302.8073669517057,"y":-911.0783300747582},"42":{"node_type":"metabolite","x":-300.8216896787453,"y":-1066.0656104746922,"bigg_id":"cpd00288","name":"D-Glucosamine phosphate","label_x":-416.197834087925,"label_y":-1095.2003760996922,"node_is_primary":true},"50":{"node_type":"metabolite","x":-48.80854892730713,"y":-1288.7705345153809,"bigg_id":"cpd00053","name":"L-Glutamine","label_x":-23.80854892730713,"label_y":-1288.7705345153809,"node_is_primary":false},"51":{"node_type":"multimarker","x":-145.4826784133911,"y":-1222.4988059997559},"52":{"node_type":"midmarker","x":-145.4826784133911,"y":-1202.4988059997559},"53":{"node_type":"multimarker","x":-145.4826784133911,"y":-1182.4988059997559},"54":{"node_type":"metabolite","x":-147.34879779815674,"y":-1367.0924339294434,"bigg_id":"cpd19035","name":"beta-D-Fructose 6-phosphate","label_x":-235.2404375076294,"label_y":-1395.3617210388184,"node_is_primary":true},"55":{"node_type":"metabolite","x":-42.7288293838501,"y":-1122.6606712341309,"bigg_id":"cpd00023","name":"L-Glutamate","label_x":-114.7288293838501,"label_y":-1092.6606712341309,"node_is_primary":false}},"text_labels":{"0":{"x":-858.7408981323242,"y":-1487.5926513671875,"text":"UDP-N-Acetylmuramate from Fructose-6-Phosphate"}},"canvas":{"x":-957.4156494140625,"y":-1657.0641479492188,"width":1475.9495544433594,"height":2596.5421142578125}}]